In [ ]:
import pandas as pd
import os

# Read the Excel file with multiple sheets (summary 1-10)
filename = r"C:\Users\OEM\Downloads\report-generator\summary 1-10.xls"

if os.path.exists(filename):
    # Read all sheets from the Excel file
    excel_file = pd.ExcelFile(filename)
    
    print(f"Available sheets: {excel_file.sheet_names}\n")
    
    # Read all sheets into a dictionary
    dataframes = {}
    for sheet_name in excel_file.sheet_names:
        # Read the sheet - looking for header row around row 9 based on screenshot
        df = pd.read_excel(excel_file, sheet_name=sheet_name, header=9)
        
        # Remove completely empty columns
        df = df.dropna(axis=1, how='all')
        
        # Keep unnamed columns that contain data (e.g., item numbers) and drop only empty ones
        unnamed_cols = [col for col in df.columns if isinstance(col, str) and col.startswith('Unnamed')]
        cols_to_drop = [col for col in unnamed_cols if df[col].isna().all()]
        if cols_to_drop:
            df = df.drop(columns=cols_to_drop)
        
        # Rename the item number column so downstream logic can find it reliably
        if 'Unnamed: 0' in df.columns and 'Item Number' not in df.columns:
            df = df.rename(columns={'Unnamed: 0': 'Item Number'})
        
        dataframes[sheet_name] = df
        print(f"Loaded sheet '{sheet_name}': {len(df)} rows, {len(df.columns)} columns")
        print(f"  Columns: {list(df.columns)[:10]}")  # Show first 10 columns
    
    print(f"\nTotal sheets loaded: {len(dataframes)}")
else:
    print(f"File not found: {filename}")
    dataframes = {}

# Access individual sheets like: dataframes['Sheet1'], dataframes['summary 1'], etc."}

In [ ]:
dataframes

In [ ]:
# Split data by item numbers while preserving column names
def split_by_items(df):
    """
    Split dataframe by item numbers (e.g., AMO0002, YUH0019, etc.)
    Returns a dictionary with item numbers as keys and their data as values
    Preserves original column names (including the item number column)
    """
    items_dict = {}
    current_item = None
    current_data = []
    column_names = df.columns  # Preserve original column names
    
    def finalize_item(current_rows, item_key):
        if not current_rows:
            return None
        item_df = pd.DataFrame(current_rows, columns=column_names)
        item_df = item_df.dropna(axis=1, how='all')
        if 'Item Number' in item_df.columns:
            item_df['Item Number'] = item_df['Item Number'].ffill().bfill().fillna(item_key)
        else:
            item_df.insert(0, 'Item Number', item_key)
        return item_df
    
    for idx, row in df.iterrows():
        # Check if this row contains an item number
        # Item numbers typically appear in the first column
        first_col_value = str(row.iloc[0]) if pd.notna(row.iloc[0]) else ''
        first_col_clean = first_col_value.strip()
        
        # Check if this looks like an item code (has letters AND numbers, not just "Item Total:")
        is_item_code = (first_col_clean and 
                       any(c.isalpha() for c in first_col_clean) and 
                       any(c.isdigit() for c in first_col_clean) and
                       'Item Total' not in first_col_value)
        
        if is_item_code:
            # Save previous item if exists
            if current_item and current_data:
                item_df = finalize_item(current_data, current_item)
                if item_df is not None:
                    items_dict[current_item] = item_df
            
            # Start new item
            current_item = first_col_clean
            current_data = [row.values]
        elif current_item:
            # Add row to current item
            current_data.append(row.values)
            
            # Check if this is the "Item Total:" row (end of current item)
            if 'Item Total:' in str(row.values):
                item_df = finalize_item(current_data, current_item)
                if item_df is not None:
                    items_dict[current_item] = item_df
                current_item = None
                current_data = []
    
    # Add last item if exists
    if current_item and current_data:
        item_df = finalize_item(current_data, current_item)
        if item_df is not None:
            items_dict[current_item] = item_df
    
    return items_dict

# Process each sheet
all_items = {}

for sheet_name, df in dataframes.items():
    print(f"\n--- Processing sheet: {sheet_name} ---")
    items = split_by_items(df)
    
    for item_num, item_df in items.items():
        print(f"  Item {item_num}: {len(item_df)} rows, {len(item_df.columns)} columns")
        # Store with sheet name prefix to avoid conflicts
        key = f"{sheet_name}_{item_num}"
        all_items[key] = item_df

print(f"\n\nTotal items extracted: {len(all_items)}")
print(f"Item keys: {list(all_items.keys())}")

In [ ]:
# Let's debug: check the first column values to see all potential item numbers
for sheet_name, df in dataframes.items():
    print(f"\n=== Sheet: {sheet_name} ===")
    print(f"Total rows: {len(df)}")
    print(f"\nFirst column unique values (first 30):")
    
    first_col = df.iloc[:, 0]
    unique_values = first_col.dropna().unique()
    
    for i, val in enumerate(unique_values[:30]):
        print(f"  {i+1}. '{val}'")
    
    print(f"\nTotal unique values in first column: {len(unique_values)}")
    
    # Check how many look like item codes (contain letters and numbers)
    potential_items = [v for v in unique_values if pd.notna(v) and str(v).strip() and any(c.isalpha() for c in str(v)) and any(c.isdigit() for c in str(v))]
    print(f"Potential item codes found: {len(potential_items)}")
    print(f"Examples: {potential_items[:10]}")

In [ ]:
# Method 1: Get the first item using list() to convert keys
first_key = list(all_items.keys())[0]
first_item = all_items[first_key]

print(f"First item key: {first_key}")
print(f"\nFirst item dataframe:")
print(first_item)

# Method 2: Alternative way using next() and iter()
# first_key = next(iter(all_items))
# first_item = all_items[first_key]

## Calculate Sales Metrics

Calculate MTD and YTD metrics based on the items data:
- **MTD Gross Sales**: Month-to-Date Sales Amount
- **MTD GP%**: Month-to-Date Gross Profit Percentage
- **YTD Gross Sales**: Year-to-Date Sales Amount
- **YTD GP%**: Year-to-Date Gross Profit Percentage

In [ ]:
import datetime

def calculate_sales_metrics(all_items, target_month=10, target_year=2025):
    """
    Calculate MTD and YTD sales metrics from items data
    
    Parameters:
    - all_items: Dictionary of item dataframes
    - target_month: Month to calculate MTD for (e.g., 10 for October)
    - target_year: Year to calculate forl    
    Note: MTD Gross Sales = sum of sales for target_month
          YTD Gross Sales = sum of sales for all months from 1 to target_month
    """
    
    # Initialize accumulators
    mtd_sales = 0
    mtd_cost = 0
    ytd_sales = 0
    ytd_cost = 0
    
    # Debug counters
    items_processed = 0
    items_with_data = 0
    
    # Process each item
    for item_key, item_df in all_items.items():
        items_processed += 1
        
        # Skip if dataframe is empty
        if item_df.empty:
            continue
        
        # Make a copy to work with
        df = item_df.copy()
        
        # Find the correct column names (handling potential variations)
        year_col = None
        period_col = None
        sales_col = None
        cost_col = None
        
        for col in df.columns:
            col_lower = str(col).strip().lower()
            if 'year' in col_lower and not year_col:
                year_col = col
            elif 'period' in col_lower and not period_col:
                period_col = col
            elif 'sales amount' in col_lower and not sales_col:
                sales_col = col
            elif 'cost of sales' in col_lower and not cost_col:
                cost_col = col
        
        # Skip if we don't have the required columns
        if not year_col or not period_col or not sales_col:
            continue
        
        # Convert to numeric
        df[year_col] = pd.to_numeric(df[year_col], errors='coerce')
        df[period_col] = pd.to_numeric(df[period_col], errors='coerce')
        df[sales_col] = pd.to_numeric(df[sales_col], errors='coerce').fillna(0)
        if cost_col:
            df[cost_col] = pd.to_numeric(df[cost_col], errors='coerce').fillna(0)
        
        # Filter for target year
        year_data = df[df[year_col] == target_year].copy()
        
        if not year_data.empty:
            items_with_data += 1
            
            # Calculate MTD (for specific month only)
            mtd_data = year_data[year_data[period_col] == target_month]
            mtd_sales += mtd_data[sales_col].sum()
            if cost_col:
                mtd_cost += mtd_data[cost_col].sum()
            
            # Calculate YTD (sum of ALL months from 1 to target_month)
            ytd_data = year_data[year_data[period_col] <= target_month]
            ytd_sales += ytd_data[sales_col].sum()
            if cost_col:
                ytd_cost += ytd_data[cost_col].sum()
    
    # Calculate GP% (Gross Profit Percentage)
    mtd_gp = ((mtd_sales - mtd_cost) / mtd_sales * 100) if mtd_sales > 0 else 0
    ytd_gp = ((ytd_sales - ytd_cost) / ytd_sales * 100) if ytd_sales > 0 else 0
    
    print(f"Debug: Processed {items_processed} items, {items_with_data} had data for year {target_year}")
    
    return {
        'MTD Gross Sales': mtd_sales,
        'MTD GP%': mtd_gp,
        'YTD Gross Sales': ytd_sales,
        'YTD GP%': ytd_gp,
        'MTD Cost': mtd_cost,
        'YTD Cost': ytd_cost
    }

# Calculate for October 2025
results_2025 = calculate_sales_metrics(all_items, target_month=10, target_year=2025)

print("\n=== 2025 Results (October) ===")
print(f"MTD Gross Sales: ${results_2025['MTD Gross Sales']:,.2f}")
print(f"MTD GP%: {results_2025['MTD GP%']:.2f}%")
print(f"YTD Gross Sales: ${results_2025['YTD Gross Sales']:,.2f}")
print(f"YTD GP%: {results_2025['YTD GP%']:.2f}%")

# Calculate for 2024 for comparison
results_2024 = calculate_sales_metrics(all_items, target_month=10, target_year=2024)

print("\n=== 2024 Results (October) ===")
print(f"MTD Gross Sales: ${results_2024['MTD Gross Sales']:,.2f}")
print(f"MTD GP%: {results_2024['MTD GP%']:.2f}%")
print(f"YTD Gross Sales: ${results_2024['YTD Gross Sales']:,.2f}")
print(f"YTD GP%: {results_2024['YTD GP%']:.2f}%")

# Calculate % Achieved (2025 vs 2024)
print("\n=== % Achieved (2025 vs 2024) ===")
mtd_achieved = (results_2025['MTD Gross Sales'] / results_2024['MTD Gross Sales'] * 100) if results_2024['MTD Gross Sales'] > 0 else 0
ytd_achieved = (results_2025['YTD Gross Sales'] / results_2024['YTD Gross Sales'] * 100) if results_2024['YTD Gross Sales'] > 0 else 0
print(f"MTD Achieved: {mtd_achieved:.0f}%")
print(f"YTD Achieved: {ytd_achieved:.0f}%")

In [ ]:
# Create a summary report matching the image format
# Calculate %Achieved for GP% - show 0% if no 2024 data to compare against
mtd_gp_achieved = "0%" if results_2024['MTD GP%'] == 0 else f"{(results_2025['MTD GP%'] / results_2024['MTD GP%'] * 100):.0f}%"
ytd_gp_achieved = "0%" if results_2024['YTD GP%'] == 0 else f"{(results_2025['YTD GP%'] / results_2024['YTD GP%'] * 100):.0f}%"

# Budget data - to be loaded from another sheet
budget_mtd_sales = None  # To be loaded
budget_mtd_gp = None     # To be loaded
budget_ytd_sales = None  # To be loaded
budget_ytd_gp = None     # To be loaded

# Calculate % Achieved vs Budget (2025 actual / 2025 budget * 100)
mtd_budget_achieved = "0%" if budget_mtd_sales is None else f"{(results_2025['MTD Gross Sales'] / budget_mtd_sales * 100):.0f}%"
mtd_gp_budget_achieved = "0%" if budget_mtd_gp is None else f"{(results_2025['MTD GP%'] / budget_mtd_gp * 100):.0f}%"
ytd_budget_achieved = "0%" if budget_ytd_sales is None else f"{(results_2025['YTD Gross Sales'] / budget_ytd_sales * 100):.0f}%"
ytd_gp_budget_achieved = "0%" if budget_ytd_gp is None else f"{(results_2025['YTD GP%'] / budget_ytd_gp * 100):.0f}%"

summary_data = {
    'Date': ['2025', '2024', '%Achieved', '2025 Budget', '% Achieved'],
    'MTD Gross Sales': [
        f"$ {results_2025['MTD Gross Sales']:,.2f}",
        f"$ {results_2024['MTD Gross Sales']:,.2f}",
        f"{mtd_achieved:.0f}%",  # 2025 MTD / 2024 MTD * 100
        "" if budget_mtd_sales is None else f"$ {budget_mtd_sales:,.2f}",
        mtd_budget_achieved  # 2025 MTD / Budget MTD * 100
    ],
    'MTD GP%': [
        f"{results_2025['MTD GP%']:.2f}%",
        f"{results_2024['MTD GP%']:.2f}%",
        mtd_gp_achieved,  # 2025 GP% / 2024 GP% * 100
        "" if budget_mtd_gp is None else f"{budget_mtd_gp:.2f}%",
        mtd_gp_budget_achieved  # 2025 GP% / Budget GP% * 100
    ],
    'YTD Gross Sales': [
        f"$ {results_2025['YTD Gross Sales']:,.2f}",
        f"$ {results_2024['YTD Gross Sales']:,.2f}",
        f"{ytd_achieved:.0f}%",  # 2025 YTD / 2024 YTD * 100
        "" if budget_ytd_sales is None else f"$ {budget_ytd_sales:,.2f}",
        ytd_budget_achieved  # 2025 YTD / Budget YTD * 100
    ],
    'YTD GP%': [
        f"{results_2025['YTD GP%']:.2f}%",
        f"{results_2024['YTD GP%']:.2f}%",
        ytd_gp_achieved,  # 2025 GP% / 2024 GP% * 100
        "" if budget_ytd_gp is None else f"{budget_ytd_gp:.2f}%",
        ytd_gp_budget_achieved  # 2025 GP% / Budget GP% * 100
    ]
}

summary_df = pd.DataFrame(summary_data)
print("\n=== Sales Summary Report ===")
print(summary_df.to_string(index=False))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Create a professional visualization of the sales summary
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Sales Performance Dashboard - October 2025', fontsize=16, fontweight='bold')

# Data for plotting
categories = ['2025', '2024']
mtd_sales_values = [results_2025['MTD Gross Sales'], results_2024['MTD Gross Sales']]
ytd_sales_values = [results_2025['YTD Gross Sales'], results_2024['YTD Gross Sales']]
mtd_gp_values = [results_2025['MTD GP%'], results_2024['MTD GP%']]
ytd_gp_values = [results_2025['YTD GP%'], results_2024['YTD GP%']]

# Color scheme
colors_2025_2024 = ['#2E86AB', '#A23B72']  # Blue for 2025, Pink for 2024

# 1. MTD Gross Sales Bar Chart
ax1 = axes[0, 0]
bars1 = ax1.bar(categories, mtd_sales_values, color=colors_2025_2024, alpha=0.8, edgecolor='black', linewidth=1.5)
ax1.set_title('MTD Gross Sales', fontsize=12, fontweight='bold')
ax1.set_ylabel('Sales Amount ($)', fontsize=10)
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
# Add value labels on bars
for bar in bars1:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
            f'${height:,.0f}',
            ha='center', va='bottom', fontsize=9, fontweight='bold')
# Add % achieved annotation
if results_2024['MTD Gross Sales'] > 0:
    ax1.text(0.5, max(mtd_sales_values) * 0.95, 
            f'% Achieved: {mtd_achieved:.0f}%',
            ha='center', fontsize=10, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
ax1.grid(axis='y', alpha=0.3, linestyle='--')

# 2. YTD Gross Sales Bar Chart
ax2 = axes[0, 1]
bars2 = ax2.bar(categories, ytd_sales_values, color=colors_2025_2024, alpha=0.8, edgecolor='black', linewidth=1.5)
ax2.set_title('YTD Gross Sales', fontsize=12, fontweight='bold')
ax2.set_ylabel('Sales Amount ($)', fontsize=10)
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
# Add value labels on bars
for bar in bars2:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'${height:,.0f}',
            ha='center', va='bottom', fontsize=9, fontweight='bold')
# Add % achieved annotation
if results_2024['YTD Gross Sales'] > 0:
    ax2.text(0.5, max(ytd_sales_values) * 0.95,
            f'% Achieved: {ytd_achieved:.0f}%',
            ha='center', fontsize=10, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
ax2.grid(axis='y', alpha=0.3, linestyle='--')

# 3. MTD GP% Bar Chart
ax3 = axes[1, 0]
bars3 = ax3.bar(categories, mtd_gp_values, color=colors_2025_2024, alpha=0.8, edgecolor='black', linewidth=1.5)
ax3.set_title('MTD Gross Profit %', fontsize=12, fontweight='bold')
ax3.set_ylabel('GP %', fontsize=10)
ax3.set_ylim(0, max(mtd_gp_values) * 1.2)
# Add value labels on bars
for bar in bars3:
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.2f}%',
            ha='center', va='bottom', fontsize=9, fontweight='bold')
# Add % achieved annotation if applicable
if results_2024['MTD GP%'] > 0:
    gp_achieved = (results_2025['MTD GP%'] / results_2024['MTD GP%'] * 100)
    ax3.text(0.5, max(mtd_gp_values) * 0.95,
            f'% Achieved: {gp_achieved:.0f}%',
            ha='center', fontsize=10, bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))
ax3.grid(axis='y', alpha=0.3, linestyle='--')

# 4. YTD GP% Bar Chart
ax4 = axes[1, 1]
bars4 = ax4.bar(categories, ytd_gp_values, color=colors_2025_2024, alpha=0.8, edgecolor='black', linewidth=1.5)
ax4.set_title('YTD Gross Profit %', fontsize=12, fontweight='bold')
ax4.set_ylabel('GP %', fontsize=10)
ax4.set_ylim(0, max(ytd_gp_values) * 1.2)
# Add value labels on bars
for bar in bars4:
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.2f}%',
            ha='center', va='bottom', fontsize=9, fontweight='bold')
# Add % achieved annotation if applicable
if results_2024['YTD GP%'] > 0:
    gp_achieved = (results_2025['YTD GP%'] / results_2024['YTD GP%'] * 100)
    ax4.text(0.5, max(ytd_gp_values) * 0.95,
            f'% Achieved: {gp_achieved:.0f}%',
            ha='center', fontsize=10, bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))
ax4.grid(axis='y', alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()

# Create a second chart showing year-over-year comparison
fig2, ax = plt.subplots(1, 1, figsize=(12, 6))

# Prepare data for grouped bar chart
x = np.arange(4)  # 4 metrics
width = 0.35  # width of bars

metrics = ['MTD Gross Sales\n(in thousands)', 'MTD GP%', 'YTD Gross Sales\n(in thousands)', 'YTD GP%']
values_2025 = [
    results_2025['MTD Gross Sales'] / 1000,  # Convert to thousands
    results_2025['MTD GP%'],
    results_2025['YTD Gross Sales'] / 1000,  # Convert to thousands
    results_2025['YTD GP%']
]
values_2024 = [
    results_2024['MTD Gross Sales'] / 1000,  # Convert to thousands
    results_2024['MTD GP%'],
    results_2024['YTD Gross Sales'] / 1000,  # Convert to thousands
    results_2024['YTD GP%']
]

bars1 = ax.bar(x - width/2, values_2025, width, label='2025', color='#2E86AB', alpha=0.8, edgecolor='black', linewidth=1.5)
bars2 = ax.bar(x + width/2, values_2024, width, label='2024', color='#A23B72', alpha=0.8, edgecolor='black', linewidth=1.5)

ax.set_title('Year-over-Year Performance Comparison', fontsize=14, fontweight='bold')
ax.set_ylabel('Value', fontsize=11)
ax.set_xticks(x)
ax.set_xticklabels(metrics, fontsize=10)
ax.legend(fontsize=11, loc='upper left')
ax.grid(axis='y', alpha=0.3, linestyle='--')

# Add value labels on bars
for bars in [bars1, bars2]:
    for i, bar in enumerate(bars):
        height = bar.get_height()
        # Format based on metric type (sales vs GP%)
        if i in [0, 2]:  # Sales metrics
            label = f'${height:,.0f}K'
        else:  # GP% metrics
            label = f'{height:.2f}%'
        ax.text(bar.get_x() + bar.get_width()/2., height,
                label, ha='center', va='bottom', fontsize=8, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✓ Charts generated successfully!")

In [ ]:
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.drawing.image import Image as XLImage
import io

# Create a new workbook
wb = Workbook()
ws = wb.active
ws.title = "Sales Summary Report"

# Write the title
ws['A1'] = "Sales Summary Report - October 2025"
ws['A1'].font = Font(size=14, bold=True)
ws['A1'].alignment = Alignment(horizontal='center')
ws.merge_cells('A1:E1')

# Define styling
header_fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True)
row_2025_fill = PatternFill(start_color="E7E6E6", end_color="E7E6E6", fill_type="solid")
row_achieved_fill = PatternFill(start_color="FFF2CC", end_color="FFF2CC", fill_type="solid")
row_budget_fill = PatternFill(start_color="D9E1F2", end_color="D9E1F2", fill_type="solid")

border_style = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

# Write headers (row 3)
headers = ['Date', 'MTD Gross Sales', 'MTD GP%', 'YTD Gross Sales', 'YTD GP%']
for col_num, header in enumerate(headers, 1):
    cell = ws.cell(row=3, column=col_num, value=header)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = Alignment(horizontal='center', vertical='center')
    cell.border = border_style

# Prepare data rows
data_rows = [
    ['2025', 
     results_2025['MTD Gross Sales'], 
     results_2025['MTD GP%'] / 100,  # Convert to decimal for percentage format
     results_2025['YTD Gross Sales'], 
     results_2025['YTD GP%'] / 100],
    ['2024', 
     results_2024['MTD Gross Sales'], 
     results_2024['MTD GP%'] / 100, 
     results_2024['YTD Gross Sales'], 
     results_2024['YTD GP%'] / 100],
    ['%Achieved', 
     mtd_achieved / 100 if results_2024['MTD Gross Sales'] > 0 else 0,
     float(mtd_gp_achieved.rstrip('%')) / 100 if mtd_gp_achieved != "0%" else 0,
     ytd_achieved / 100 if results_2024['YTD Gross Sales'] > 0 else 0,
     float(ytd_gp_achieved.rstrip('%')) / 100 if ytd_gp_achieved != "0%" else 0],
    ['2025 Budget', '', '', '', ''],
    ['% Achieved', 0, 0, 0, 0]  # Budget comparison - currently 0%
]

# Write data rows with formatting
for row_idx, row_data in enumerate(data_rows, start=4):
    for col_idx, value in enumerate(row_data, start=1):
        cell = ws.cell(row=row_idx, column=col_idx, value=value)
        cell.border = border_style
        cell.alignment = Alignment(horizontal='right' if col_idx > 1 else 'left', vertical='center')
        
        # Apply row-specific fills
        if row_data[0] == '2025':
            cell.fill = row_2025_fill
        elif '%Achieved' in row_data[0]:
            cell.fill = row_achieved_fill
        elif 'Budget' in row_data[0]:
            cell.fill = row_budget_fill
        
        # Format numbers
        if col_idx > 1 and value != '':
            if col_idx in [2, 4]:  # Sales columns
                cell.number_format = '$#,##0.00'
            elif col_idx in [3, 5]:  # GP% columns
                cell.number_format = '0.00%'

# Set column widths
ws.column_dimensions['A'].width = 15
ws.column_dimensions['B'].width = 18
ws.column_dimensions['C'].width = 12
ws.column_dimensions['D'].width = 18
ws.column_dimensions['E'].width = 12

# Add row heights
ws.row_dimensions[1].height = 25
ws.row_dimensions[3].height = 20

# Save charts as images and add to Excel
# Chart 1: Dashboard
fig_dashboard, axes = plt.subplots(2, 2, figsize=(12, 9))
fig_dashboard.suptitle('Sales Performance Dashboard - October 2025', fontsize=14, fontweight='bold')

categories = ['2025', '2024']
mtd_sales_values = [results_2025['MTD Gross Sales'], results_2024['MTD Gross Sales']]
ytd_sales_values = [results_2025['YTD Gross Sales'], results_2024['YTD Gross Sales']]
mtd_gp_values = [results_2025['MTD GP%'], results_2024['MTD GP%']]
ytd_gp_values = [results_2025['YTD GP%'], results_2024['YTD GP%']]
colors_2025_2024 = ['#2E86AB', '#A23B72']

# MTD Gross Sales
ax1 = axes[0, 0]
bars1 = ax1.bar(categories, mtd_sales_values, color=colors_2025_2024, alpha=0.8, edgecolor='black', linewidth=1.5)
ax1.set_title('MTD Gross Sales', fontsize=11, fontweight='bold')
ax1.set_ylabel('Sales Amount ($)', fontsize=9)
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
for bar in bars1:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height, f'${height:,.0f}',
            ha='center', va='bottom', fontsize=8, fontweight='bold')
if results_2024['MTD Gross Sales'] > 0:
    ax1.text(0.5, max(mtd_sales_values) * 0.92, f'% Achieved: {mtd_achieved:.0f}%',
            ha='center', fontsize=9, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
ax1.grid(axis='y', alpha=0.3, linestyle='--')

# YTD Gross Sales
ax2 = axes[0, 1]
bars2 = ax2.bar(categories, ytd_sales_values, color=colors_2025_2024, alpha=0.8, edgecolor='black', linewidth=1.5)
ax2.set_title('YTD Gross Sales', fontsize=11, fontweight='bold')
ax2.set_ylabel('Sales Amount ($)', fontsize=9)
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
for bar in bars2:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height, f'${height:,.0f}',
            ha='center', va='bottom', fontsize=8, fontweight='bold')
if results_2024['YTD Gross Sales'] > 0:
    ax2.text(0.5, max(ytd_sales_values) * 0.92, f'% Achieved: {ytd_achieved:.0f}%',
            ha='center', fontsize=9, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
ax2.grid(axis='y', alpha=0.3, linestyle='--')

# MTD GP%
ax3 = axes[1, 0]
bars3 = ax3.bar(categories, mtd_gp_values, color=colors_2025_2024, alpha=0.8, edgecolor='black', linewidth=1.5)
ax3.set_title('MTD Gross Profit %', fontsize=11, fontweight='bold')
ax3.set_ylabel('GP %', fontsize=9)
ax3.set_ylim(0, max(mtd_gp_values) * 1.2)
for bar in bars3:
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height, f'{height:.2f}%',
            ha='center', va='bottom', fontsize=8, fontweight='bold')
if results_2024['MTD GP%'] > 0:
    gp_achieved = (results_2025['MTD GP%'] / results_2024['MTD GP%'] * 100)
    ax3.text(0.5, max(mtd_gp_values) * 0.92, f'% Achieved: {gp_achieved:.0f}%',
            ha='center', fontsize=9, bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))
ax3.grid(axis='y', alpha=0.3, linestyle='--')

# YTD GP%
ax4 = axes[1, 1]
bars4 = ax4.bar(categories, ytd_gp_values, color=colors_2025_2024, alpha=0.8, edgecolor='black', linewidth=1.5)
ax4.set_title('YTD Gross Profit %', fontsize=11, fontweight='bold')
ax4.set_ylabel('GP %', fontsize=9)
ax4.set_ylim(0, max(ytd_gp_values) * 1.2)
for bar in bars4:
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height, f'{height:.2f}%',
            ha='center', va='bottom', fontsize=8, fontweight='bold')
if results_2024['YTD GP%'] > 0:
    gp_achieved = (results_2025['YTD GP%'] / results_2024['YTD GP%'] * 100)
    ax4.text(0.5, max(ytd_gp_values) * 0.92, f'% Achieved: {gp_achieved:.0f}%',
            ha='center', fontsize=9, bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))
ax4.grid(axis='y', alpha=0.3, linestyle='--')

plt.tight_layout()

# Save dashboard chart to bytes
img_buffer1 = io.BytesIO()
fig_dashboard.savefig(img_buffer1, format='png', dpi=150, bbox_inches='tight')
img_buffer1.seek(0)
plt.close(fig_dashboard)

# Create a new sheet for the dashboard chart
ws_chart1 = wb.create_sheet("Dashboard Chart")
img1 = XLImage(img_buffer1)
ws_chart1.add_image(img1, 'A1')

# Chart 2: Comparison chart
fig_comparison, ax = plt.subplots(1, 1, figsize=(11, 6))
x = np.arange(4)
width = 0.35
metrics = ['MTD Gross Sales\n(in thousands)', 'MTD GP%', 'YTD Gross Sales\n(in thousands)', 'YTD GP%']
values_2025 = [
    results_2025['MTD Gross Sales'] / 1000,
    results_2025['MTD GP%'],
    results_2025['YTD Gross Sales'] / 1000,
    results_2025['YTD GP%']
]
values_2024 = [
    results_2024['MTD Gross Sales'] / 1000,
    results_2024['MTD GP%'],
    results_2024['YTD Gross Sales'] / 1000,
    results_2024['YTD GP%']
]

bars1 = ax.bar(x - width/2, values_2025, width, label='2025', color='#2E86AB', alpha=0.8, edgecolor='black', linewidth=1.5)
bars2 = ax.bar(x + width/2, values_2024, width, label='2024', color='#A23B72', alpha=0.8, edgecolor='black', linewidth=1.5)

ax.set_title('Year-over-Year Performance Comparison', fontsize=13, fontweight='bold')
ax.set_ylabel('Value', fontsize=10)
ax.set_xticks(x)
ax.set_xticklabels(metrics, fontsize=9)
ax.legend(fontsize=10, loc='upper left')
ax.grid(axis='y', alpha=0.3, linestyle='--')

for bars in [bars1, bars2]:
    for i, bar in enumerate(bars):
        height = bar.get_height()
        if i in [0, 2]:
            label = f'${height:,.0f}K'
        else:
            label = f'{height:.2f}%'
        ax.text(bar.get_x() + bar.get_width()/2., height,
                label, ha='center', va='bottom', fontsize=7, fontweight='bold')

plt.tight_layout()

# Save comparison chart to bytes
img_buffer2 = io.BytesIO()
fig_comparison.savefig(img_buffer2, format='png', dpi=150, bbox_inches='tight')
img_buffer2.seek(0)
plt.close(fig_comparison)

# Create a new sheet for the comparison chart
ws_chart2 = wb.create_sheet("Comparison Chart")
img2 = XLImage(img_buffer2)
ws_chart2.add_image(img2, 'A1')

# Save the workbook
output_filename = r"C:\Users\OEM\Downloads\report-generator\Sales_Summary_Report.xlsx"
wb.save(output_filename)

print(f"\n✓ Excel file created successfully: {output_filename}")
print(f"  - Sheet 1: Sales Summary Report (formatted table)")
print(f"  - Sheet 2: Dashboard Chart (4-panel visualization)")
print(f"  - Sheet 3: Comparison Chart (year-over-year comparison)")
print(f"\nThe Excel file includes:")
print(f"  • Formatted summary table matching the screenshot layout")
print(f"  • Professional styling with colors and borders")
print(f"  • Currency and percentage formatting")
print(f"  • Two chart visualizations as images")

In [ ]:
# Aggregate items into brands, compute Top-10 MTD by Brand and export to Excel
import re
from collections import defaultdict
import pandas as pd
import io
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side

# Parameters
target_month = 10
target_year = 2025
prev_year = target_year - 1

# Helper: extract letter prefix from item code (e.g., AMO0002 -> AMO)
def extract_brand_code(item_key):
    # item_key format from earlier: "<sheet>_<itemcode>"; get the last part
    code = str(item_key).split('_')[-1]
    m = re.match(r'([A-Za-z]+)', code)
    return m.group(1).upper() if m else code.upper()

# Helper: compute metrics for a single item dataframe and a given year
def compute_item_year_metrics(item_df, year, month):
    # Find col names similar to earlier logic
    df = item_df.copy()
    year_col = None
    period_col = None
    sales_col = None
    cost_col = None
    desc_col = None
    for col in df.columns:
        col_lower = str(col).strip().lower()
        if 'year' in col_lower and not year_col:
            year_col = col
        elif 'period' in col_lower and not period_col:
            period_col = col
        elif 'sales amount' in col_lower and not sales_col:
            sales_col = col
        elif 'cost of sales' in col_lower and not cost_col:
            cost_col = col
        elif any(k in col_lower for k in ['description','item','name']) and not desc_col:
            desc_col = col

    # If essential cols missing, return zeros
    if not year_col or not period_col or not sales_col:
        return {'mtd_sales':0.0,'mtd_cost':0.0,'ytd_sales':0.0,'ytd_cost':0.0,'desc':None}

    df[year_col] = pd.to_numeric(df[year_col], errors='coerce')
    df[period_col] = pd.to_numeric(df[period_col], errors='coerce')
    df[sales_col] = pd.to_numeric(df[sales_col], errors='coerce').fillna(0)
    if cost_col:
        df[cost_col] = pd.to_numeric(df[cost_col], errors='coerce').fillna(0)

    year_data = df[df[year_col] == year]
    if year_data.empty:
        return {'mtd_sales':0.0,'mtd_cost':0.0,'ytd_sales':0.0,'ytd_cost':0.0,'desc':None}

    mtd_data = year_data[year_data[period_col] == month]
    ytd_data = year_data[year_data[period_col] <= month]

    mtd_sales = float(mtd_data[sales_col].sum())
    mtd_cost = float(mtd_data[cost_col].sum()) if cost_col else 0.0
    ytd_sales = float(ytd_data[sales_col].sum())
    ytd_cost = float(ytd_data[cost_col].sum()) if cost_col else 0.0

    # get a description sample (first non-null in desc_col)
    desc_val = None
    if desc_col is not None:
        non_nulls = df[desc_col].dropna().astype(str)
        if len(non_nulls):
            desc_val = non_nulls.iloc[0]

    return {'mtd_sales':mtd_sales,'mtd_cost':mtd_cost,'ytd_sales':ytd_sales,'ytd_cost':ytd_cost,'desc':desc_val}

# Group items by brand code
brands = defaultdict(list)
item_desc_samples = defaultdict(list)

for item_key, item_df in all_items.items():
    brand_code = extract_brand_code(item_key)
    brands[brand_code].append((item_key, item_df))

# Aggregate per brand for both years
brand_metrics = []
for brand_code, items in brands.items():
    agg = {
        'brand_code': brand_code,
        'brand_name': None,
        '2025_mtd_sales': 0.0,
        '2025_mtd_cost': 0.0,
        '2025_ytd_sales': 0.0,
        '2025_ytd_cost': 0.0,
        '2024_mtd_sales': 0.0,
        '2024_mtd_cost': 0.0,
        '2024_ytd_sales': 0.0,
        '2024_ytd_cost': 0.0,
    }
    desc_sets = []
    for item_key, item_df in items:
        # compute for 2025
        metrics_2025 = compute_item_year_metrics(item_df, target_year, target_month)
        metrics_2024 = compute_item_year_metrics(item_df, prev_year, target_month)
        agg['2025_mtd_sales'] += metrics_2025['mtd_sales']
        agg['2025_mtd_cost'] += metrics_2025['mtd_cost']
        agg['2025_ytd_sales'] += metrics_2025['ytd_sales']
        agg['2025_ytd_cost'] += metrics_2025['ytd_cost']
        agg['2024_mtd_sales'] += metrics_2024['mtd_sales']
        agg['2024_mtd_cost'] += metrics_2024['mtd_cost']
        agg['2024_ytd_sales'] += metrics_2024['ytd_sales']
        agg['2024_ytd_cost'] += metrics_2024['ytd_cost']

        # collect desc first 3 words set
        desc = metrics_2025.get('desc') or metrics_2024.get('desc')
        if desc:
            words = [w.strip().upper() for w in re.split(r'\s+', str(desc)) if w.strip()]
            first3 = words[:3]
            if first3:
                desc_sets.append(set(first3))

    # Determine brand_name: intersection of all desc_sets first3 words
    if desc_sets:
        common = set.intersection(*desc_sets) if len(desc_sets) > 1 else desc_sets[0]
        if common:
            # preserve order by looking at first item's desc
            sample_desc = None
            for _, df in items:
                # attempt to find description column again
                for col in df.columns:
                    if any(k in str(col).lower() for k in ['description','item','name']):
                        non_nulls = df[col].dropna().astype(str)
                        if len(non_nulls):
                            sample_desc = non_nulls.iloc[0]
                            break
                if sample_desc:
                    break
            if sample_desc:
                sample_words = [w.strip() for w in re.split(r'\s+', sample_desc) if w.strip()]
                brand_words = [w for w in sample_words[:3] if w.upper() in common]
                brand_name = ' '.join(brand_words)
            else:
                brand_name = ' '.join(list(common))
        else:
            brand_name = brand_code
    else:
        brand_name = brand_code

    agg['brand_name'] = brand_name

    # compute GP% for each year
    def compute_gp(sales, cost):
        return ((sales - cost) / sales * 100) if sales > 0 else 0.0

    agg['2025_mtd_gp'] = compute_gp(agg['2025_mtd_sales'], agg['2025_mtd_cost'])
    agg['2025_ytd_gp'] = compute_gp(agg['2025_ytd_sales'], agg['2025_ytd_cost'])
    agg['2024_mtd_gp'] = compute_gp(agg['2024_mtd_sales'], agg['2024_mtd_cost'])
    agg['2024_ytd_gp'] = compute_gp(agg['2024_ytd_sales'], agg['2024_ytd_cost'])

    # % Achieved vs 2024 for MTD
    agg['mtd_achieved_pct'] = (agg['2025_mtd_sales'] / agg['2024_mtd_sales'] * 100) if agg['2024_mtd_sales'] > 0 else 0.0

    brand_metrics.append(agg)

# Convert to DataFrame and pick top 10 by 2025 MTD sales
bm_df = pd.DataFrame(brand_metrics)
if bm_df.empty:
    print('\nNo brand metrics could be computed (missing columns or data).')
else:
    bm_df = bm_df.sort_values(by='2025_mtd_sales', ascending=False).head(10)

    # Format columns similar to screenshot
    output_rows = []
    for i, row in bm_df.reset_index(drop=True).iterrows():
        output_rows.append({
            'Rank': i+1,
            'Brand': row['brand_name'],
            '2025 MTD Gross Sales': row['2025_mtd_sales'],
            '2025 MTD GP%': row['2025_mtd_gp'],
            '2024 MTD Gross Sales': row['2024_mtd_sales'],
            '2024 MTD GP%': row['2024_mtd_gp'],
            '2025 MTD Budget ($)': None,
            '% Achieved vs 2024': row['mtd_achieved_pct'],
            '% Achieved vs Budget': None
        })

    out_df = pd.DataFrame(output_rows)

    # Save to Excel with formatting (openpyxl)
    out_path = r"C:\Users\OEM\Downloads\report-generator\Top10_Brand_MTD_Report.xlsx"
    wb = Workbook()
    ws = wb.active
    ws.title = 'Top 10 MTD Brands'

    # Header row
    headers = list(out_df.columns)
    for c_idx, h in enumerate(headers, start=1):
        cell = ws.cell(row=2, column=c_idx, value=h)
        cell.font = Font(bold=True, color='FFFFFF')
        cell.fill = PatternFill(start_color='4F81BD', end_color='4F81BD', fill_type='solid')
        cell.alignment = Alignment(horizontal='center')
        cell.border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))

    # Write data rows
    for r_idx, row in enumerate(out_df.itertuples(index=False), start=3):
        for c_idx, value in enumerate(row, start=1):
            cell = ws.cell(row=r_idx, column=c_idx, value=value)
            cell.border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
            if headers[c_idx-1] in ['2025 MTD Gross Sales','2024 MTD Gross Sales','2025 MTD Budget ($)'] and value is not None:
                cell.number_format = '$#,##0.00'
                cell.alignment = Alignment(horizontal='right')
            elif headers[c_idx-1] in ['2025 MTD GP%','2024 MTD GP%','% Achieved vs 2024','% Achieved vs Budget'] and value is not None:
                cell.number_format = '0.00%'
                cell.alignment = Alignment(horizontal='right')
            else:
                cell.alignment = Alignment(horizontal='left' if c_idx==2 else 'right')

    # Adjust column widths
    widths = [6, 22, 18, 12, 18, 12, 18, 14, 16]
    for i, w in enumerate(widths, start=1):
        ws.column_dimensions[ws.cell(row=2, column=i).column_letter].width = w

    # Title in A1
    ws.merge_cells('A1:I1')
    tcell = ws['A1']
    tcell.value = f'Top 10 MTD Brand Performance - {target_month}/{target_year}'
    tcell.font = Font(size=14, bold=True)
    tcell.alignment = Alignment(horizontal='center')

    wb.save(out_path)
    print(f"\n✓ Top-10 brand report created: {out_path}")

# End of brand aggregation & export cell

In [ ]:
# Generate Top-10 Brand Reports for both MTD and YTD
import re
from collections import defaultdict
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side

# Parameters
target_month = 10
target_year = 2025
prev_year = target_year - 1

# Helper: extract letter prefix from item code (e.g., AMO0002 -> AMO)
def extract_brand_code(item_key):
    code = str(item_key).split('_')[-1]
    m = re.match(r'([A-Za-z]+)', code)
    return m.group(1).upper() if m else code.upper()

# Helper: compute metrics for a single item dataframe and a given year
def compute_item_year_metrics(item_df, year, month):
    df = item_df.copy()
    year_col = None
    period_col = None
    sales_col = None
    cost_col = None
    desc_col = None
    
    for col in df.columns:
        col_lower = str(col).strip().lower()
        if 'year' in col_lower and not year_col:
            year_col = col
        elif 'period' in col_lower and not period_col:
            period_col = col
        elif 'sales amount' in col_lower and not sales_col:
            sales_col = col
        elif 'cost of sales' in col_lower and not cost_col:
            cost_col = col
        elif any(k in col_lower for k in ['description','item','name']) and not desc_col:
            desc_col = col

    if not year_col or not period_col or not sales_col:
        return {'mtd_sales':0.0,'mtd_cost':0.0,'ytd_sales':0.0,'ytd_cost':0.0,'desc':None}

    df[year_col] = pd.to_numeric(df[year_col], errors='coerce')
    df[period_col] = pd.to_numeric(df[period_col], errors='coerce')
    df[sales_col] = pd.to_numeric(df[sales_col], errors='coerce').fillna(0)
    if cost_col:
        df[cost_col] = pd.to_numeric(df[cost_col], errors='coerce').fillna(0)

    year_data = df[df[year_col] == year]
    if year_data.empty:
        return {'mtd_sales':0.0,'mtd_cost':0.0,'ytd_sales':0.0,'ytd_cost':0.0,'desc':None}

    mtd_data = year_data[year_data[period_col] == month]
    ytd_data = year_data[year_data[period_col] <= month]

    mtd_sales = float(mtd_data[sales_col].sum())
    mtd_cost = float(mtd_data[cost_col].sum()) if cost_col else 0.0
    ytd_sales = float(ytd_data[sales_col].sum())
    ytd_cost = float(ytd_data[cost_col].sum()) if cost_col else 0.0

    desc_val = None
    if desc_col is not None:
        non_nulls = df[desc_col].dropna().astype(str)
        if len(non_nulls):
            desc_val = non_nulls.iloc[0]

    return {'mtd_sales':mtd_sales,'mtd_cost':mtd_cost,'ytd_sales':ytd_sales,'ytd_cost':ytd_cost,'desc':desc_val}

# Group items by brand code (exclude MX brand)
brands = defaultdict(list)
for item_key, item_df in all_items.items():
    brand_code = extract_brand_code(item_key)
    # Skip MX brand
    if brand_code == 'MX':
        continue
    brands[brand_code].append((item_key, item_df))

# Aggregate per brand for both MTD and YTD
brand_metrics = []
for brand_code, items in brands.items():
    agg = {
        'brand_code': brand_code,
        'brand_name': None,
        '2025_mtd_sales': 0.0,
        '2025_mtd_cost': 0.0,
        '2025_ytd_sales': 0.0,
        '2025_ytd_cost': 0.0,
        '2024_mtd_sales': 0.0,
        '2024_mtd_cost': 0.0,
        '2024_ytd_sales': 0.0,
        '2024_ytd_cost': 0.0,
    }
    desc_sets = []
    
    for item_key, item_df in items:
        metrics_2025 = compute_item_year_metrics(item_df, target_year, target_month)
        metrics_2024 = compute_item_year_metrics(item_df, prev_year, target_month)
        
        agg['2025_mtd_sales'] += metrics_2025['mtd_sales']
        agg['2025_mtd_cost'] += metrics_2025['mtd_cost']
        agg['2025_ytd_sales'] += metrics_2025['ytd_sales']
        agg['2025_ytd_cost'] += metrics_2025['ytd_cost']
        agg['2024_mtd_sales'] += metrics_2024['mtd_sales']
        agg['2024_mtd_cost'] += metrics_2024['mtd_cost']
        agg['2024_ytd_sales'] += metrics_2024['ytd_sales']
        agg['2024_ytd_cost'] += metrics_2024['ytd_cost']
        
        desc = metrics_2025.get('desc') or metrics_2024.get('desc')
        if desc:
            words = [w.strip().upper() for w in re.split(r'\s+', str(desc)) if w.strip()]
            first3 = words[:3]
            if first3:
                desc_sets.append(set(first3))
    
    # Determine brand name from common words
    if desc_sets:
        common = set.intersection(*desc_sets) if len(desc_sets) > 1 else desc_sets[0]
        if common:
            sample_desc = None
            for _, df in items:
                for col in df.columns:
                    if any(k in str(col).lower() for k in ['description','item','name']):
                        non_nulls = df[col].dropna().astype(str)
                        if len(non_nulls):
                            sample_desc = non_nulls.iloc[0]
                            break
                if sample_desc:
                    break
            if sample_desc:
                sample_words = [w.strip() for w in re.split(r'\s+', sample_desc) if w.strip()]
                brand_words = [w for w in sample_words[:3] if w.upper() in common]
                brand_name = ' '.join(brand_words)
            else:
                brand_name = ' '.join(list(common))
        else:
            brand_name = brand_code
    else:
        brand_name = brand_code
    
    agg['brand_name'] = brand_name
    
    # Compute GP%
    def compute_gp(sales, cost):
        return ((sales - cost) / sales * 100) if sales > 0 else 0.0
    
    agg['2025_mtd_gp'] = compute_gp(agg['2025_mtd_sales'], agg['2025_mtd_cost'])
    agg['2025_ytd_gp'] = compute_gp(agg['2025_ytd_sales'], agg['2025_ytd_cost'])
    agg['2024_mtd_gp'] = compute_gp(agg['2024_mtd_sales'], agg['2024_mtd_cost'])
    agg['2024_ytd_gp'] = compute_gp(agg['2024_ytd_sales'], agg['2024_ytd_cost'])
    
    # % Achieved
    agg['mtd_achieved_pct'] = (agg['2025_mtd_sales'] / agg['2024_mtd_sales'] * 100) if agg['2024_mtd_sales'] > 0 else 0.0
    agg['ytd_achieved_pct'] = (agg['2025_ytd_sales'] / agg['2024_ytd_sales'] * 100) if agg['2024_ytd_sales'] > 0 else 0.0
    
    brand_metrics.append(agg)

# Convert to DataFrame
bm_df = pd.DataFrame(brand_metrics)

if bm_df.empty:
    print('\nNo brand metrics could be computed (missing columns or data).')
else:
    # Create Top 10 MTD Report
    print(f"\n{'='*80}")
    print("TOP 10 BRANDS BY MTD GROSS SALES")
    print(f"{'='*80}\n")
    
    top10_mtd = bm_df.sort_values(by='2025_mtd_sales', ascending=False).head(10)
    
    mtd_output = []
    for i, row in top10_mtd.reset_index(drop=True).iterrows():
        mtd_output.append({
            'Rank': i+1,
            'Brand': row['brand_name'],
            '2025 MTD Sales': f"${row['2025_mtd_sales']:,.2f}",
            '2025 MTD GP%': f"{row['2025_mtd_gp']:.2f}%",
            '2024 MTD Sales': f"${row['2024_mtd_sales']:,.2f}",
            '2024 MTD GP%': f"{row['2024_mtd_gp']:.2f}%",
            '% Achieved': f"{row['mtd_achieved_pct']:.2f}%"
        })
    
    mtd_preview = pd.DataFrame(mtd_output)
    print(mtd_preview.to_string(index=False))
    
    # Create Top 10 YTD Report
    print(f"\n\n{'='*80}")
    print("TOP 10 BRANDS BY YTD GROSS SALES")
    print(f"{'='*80}\n")
    
    top10_ytd = bm_df.sort_values(by='2025_ytd_sales', ascending=False).head(10)
    
    ytd_output = []
    for i, row in top10_ytd.reset_index(drop=True).iterrows():
        ytd_output.append({
            'Rank': i+1,
            'Brand': row['brand_name'],
            '2025 YTD Sales': f"${row['2025_ytd_sales']:,.2f}",
            '2025 YTD GP%': f"{row['2025_ytd_gp']:.2f}%",
            '2024 YTD Sales': f"${row['2024_ytd_sales']:,.2f}",
            '2024 YTD GP%': f"{row['2024_ytd_gp']:.2f}%",
            '% Achieved': f"{row['ytd_achieved_pct']:.2f}%"
        })
    
    ytd_preview = pd.DataFrame(ytd_output)
    print(ytd_preview.to_string(index=False))
    
    # Save both reports to Excel files with charts
    def create_brand_excel(df_sorted, metric_type, file_suffix):
        import matplotlib.pyplot as plt
        import numpy as np
        import io
        from openpyxl.drawing.image import Image as XLImage
        
        wb = Workbook()
        ws = wb.active
        ws.title = f'Top 10 {metric_type} Brands'
        
        # Title
        ws.merge_cells('A1:I1')
        tcell = ws['A1']
        tcell.value = f'Top 10 {metric_type} Brand Performance - {target_month}/{target_year}'
        tcell.font = Font(size=14, bold=True)
        tcell.alignment = Alignment(horizontal='center')
        
        # Header styling
        header_fill = PatternFill(start_color="FFEB9C", end_color="FFEB9C", fill_type="solid")
        green_fill = PatternFill(start_color="C6E0B4", end_color="C6E0B4", fill_type="solid")
        blue_fill = PatternFill(start_color="BDD7EE", end_color="BDD7EE", fill_type="solid")
        
        headers = ['', 'Brand', 
                   f'{target_year} {metric_type}\nGross Sales', f'{target_year} {metric_type}\nGP%',
                   f'{prev_year} {metric_type}\nGross Sales', f'{prev_year} {metric_type}\nGP%',
                   f'{target_year} {metric_type}\nBudget ($)', 
                   f'% Achieved vs\n{prev_year}', '% Achieved vs\nBudget']
        
        for c_idx, h in enumerate(headers, start=1):
            cell = ws.cell(row=2, column=c_idx, value=h)
            cell.font = Font(bold=True, size=10)
            cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
            cell.border = Border(left=Side(style='thin'), right=Side(style='thin'),
                               top=Side(style='thin'), bottom=Side(style='thin'))
            
            if c_idx in [3, 4]:
                cell.fill = header_fill
            elif c_idx in [5, 6]:
                cell.fill = green_fill
            elif c_idx in [7, 8, 9]:
                cell.fill = blue_fill
        
        # Write data
        sales_key = f'{target_year}_{metric_type.lower()}_sales'
        gp_key = f'{target_year}_{metric_type.lower()}_gp'
        prev_sales_key = f'{prev_year}_{metric_type.lower()}_sales'
        prev_gp_key = f'{prev_year}_{metric_type.lower()}_gp'
        achieved_key = f'{metric_type.lower()}_achieved_pct'
        
        for r_idx, row in df_sorted.reset_index(drop=True).iterrows():
            rank = r_idx + 1
            row_data = [
                rank, 
                row['brand_name'],
                row[sales_key],
                row[gp_key] / 100,
                row[prev_sales_key],
                row[prev_gp_key] / 100,
                None,
                row[achieved_key] / 100,
                None
            ]
            
            excel_row = r_idx + 3
            for c_idx, value in enumerate(row_data, start=1):
                cell = ws.cell(row=excel_row, column=c_idx, value=value)
                cell.border = Border(left=Side(style='thin'), right=Side(style='thin'),
                                   top=Side(style='thin'), bottom=Side(style='thin'))
                
                if c_idx in [3, 5, 7]:
                    cell.number_format = '$#,##0.00'
                    cell.alignment = Alignment(horizontal='right')
                elif c_idx in [4, 6, 8, 9]:
                    cell.number_format = '0.00%'
                    cell.alignment = Alignment(horizontal='right')
                elif c_idx == 1:
                    cell.alignment = Alignment(horizontal='center')
                else:
                    cell.alignment = Alignment(horizontal='left')
        
        # Column widths
        widths = [6, 22, 18, 12, 18, 12, 18, 14, 16]
        for i, w in enumerate(widths, start=1):
            ws.column_dimensions[ws.cell(row=2, column=i).column_letter].width = w
        
        ws.row_dimensions[1].height = 25
        ws.row_dimensions[2].height = 35
        
        # Create charts
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
        fig.suptitle(f'Top 10 Brands - {metric_type} Performance ({target_month}/{target_year})', 
                     fontsize=14, fontweight='bold')
        
        # Extract data for charts
        brands = df_sorted['brand_name'].head(10).tolist()
        sales_2025 = df_sorted[sales_key].head(10).tolist()
        sales_2024 = df_sorted[prev_sales_key].head(10).tolist()
        gp_2025 = df_sorted[gp_key].head(10).tolist()
        gp_2024 = df_sorted[prev_gp_key].head(10).tolist()
        
        # Chart 1: Sales Comparison
        x = np.arange(len(brands))
        width = 0.35
        
        bars1 = ax1.barh(x - width/2, sales_2025, width, label=str(target_year), 
                        color='#2E86AB', alpha=0.8, edgecolor='black', linewidth=1)
        bars2 = ax1.barh(x + width/2, sales_2024, width, label=str(prev_year), 
                        color='#A23B72', alpha=0.8, edgecolor='black', linewidth=1)
        
        ax1.set_xlabel('Sales Amount ($)', fontsize=10, fontweight='bold')
        ax1.set_ylabel('Brand', fontsize=10, fontweight='bold')
        ax1.set_title(f'{metric_type} Gross Sales', fontsize=11, fontweight='bold')
        ax1.set_yticks(x)
        ax1.set_yticklabels(brands, fontsize=8)
        ax1.legend(fontsize=9)
        ax1.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))
        ax1.grid(axis='x', alpha=0.3, linestyle='--')
        ax1.invert_yaxis()
        
        # Add value labels
        for bars in [bars1, bars2]:
            for bar in bars:
                width_val = bar.get_width()
                ax1.text(width_val, bar.get_y() + bar.get_height()/2.,
                        f'${width_val/1000:.0f}K',
                        ha='left', va='center', fontsize=7, fontweight='bold')
        
        # Chart 2: GP% Comparison
        bars3 = ax2.barh(x - width/2, gp_2025, width, label=str(target_year), 
                        color='#2E86AB', alpha=0.8, edgecolor='black', linewidth=1)
        bars4 = ax2.barh(x + width/2, gp_2024, width, label=str(prev_year), 
                        color='#A23B72', alpha=0.8, edgecolor='black', linewidth=1)
        
        ax2.set_xlabel('GP %', fontsize=10, fontweight='bold')
        ax2.set_title(f'{metric_type} Gross Profit %', fontsize=11, fontweight='bold')
        ax2.set_yticks(x)
        ax2.set_yticklabels(brands, fontsize=8)
        ax2.legend(fontsize=9)
        ax2.grid(axis='x', alpha=0.3, linestyle='--')
        ax2.invert_yaxis()
        
        # Add value labels
        for bars in [bars3, bars4]:
            for bar in bars:
                width_val = bar.get_width()
                ax2.text(width_val, bar.get_y() + bar.get_height()/2.,
                        f'{width_val:.1f}%',
                        ha='left', va='center', fontsize=7, fontweight='bold')
        
        plt.tight_layout()
        
        # Save chart to bytes
        img_buffer = io.BytesIO()
        fig.savefig(img_buffer, format='png', dpi=150, bbox_inches='tight')
        img_buffer.seek(0)
        plt.close(fig)
        
        # Add chart to new sheet
        ws_chart = wb.create_sheet(f"{metric_type} Charts")
        img = XLImage(img_buffer)
        ws_chart.add_image(img, 'A1')
        
        # Save
        out_path = rf"C:\Users\OEM\Downloads\report-generator\Top10_Brand_{metric_type}_Report.xlsx"
        wb.save(out_path)
        return out_path
    
    # Generate both Excel files
    mtd_path = create_brand_excel(top10_mtd, 'MTD', 'mtd')
    ytd_path = create_brand_excel(top10_ytd, 'YTD', 'ytd')
    
    print(f"\n\n✓ Excel files created successfully:")
    print(f"  • MTD Report: {mtd_path}")
    print(f"  • YTD Report: {ytd_path}")
    print(f"\nTotal brands identified: {len(brands)}")

In [ ]:
# Generate Top-20 SKU Reports for both MTD and YTD
import re
from collections import defaultdict
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
import matplotlib.pyplot as plt
import numpy as np
import io
from openpyxl.drawing.image import Image as XLImage

# Parameters
target_month = 10
target_year = 2025
prev_year = target_year - 1

# Helper functions already defined above
# extract_brand_code, compute_item_year_metrics

def get_sku_code_and_name(item_df):
    """Extract original item code and a readable name from the dataframe."""
    if item_df.empty:
        return '', ''

    first_row = item_df.iloc[0]

    # Extract code from Item Number column
    code = ''
    for col in item_df.columns:
        col_lower = str(col).lower()
        if 'item' in col_lower and any(k in col_lower for k in ['no', 'number', '#', 'code']):
            val = first_row[col]
            if pd.notna(val):
                val_str = str(val).strip()
                if val_str:
                    code = val_str.split()[0]
                    break

    if not code:
        raw_value = str(first_row.iloc[0]).strip()
        match = re.search(r'([A-Za-z]+\d+)', raw_value)
        if match:
            code = match.group(1)
        elif raw_value:
            code = raw_value.split()[0]

    # Extract name - look through all columns for a descriptive text
    name = ''
    # First try specific description columns
    for col in item_df.columns:
        col_lower = str(col).lower()
        if any(key in col_lower for key in ['description', 'item name', 'item desc']):
            val = first_row[col]
            if pd.notna(val):
                val_str = str(val).strip()
                if val_str and (not code or not val_str.upper().startswith(code.upper())):
                    name = val_str
                    break
    
    # If no name found, search all columns for a string that looks like a description
    # (contains spaces and is longer than the code)
    if not name:
        for col in item_df.columns:
            if col == 'Item Number':
                continue
            val = first_row[col]
            if pd.notna(val):
                val_str = str(val).strip()
                # Check if this looks like a product description
                if val_str and len(val_str) > 10 and ' ' in val_str and not val_str.replace('.','').replace(',','').isdigit():
                    # Make sure it's not just the code repeated
                    if not code or not val_str.upper().startswith(code.upper()):
                        name = val_str
                        break
                    # If it starts with code, extract the remainder
                    elif code and val_str.upper().startswith(code.upper()):
                        remainder = val_str[len(code):].strip(' -:_')
                        if remainder and len(remainder) > 3:
                            name = remainder
                            break

    if not name:
        raw_value = str(first_row.iloc[0]).strip()
        if code and raw_value.upper().startswith(code.upper()):
            remainder = raw_value[len(code):].strip(' -:_')
            if remainder:
                name = remainder
        elif raw_value and raw_value != code:
            name = raw_value

    if not code:
        code = name or ''
    if not name:
        name = code

    return code.strip(), name.strip()


In [ ]:
    """Create Top-20 SKU MTD or YTD Performance report (excludes MX products)"""
    
    # Collect all SKUs with their metrics
    sku_metrics = []
    
    for item_key, item_df in all_items.items():
        print(item_key, item_df)
        break


In [ ]:
import pandas as pd
import os

def extract_excel_sheets(excel_file_path: str, output_folder: str = ".") -> list[str]:
    """
    Extract all sheets from an Excel file and save each as a separate Excel file.
    
    Args:
        excel_file_path: Path to the input Excel file
        output_folder: Folder to save the extracted sheets (default: current folder)
    
    Returns:
        List of saved file paths
    """
    # Create output folder if it doesn't exist
    if output_folder != "." and not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Get the base filename without extension
    base_name = os.path.splitext(os.path.basename(excel_file_path))[0]
    
    # Read all sheets from the Excel file
    excel_file = pd.ExcelFile(excel_file_path)
    sheet_names = excel_file.sheet_names
    
    saved_files = []
    
    for sheet_name in sheet_names:
        # Read the sheet
        df = pd.read_excel(excel_file, sheet_name=sheet_name)
        
        # Clean sheet name for use as filename (remove invalid characters)
        clean_sheet_name = "".join(c if c.isalnum() or c in (' ', '-', '_') else '_' for c in sheet_name)
        
        # Create output filename
        output_filename = f"{base_name}_{clean_sheet_name}.xlsx"
        output_path = os.path.join(output_folder, output_filename)
        
        # Save the sheet as a separate Excel file
        df.to_excel(output_path, index=False, sheet_name=sheet_name)
        saved_files.append(output_path)
        print(f"Saved: {output_path}")
    
    print(f"\nTotal sheets extracted: {len(saved_files)}")
    return saved_files


# Example usage:
saved_files = extract_excel_sheets("NZ Sales Act + Fcst 2023-2026.xlsx")
# Or specify an output folder:
# saved_files = extract_excel_sheets("your_excel_file.xlsx", output_folder="extracted_sheets")

In [ ]:
import pandas as pd

def join_sales_customer_with_channel(
    sales_customer_file: str = "SalesCustomerList.xlsx",
    cust_master_file: str = "CustMaster.xlsx",
    output_file: str = None
) -> pd.DataFrame:
    """
    Join SalesCustomerList with CustMaster to add the Channel column.
    
    Args:
        sales_customer_file: Path to the SalesCustomerList Excel file
        cust_master_file: Path to the CustMaster Excel file
        output_file: Optional path to save the result. If None, doesn't save.
    
    Returns:
        DataFrame with all SalesCustomerList data plus the Channel column
    """
    # Read both Excel files
    sales_df = pd.read_excel(sales_customer_file)
    cust_master_df = pd.read_excel(cust_master_file)
    
    print(f"SalesCustomerList: {len(sales_df)} rows")
    print(f"CustMaster: {len(cust_master_df)} rows")
    
    # Select only the columns needed from CustMaster for the join
    # CustID will be used to match with Customer # in SalesCustomerList
    cust_channel_df = cust_master_df[['CustID', 'Channel']].copy()
    
    # Perform left join to keep all rows from SalesCustomerList
    # and add Channel from CustMaster where Customer # matches CustID
    result_df = sales_df.merge(
        cust_channel_df,
        left_on='Customer #',
        right_on='CustID',
        how='left'
    )
    
    # Drop the duplicate CustID column (we already have Customer #)
    result_df = result_df.drop(columns=['CustID'])
    
    # Report matching stats
    matched = result_df['Channel'].notna().sum()
    unmatched = result_df['Channel'].isna().sum()
    print(f"\nMatched customers: {matched}")
    print(f"Unmatched customers: {unmatched}")
    
    if unmatched > 0:
        print(f"\nUnmatched Customer #s:")
        print(result_df[result_df['Channel'].isna()]['Customer #'].tolist())
    
    # Save to file if output path is provided
    if output_file:
        result_df.to_excel(output_file, index=False)
        print(f"\nResult saved to: {output_file}")
    
    return result_df


# Example usage:
result = join_sales_customer_with_channel(
    sales_customer_file="SalesCustomerList.xlsx",
    cust_master_file="CustMaster.xlsx",
    output_file="SalesCustomerList_with_Channel.xlsx"
)

# Display the result
print("\nResult preview:")
print(result.head(10))

In [ ]:
import pandas as pd

def pivot_sales_by_customer_year(
    sales_file: str = "sales detail 2024-2025.xls",
    output_file: str = None
) -> pd.DataFrame:
    """
    Read sales details file and pivot it so that:
    - Columns are years (2024, 2025, etc.)
    - Rows are customers
    - Cell values are sum of Sales Amount
    
    Args:
        sales_file: Path to the sales details Excel file
        output_file: Optional path to save the result
    
    Returns:
        Pivoted DataFrame with customers as rows and years as columns
    """
    # Read the Excel file - data starts at row 10 (0-indexed)
    # The header structure is complex, so we read without header first
    df_raw = pd.read_excel(sales_file, header=None)
    
    # Find the actual data rows - look for rows where first column is a year (numeric)
    # or where it's a customer code followed by year data
    
    # Based on the file structure:
    # - Row 10 has column headers
    # - Row 11 onwards has item/customer/data rows
    # - Customer rows have customer code in column 0
    # - Data rows have Year in column 0, Period in column 1
    
    # Define column indices based on the file structure
    col_year = 0           # Year column
    col_period = 1         # Period column  
    col_type = 2           # Type (IN, etc.)
    col_sales_amount = 17  # Sales Amount column (based on screenshot)
    col_customer = 0       # Customer number appears in same column as year
    
    # Parse the data
    data_rows = []
    current_customer = None
    current_customer_name = None
    
    for idx in range(11, len(df_raw)):  # Start after headers
        row = df_raw.iloc[idx]
        first_cell = row.iloc[0]
        
        # Skip empty rows
        if pd.isna(first_cell):
            continue
        
        first_cell_str = str(first_cell).strip()
        
        # Check if this is a customer header row (alphanumeric code, not a year)
        # Customer codes like ACEC001, ASIA008, etc.
        if first_cell_str and not first_cell_str.isdigit() and not first_cell_str.startswith('Item'):
            # Check if it looks like a customer code (contains letters and numbers)
            if any(c.isalpha() for c in first_cell_str) and any(c.isdigit() for c in first_cell_str):
                current_customer = first_cell_str
                # Customer name might be in column 7 based on structure
                cust_name = row.iloc[7] if pd.notna(row.iloc[7]) else ''
                current_customer_name = str(cust_name).strip() if cust_name else current_customer
                continue
        
        # Check if this is a data row (Year is numeric like 2024, 2025)
        if first_cell_str.isdigit() and len(first_cell_str) == 4:
            year = int(first_cell_str)
            
            # Get sales amount - column index 17 based on the file structure
            sales_amount = row.iloc[col_sales_amount]
            
            if pd.notna(sales_amount) and current_customer:
                try:
                    sales_value = float(sales_amount)
                    data_rows.append({
                        'Customer #': current_customer,
                        'Customer Name': current_customer_name,
                        'Year': year,
                        'Sales Amount': sales_value
                    })
                except (ValueError, TypeError):
                    pass
    
    # Create DataFrame from parsed data
    df_data = pd.DataFrame(data_rows)
    
    if df_data.empty:
        print("No data found!")
        return pd.DataFrame()
    
    print(f"Parsed {len(df_data)} transaction rows")
    print(f"Unique customers: {df_data['Customer #'].nunique()}")
    print(f"Years found: {sorted(df_data['Year'].unique())}")
    
    # Pivot the data: rows = customers, columns = years, values = sum of sales
    pivot_df = df_data.pivot_table(
        index=['Customer #', 'Customer Name'],
        columns='Year',
        values='Sales Amount',
        aggfunc='sum',
        fill_value=0
    )
    
    # Flatten the column index (just year numbers)
    pivot_df.columns = [str(col) for col in pivot_df.columns]
    
    # Reset index to make Customer # and Customer Name regular columns
    pivot_df = pivot_df.reset_index()
    
    # Add a Total column
    year_columns = [col for col in pivot_df.columns if col.isdigit()]
    pivot_df['Total'] = pivot_df[year_columns].sum(axis=1)
    
    # Sort by Total descending
    pivot_df = pivot_df.sort_values('Total', ascending=False)
    
    print(f"\nResult: {len(pivot_df)} customers x {len(year_columns)} years")
    
    # Save to file if output path is provided
    if output_file:
        pivot_df.to_excel(output_file, index=False)
        print(f"Result saved to: {output_file}")
    
    return pivot_df


# Run the function
result_pivot = pivot_sales_by_customer_year(
    sales_file="sales detail 2024-2025.xls",
    output_file="Sales_by_Customer_Year.xlsx"
)

# Display the result
print("\nTop 20 customers by total sales:")
print(result_pivot.head(20).to_string())

In [1]:
import pandas as pd

def pivot_sales_by_customer_year_month(
    sales_file: str = "sales detail 2024-2025.xls",
    output_file: str = None
) -> pd.DataFrame:
    """
    Read sales details file and pivot it so that:
    - Columns are Year-Month combinations (2024-01, 2024-02, etc.)
    - Rows are customers
    - Cell values are sum of Sales Amount
    
    Args:
        sales_file: Path to the sales details Excel file
        output_file: Optional path to save the result
    
    Returns:
        Pivoted DataFrame with customers as rows and year-month as columns
    """
    # Read the Excel file - data starts at row 10 (0-indexed)
    # The header structure is complex, so we read without header first
    df_raw = pd.read_excel(sales_file, header=None)
    
    # Based on the file structure:
    # - Row 10 has column headers
    # - Row 11 onwards has item/customer/data rows
    # - Customer rows have customer code in column 0
    # - Data rows have Year in column 0, Period (Month) in column 1
    
    # Define column indices based on the file structure
    col_year = 0           # Year column (Column A)
    col_period = 1         # Period/Month column (Column B - "Prd.")
    col_type = 2           # Type (IN, etc.) - Column C
    col_sales_amount = 17  # Sales Amount column (Column R based on screenshot)
    
    # Parse the data
    data_rows = []
    current_customer = None
    current_customer_name = None
    
    for idx in range(11, len(df_raw)):  # Start after headers
        row = df_raw.iloc[idx]
        first_cell = row.iloc[0]
        
        # Skip empty rows
        if pd.isna(first_cell):
            continue
        
        first_cell_str = str(first_cell).strip()
        
        # Check if this is a customer header row (alphanumeric code, not a year)
        # Customer codes like ACEC001, ASIA008, etc.
        if first_cell_str and not first_cell_str.isdigit() and not first_cell_str.startswith('Item'):
            # Check if it looks like a customer code (contains letters and numbers)
            if any(c.isalpha() for c in first_cell_str) and any(c.isdigit() for c in first_cell_str):
                current_customer = first_cell_str
                # Customer name might be in column 7 based on structure
                cust_name = row.iloc[7] if pd.notna(row.iloc[7]) else ''
                current_customer_name = str(cust_name).strip() if cust_name else current_customer
                continue
        
        # Check if this is a data row (Year is numeric like 2024, 2025)
        if first_cell_str.isdigit() and len(first_cell_str) == 4:
            year = int(first_cell_str)
            
            # Get month/period from column B (index 1)
            period_cell = row.iloc[col_period]
            month = None
            if pd.notna(period_cell):
                try:
                    month = int(float(period_cell))
                except (ValueError, TypeError):
                    month = None
            
            # Get sales amount - column index 17 based on the file structure
            sales_amount = row.iloc[col_sales_amount]
            
            if pd.notna(sales_amount) and current_customer and month is not None:
                try:
                    sales_value = float(sales_amount)
                    data_rows.append({
                        'Customer #': current_customer,
                        'Customer Name': current_customer_name,
                        'Year': year,
                        'Month': month,
                        'Sales Amount': sales_value
                    })
                except (ValueError, TypeError):
                    pass
    
    # Create DataFrame from parsed data
    df_data = pd.DataFrame(data_rows)
    
    if df_data.empty:
        print("No data found!")
        return pd.DataFrame()
    
    print(f"Parsed {len(df_data)} transaction rows")
    print(f"Unique customers: {df_data['Customer #'].nunique()}")
    print(f"Years found: {sorted(df_data['Year'].unique())}")
    print(f"Months found: {sorted(df_data['Month'].unique())}")
    
    # Create Year-Month column for pivoting (format: "2024-01", "2024-02", etc.)
    df_data['Year-Month'] = df_data['Year'].astype(str) + '-' + df_data['Month'].astype(str).str.zfill(2)
    
    # Pivot the data: rows = customers, columns = year-month, values = sum of sales
    pivot_df = df_data.pivot_table(
        index=['Customer #', 'Customer Name'],
        columns='Year-Month',
        values='Sales Amount',
        aggfunc='sum',
        fill_value=0
    )
    
    # Sort columns chronologically
    pivot_df = pivot_df.reindex(sorted(pivot_df.columns), axis=1)
    
    # Reset index to make Customer # and Customer Name regular columns
    pivot_df = pivot_df.reset_index()
    
    # Add yearly totals
    year_month_columns = [col for col in pivot_df.columns if '-' in str(col)]
    years = sorted(set(col.split('-')[0] for col in year_month_columns))
    
    for year in years:
        year_cols = [col for col in year_month_columns if col.startswith(f"{year}-")]
        pivot_df[f'{year} Total'] = pivot_df[year_cols].sum(axis=1)
    
    # Add grand total column
    pivot_df['Grand Total'] = pivot_df[year_month_columns].sum(axis=1)
    
    # Sort by Grand Total descending
    pivot_df = pivot_df.sort_values('Grand Total', ascending=False)
    
    print(f"\nResult: {len(pivot_df)} customers x {len(year_month_columns)} year-months")
    
    # Save to file if output path is provided
    if output_file:
        pivot_df.to_excel(output_file, index=False)
        print(f"Result saved to: {output_file}")
    
    return pivot_df


# Run the function
result_pivot = pivot_sales_by_customer_year_month(
    sales_file="sales detail 2024-2025.xls",
    output_file="Sales_by_Customer_Year_Month.xlsx"
)

# Display the result
print("\nTop 20 customers by total sales:")
print(result_pivot.head(20).to_string())

Parsed 55468 transaction rows
Unique customers: 351
Years found: [np.int64(2024), np.int64(2025)]
Months found: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12)]

Result: 351 customers x 23 year-months
Result saved to: Sales_by_Customer_Year_Month.xlsx

Top 20 customers by total sales:
Year-Month Customer #                                              Customer Name    2024-01    2024-02   2024-03   2024-04   2024-05    2024-06    2024-07    2024-08   2024-09   2024-10   2024-11    2024-12    2025-01   2025-02   2025-03   2025-04    2025-05    2025-06   2025-07   2025-08    2025-09   2025-10    2025-11  2024 Total  2025 Total  Grand Total
93            FOOD004                  Woolworths New Zealand Limited - Auckland  112926.23  103704.25  88216.76  82535.34  73279.40  108161.74  104362.31   41259.52  94989.91  77396.86  97477.43  134818.23   59681.11  83984.00  93161.80  70583

In [2]:
import pandas as pd

def add_channel_and_sales_rep_to_sales(
    sales_pivot_file: str = "Sales_by_Customer_Year_Month.xlsx",
    customer_list_file: str = "SalesCustomerList_with_Channel.xlsx",
    output_file: str = None
) -> pd.DataFrame:
    """
    Join the pivoted sales data with SalesCustomerList to add Sales Rep and Channel columns.
    
    Args:
        sales_pivot_file: Path to the Sales_by_Customer_Year_Month.xlsx file
        customer_list_file: Path to the SalesCustomerList_with_Channel.xlsx file
        output_file: Optional path to save the result
    
    Returns:
        DataFrame with Sales Rep and Channel columns added
    """
    # Read both files
    sales_df = pd.read_excel(sales_pivot_file)
    customer_df = pd.read_excel(customer_list_file)
    
    print(f"Sales pivot data: {len(sales_df)} rows")
    print(f"Customer list: {len(customer_df)} rows")
    print(f"\nSales columns: {list(sales_df.columns[:5])}...")
    print(f"Customer columns: {list(customer_df.columns)}")
    
    # Select only the columns we need from customer list
    # Assuming columns are: Customer #, Customer Description, Sales Rep, Channel
    customer_cols = ['Customer #']
    
    # Find Sales Rep column
    sales_rep_col = None
    for col in customer_df.columns:
        if 'sales' in col.lower() and 'rep' in col.lower():
            sales_rep_col = col
            break
    if sales_rep_col:
        customer_cols.append(sales_rep_col)
        print(f"Found Sales Rep column: '{sales_rep_col}'")
    
    # Find Channel column
    channel_col = None
    for col in customer_df.columns:
        if 'channel' in col.lower():
            channel_col = col
            break
    if channel_col:
        customer_cols.append(channel_col)
        print(f"Found Channel column: '{channel_col}'")
    
    # Create subset of customer data for joining
    customer_subset = customer_df[customer_cols].copy()
    
    # Remove duplicates (in case a customer appears multiple times)
    customer_subset = customer_subset.drop_duplicates(subset=['Customer #'])
    
    # Perform left join to add Sales Rep and Channel to sales data
    result_df = sales_df.merge(
        customer_subset,
        on='Customer #',
        how='left'
    )
    
    # Reorder columns to put Sales Rep and Channel after Customer Name
    # Find the position of Customer Name column
    cols = list(result_df.columns)
    
    # Remove Sales Rep and Channel from current positions
    if sales_rep_col and sales_rep_col in cols:
        cols.remove(sales_rep_col)
    if channel_col and channel_col in cols:
        cols.remove(channel_col)
    
    # Insert them after Customer Name (index 1, so insert at 2)
    insert_pos = 2
    if channel_col:
        cols.insert(insert_pos, channel_col)
    if sales_rep_col:
        cols.insert(insert_pos, sales_rep_col)
    
    result_df = result_df[cols]
    
    # Report matching stats
    if sales_rep_col:
        matched_rep = result_df[sales_rep_col].notna().sum()
        print(f"\nMatched Sales Rep: {matched_rep}/{len(result_df)}")
    
    if channel_col:
        matched_channel = result_df[channel_col].notna().sum()
        print(f"Matched Channel: {matched_channel}/{len(result_df)}")
        
        # Show channel distribution
        print(f"\nChannel distribution:")
        channel_counts = result_df[channel_col].value_counts(dropna=False)
        for channel, count in channel_counts.head(10).items():
            print(f"  {channel}: {count}")
    
    # Save to file if output path is provided
    if output_file:
        result_df.to_excel(output_file, index=False)
        print(f"\nResult saved to: {output_file}")
    
    return result_df


# Run the function
result = add_channel_and_sales_rep_to_sales(
    sales_pivot_file="Sales_by_Customer_Year_Month.xlsx",
    customer_list_file="SalesCustomerList_with_Channel.xlsx",
    output_file="Sales_by_Customer_Year_Month_with_Channel.xlsx"
)

# Display preview
print("\nResult preview (first 10 rows, first 8 columns):")
print(result.iloc[:10, :8].to_string())

Sales pivot data: 351 rows
Customer list: 660 rows

Sales columns: ['Customer #', 'Customer Name', '2024-01', '2024-02', '2024-03']...
Customer columns: ['Count', 'Customer #', 'Customer Description', 'Sales Rep', 'Grade', 'Penetration', 'Call/Review Cycle', 'Contact name', 'Contact phone number', 'Contact email', 'Alternate contact', 'Notes', 'Channel']
Found Sales Rep column: 'Sales Rep'
Found Channel column: 'Channel'

Matched Sales Rep: 333/351
Matched Channel: 332/351

Channel distribution:
  Retail Group: 187
  Food Service: 85
  Liquor Group: 21
  nan: 19
  Sub - Distributor: 17
  Mainstream: 10
  Staff: 4
  Indent: 4
  Cash Sales: 2
  Food Ethnic: 2

Result saved to: Sales_by_Customer_Year_Month_with_Channel.xlsx

Result preview (first 10 rows, first 8 columns):
  Customer #                                          Customer Name Sales Rep            Channel    2024-01    2024-02   2024-03   2024-04
0    FOOD004              Woolworths New Zealand Limited - Auckland        RM   

In [3]:
import pandas as pd

def generate_sales_by_customer_with_channel(
    sales_file: str = "sales detail 2024-2025.xls",
    customer_list_file: str = "SalesCustomerList_with_Channel.xlsx",
    output_file: str = "Sales_by_Customer_Year_Month_with_Channel.xlsx"
) -> pd.DataFrame:
    """
    Read sales details file, pivot by customer and year-month, then join with
    customer list to add Sales Rep and Channel columns.
    
    Args:
        sales_file: Path to the sales details Excel file
        customer_list_file: Path to the SalesCustomerList_with_Channel.xlsx file
        output_file: Path to save the final result
    
    Returns:
        DataFrame with customers as rows, year-month as columns, plus Sales Rep and Channel
    """
    # ============================================================
    # STEP 1: Parse and pivot sales data
    # ============================================================
    print("=" * 60)
    print("STEP 1: Reading and parsing sales data...")
    print("=" * 60)
    
    df_raw = pd.read_excel(sales_file, header=None)
    
    # Column indices based on file structure
    col_period = 1         # Period/Month column (Column B)
    col_sales_amount = 17  # Sales Amount column (Column R)
    
    # Parse the data
    data_rows = []
    current_customer = None
    current_customer_name = None
    
    for idx in range(11, len(df_raw)):  # Start after headers
        row = df_raw.iloc[idx]
        first_cell = row.iloc[0]
        
        # Skip empty rows
        if pd.isna(first_cell):
            continue
        
        first_cell_str = str(first_cell).strip()
        
        # Check if this is a customer header row
        if first_cell_str and not first_cell_str.isdigit() and not first_cell_str.startswith('Item'):
            if any(c.isalpha() for c in first_cell_str) and any(c.isdigit() for c in first_cell_str):
                current_customer = first_cell_str
                cust_name = row.iloc[7] if pd.notna(row.iloc[7]) else ''
                current_customer_name = str(cust_name).strip() if cust_name else current_customer
                continue
        
        # Check if this is a data row (Year is numeric like 2024, 2025)
        if first_cell_str.isdigit() and len(first_cell_str) == 4:
            year = int(first_cell_str)
            
            # Get month/period
            period_cell = row.iloc[col_period]
            month = None
            if pd.notna(period_cell):
                try:
                    month = int(float(period_cell))
                except (ValueError, TypeError):
                    month = None
            
            # Get sales amount
            sales_amount = row.iloc[col_sales_amount]
            
            if pd.notna(sales_amount) and current_customer and month is not None:
                try:
                    sales_value = float(sales_amount)
                    data_rows.append({
                        'Customer #': current_customer,
                        'Customer Name': current_customer_name,
                        'Year': year,
                        'Month': month,
                        'Sales Amount': sales_value
                    })
                except (ValueError, TypeError):
                    pass
    
    # Create DataFrame from parsed data
    df_data = pd.DataFrame(data_rows)
    
    if df_data.empty:
        print("No data found!")
        return pd.DataFrame()
    
    print(f"Parsed {len(df_data)} transaction rows")
    print(f"Unique customers: {df_data['Customer #'].nunique()}")
    print(f"Years found: {sorted(df_data['Year'].unique())}")
    print(f"Months found: {sorted(df_data['Month'].unique())}")
    
    # Create Year-Month column for pivoting
    df_data['Year-Month'] = df_data['Year'].astype(str) + '-' + df_data['Month'].astype(str).str.zfill(2)
    
    # Pivot the data
    pivot_df = df_data.pivot_table(
        index=['Customer #', 'Customer Name'],
        columns='Year-Month',
        values='Sales Amount',
        aggfunc='sum',
        fill_value=0
    )
    
    # Sort columns chronologically
    pivot_df = pivot_df.reindex(sorted(pivot_df.columns), axis=1)
    pivot_df = pivot_df.reset_index()
    
    # Add yearly totals
    year_month_columns = [col for col in pivot_df.columns if '-' in str(col)]
    years = sorted(set(col.split('-')[0] for col in year_month_columns))
    
    for year in years:
        year_cols = [col for col in year_month_columns if col.startswith(f"{year}-")]
        pivot_df[f'{year} Total'] = pivot_df[year_cols].sum(axis=1)
    
    # Add grand total column
    pivot_df['Grand Total'] = pivot_df[year_month_columns].sum(axis=1)
    
    print(f"\nPivot result: {len(pivot_df)} customers x {len(year_month_columns)} year-months")
    
    # ============================================================
    # STEP 2: Join with customer list to add Sales Rep and Channel
    # ============================================================
    print("\n" + "=" * 60)
    print("STEP 2: Joining with customer list...")
    print("=" * 60)
    
    customer_df = pd.read_excel(customer_list_file)
    print(f"Customer list: {len(customer_df)} rows")
    print(f"Customer columns: {list(customer_df.columns)}")
    
    # Find Sales Rep column
    sales_rep_col = None
    for col in customer_df.columns:
        if 'sales' in col.lower() and 'rep' in col.lower():
            sales_rep_col = col
            break
    if sales_rep_col:
        print(f"Found Sales Rep column: '{sales_rep_col}'")
    
    # Find Channel column
    channel_col = None
    for col in customer_df.columns:
        if 'channel' in col.lower():
            channel_col = col
            break
    if channel_col:
        print(f"Found Channel column: '{channel_col}'")
    
    # Build list of columns to join
    customer_cols = ['Customer #']
    if sales_rep_col:
        customer_cols.append(sales_rep_col)
    if channel_col:
        customer_cols.append(channel_col)
    
    # Create subset and remove duplicates
    customer_subset = customer_df[customer_cols].copy()
    customer_subset = customer_subset.drop_duplicates(subset=['Customer #'])
    
    # Perform left join
    result_df = pivot_df.merge(
        customer_subset,
        on='Customer #',
        how='left'
    )
    
    # Reorder columns to put Sales Rep and Channel after Customer Name
    cols = list(result_df.columns)
    
    if sales_rep_col and sales_rep_col in cols:
        cols.remove(sales_rep_col)
    if channel_col and channel_col in cols:
        cols.remove(channel_col)
    
    # Insert after Customer Name (index 1, so insert at 2)
    insert_pos = 2
    if channel_col:
        cols.insert(insert_pos, channel_col)
    if sales_rep_col:
        cols.insert(insert_pos, sales_rep_col)
    
    result_df = result_df[cols]
    
    # Sort by Grand Total descending
    result_df = result_df.sort_values('Grand Total', ascending=False)
    
    # ============================================================
    # STEP 3: Report statistics and save
    # ============================================================
    print("\n" + "=" * 60)
    print("STEP 3: Final statistics")
    print("=" * 60)
    
    if sales_rep_col:
        matched_rep = result_df[sales_rep_col].notna().sum()
        print(f"Matched Sales Rep: {matched_rep}/{len(result_df)}")
    
    if channel_col:
        matched_channel = result_df[channel_col].notna().sum()
        print(f"Matched Channel: {matched_channel}/{len(result_df)}")
        
        print(f"\nChannel distribution:")
        channel_counts = result_df[channel_col].value_counts(dropna=False)
        for channel, count in channel_counts.head(10).items():
            print(f"  {channel}: {count}")
    
    # Save to file
    if output_file:
        result_df.to_excel(output_file, index=False)
        print(f"\n✓ Result saved to: {output_file}")
    
    print(f"\nFinal result: {len(result_df)} customers x {len(result_df.columns)} columns")
    
    return result_df


# Run the combined function
result = generate_sales_by_customer_with_channel(
    sales_file="sales detail 2024-2025.xls",
    customer_list_file="SalesCustomerList_with_Channel.xlsx",
    output_file="Sales_by_Customer_Year_Month_with_Channel.xlsx"
)

# Display preview
print("\n" + "=" * 60)
print("RESULT PREVIEW (first 10 rows, first 8 columns):")
print("=" * 60)
print(result.iloc[:10, :8].to_string())

STEP 1: Reading and parsing sales data...
Parsed 55468 transaction rows
Unique customers: 351
Years found: [np.int64(2024), np.int64(2025)]
Months found: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12)]

Pivot result: 351 customers x 23 year-months

STEP 2: Joining with customer list...
Customer list: 660 rows
Customer columns: ['Count', 'Customer #', 'Customer Description', 'Sales Rep', 'Grade', 'Penetration', 'Call/Review Cycle', 'Contact name', 'Contact phone number', 'Contact email', 'Alternate contact', 'Notes', 'Channel']
Found Sales Rep column: 'Sales Rep'
Found Channel column: 'Channel'

STEP 3: Final statistics
Matched Sales Rep: 333/351
Matched Channel: 332/351

Channel distribution:
  Retail Group: 187
  Food Service: 85
  Liquor Group: 21
  nan: 19
  Sub - Distributor: 17
  Mainstream: 10
  Staff: 4
  Indent: 4
  Cash Sales: 2
  Food Ethnic: 2

✓ Result saved to: S

In [4]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
import matplotlib.pyplot as plt
import numpy as np
import io
from openpyxl.drawing.image import Image as XLImage

def generate_top10_customers_by_channel(
    sales_file: str = "sales detail 2024-2025.xls",
    customer_list_file: str = "SalesCustomerList_with_Channel.xlsx",
    target_month: int = 10,
    target_year: int = 2025,
    output_file: str = "Top10_Customers_by_Channel.xlsx"
) -> dict:
    """
    Generate Top 10 Customer performance reports by Channel for both MTD and YTD.
    
    Args:
        sales_file: Path to the sales details Excel file
        customer_list_file: Path to the SalesCustomerList_with_Channel.xlsx file
        target_month: Month for MTD calculation
        target_year: Year for calculations
        output_file: Path to save the Excel report
    
    Returns:
        Dictionary with channel-wise top 10 customer data for MTD and YTD
    """
    prev_year = target_year - 1
    
    # ============================================================
    # STEP 1: Parse sales data
    # ============================================================
    print("=" * 70)
    print("STEP 1: Reading and parsing sales data...")
    print("=" * 70)
    
    df_raw = pd.read_excel(sales_file, header=None)
    
    col_period = 1
    col_sales_amount = 17
    
    data_rows = []
    current_customer = None
    current_customer_name = None
    
    for idx in range(11, len(df_raw)):
        row = df_raw.iloc[idx]
        first_cell = row.iloc[0]
        
        if pd.isna(first_cell):
            continue
        
        first_cell_str = str(first_cell).strip()
        
        if first_cell_str and not first_cell_str.isdigit() and not first_cell_str.startswith('Item'):
            if any(c.isalpha() for c in first_cell_str) and any(c.isdigit() for c in first_cell_str):
                current_customer = first_cell_str
                cust_name = row.iloc[7] if pd.notna(row.iloc[7]) else ''
                current_customer_name = str(cust_name).strip() if cust_name else current_customer
                continue
        
        if first_cell_str.isdigit() and len(first_cell_str) == 4:
            year = int(first_cell_str)
            
            period_cell = row.iloc[col_period]
            month = None
            if pd.notna(period_cell):
                try:
                    month = int(float(period_cell))
                except (ValueError, TypeError):
                    month = None
            
            sales_amount = row.iloc[col_sales_amount]
            
            if pd.notna(sales_amount) and current_customer and month is not None:
                try:
                    sales_value = float(sales_amount)
                    data_rows.append({
                        'Customer #': current_customer,
                        'Customer Name': current_customer_name,
                        'Year': year,
                        'Month': month,
                        'Sales Amount': sales_value
                    })
                except (ValueError, TypeError):
                    pass
    
    df_sales = pd.DataFrame(data_rows)
    
    if df_sales.empty:
        print("No sales data found!")
        return {}
    
    print(f"Parsed {len(df_sales)} transaction rows")
    print(f"Unique customers: {df_sales['Customer #'].nunique()}")
    
    # ============================================================
    # STEP 2: Join with customer list to get Channel
    # ============================================================
    print("\n" + "=" * 70)
    print("STEP 2: Joining with customer list to get Channel...")
    print("=" * 70)
    
    customer_df = pd.read_excel(customer_list_file)
    
    # Find Channel and Sales Rep columns
    channel_col = None
    sales_rep_col = None
    for col in customer_df.columns:
        if 'channel' in col.lower():
            channel_col = col
        if 'sales' in col.lower() and 'rep' in col.lower():
            sales_rep_col = col
    
    if not channel_col:
        print("ERROR: Channel column not found in customer list!")
        return {}
    
    print(f"Found Channel column: '{channel_col}'")
    if sales_rep_col:
        print(f"Found Sales Rep column: '{sales_rep_col}'")
    
    # Create lookup for customer -> channel/sales rep
    customer_cols = ['Customer #', channel_col]
    if sales_rep_col:
        customer_cols.append(sales_rep_col)
    
    customer_lookup = customer_df[customer_cols].drop_duplicates(subset=['Customer #'])
    
    # Merge to add channel to sales data
    df_sales = df_sales.merge(customer_lookup, on='Customer #', how='left')
    
    # Fill missing channels with 'Unknown'
    df_sales[channel_col] = df_sales[channel_col].fillna('Unknown')
    
    print(f"\nChannel distribution in sales:")
    channel_counts = df_sales.groupby(channel_col)['Customer #'].nunique()
    for channel, count in channel_counts.items():
        print(f"  {channel}: {count} customers")
    
    # ============================================================
    # STEP 3: Calculate MTD and YTD metrics by customer
    # ============================================================
    print("\n" + "=" * 70)
    print("STEP 3: Calculating MTD and YTD metrics by customer...")
    print("=" * 70)
    
    # Filter for target years
    df_current = df_sales[df_sales['Year'] == target_year].copy()
    df_prev = df_sales[df_sales['Year'] == prev_year].copy()
    
    # Calculate MTD (specific month only)
    df_mtd_current = df_current[df_current['Month'] == target_month]
    df_mtd_prev = df_prev[df_prev['Month'] == target_month]
    
    # Calculate YTD (months 1 to target_month)
    df_ytd_current = df_current[df_current['Month'] <= target_month]
    df_ytd_prev = df_prev[df_prev['Month'] <= target_month]
    
    # Aggregate by customer
    def aggregate_by_customer(df, channel_col, sales_rep_col):
        agg_cols = ['Customer #', 'Customer Name', channel_col]
        if sales_rep_col and sales_rep_col in df.columns:
            agg_cols.append(sales_rep_col)
        
        result = df.groupby(agg_cols, dropna=False).agg({
            'Sales Amount': 'sum'
        }).reset_index()
        return result
    
    mtd_current_agg = aggregate_by_customer(df_mtd_current, channel_col, sales_rep_col)
    mtd_prev_agg = aggregate_by_customer(df_mtd_prev, channel_col, sales_rep_col)
    ytd_current_agg = aggregate_by_customer(df_ytd_current, channel_col, sales_rep_col)
    ytd_prev_agg = aggregate_by_customer(df_ytd_prev, channel_col, sales_rep_col)
    
    # Rename columns
    mtd_current_agg = mtd_current_agg.rename(columns={'Sales Amount': f'{target_year}_MTD_Sales'})
    mtd_prev_agg = mtd_prev_agg.rename(columns={'Sales Amount': f'{prev_year}_MTD_Sales'})
    ytd_current_agg = ytd_current_agg.rename(columns={'Sales Amount': f'{target_year}_YTD_Sales'})
    ytd_prev_agg = ytd_prev_agg.rename(columns={'Sales Amount': f'{prev_year}_YTD_Sales'})
    
    # Merge all metrics
    merge_cols = ['Customer #', 'Customer Name', channel_col]
    if sales_rep_col and sales_rep_col in mtd_current_agg.columns:
        merge_cols.append(sales_rep_col)
    
    # Start with MTD current
    customer_metrics = mtd_current_agg.copy()
    
    # Merge MTD previous
    prev_merge_cols = ['Customer #']
    customer_metrics = customer_metrics.merge(
        mtd_prev_agg[['Customer #', f'{prev_year}_MTD_Sales']],
        on='Customer #',
        how='outer'
    )
    
    # Merge YTD current
    customer_metrics = customer_metrics.merge(
        ytd_current_agg[['Customer #', f'{target_year}_YTD_Sales']],
        on='Customer #',
        how='outer'
    )
    
    # Merge YTD previous
    customer_metrics = customer_metrics.merge(
        ytd_prev_agg[['Customer #', f'{prev_year}_YTD_Sales']],
        on='Customer #',
        how='outer'
    )
    
    # Fill NaN values with 0 for sales columns
    sales_cols = [f'{target_year}_MTD_Sales', f'{prev_year}_MTD_Sales', 
                  f'{target_year}_YTD_Sales', f'{prev_year}_YTD_Sales']
    for col in sales_cols:
        if col in customer_metrics.columns:
            customer_metrics[col] = customer_metrics[col].fillna(0)
    
    # Fill customer info from any available source
    # Get complete customer info
    all_customers = pd.concat([
        df_sales[['Customer #', 'Customer Name', channel_col] + 
                 ([sales_rep_col] if sales_rep_col else [])]
    ]).drop_duplicates(subset=['Customer #'])
    
    # Update missing customer info
    customer_metrics = customer_metrics.drop(columns=['Customer Name', channel_col] + 
                                              ([sales_rep_col] if sales_rep_col and sales_rep_col in customer_metrics.columns else []), 
                                              errors='ignore')
    customer_metrics = customer_metrics.merge(all_customers, on='Customer #', how='left')
    
    # Calculate % Achieved
    customer_metrics['MTD_Achieved_%'] = customer_metrics.apply(
        lambda x: (x[f'{target_year}_MTD_Sales'] / x[f'{prev_year}_MTD_Sales'] * 100) 
        if x[f'{prev_year}_MTD_Sales'] > 0 else 0, axis=1
    )
    customer_metrics['YTD_Achieved_%'] = customer_metrics.apply(
        lambda x: (x[f'{target_year}_YTD_Sales'] / x[f'{prev_year}_YTD_Sales'] * 100) 
        if x[f'{prev_year}_YTD_Sales'] > 0 else 0, axis=1
    )
    
    print(f"Total customers with metrics: {len(customer_metrics)}")
    
    # ============================================================
    # STEP 4: Generate Top 10 by Channel for MTD and YTD
    # ============================================================
    print("\n" + "=" * 70)
    print("STEP 4: Generating Top 10 customers by Channel...")
    print("=" * 70)
    
    channels = customer_metrics[channel_col].dropna().unique()
    channels = [c for c in channels if c != 'Unknown']
    channels = sorted(channels)
    
    print(f"Channels found: {channels}")
    
    results = {}
    
    for channel in channels:
        channel_data = customer_metrics[customer_metrics[channel_col] == channel].copy()
        
        if channel_data.empty:
            continue
        
        # Top 10 MTD
        top10_mtd = channel_data.nlargest(10, f'{target_year}_MTD_Sales')
        
        # Top 10 YTD
        top10_ytd = channel_data.nlargest(10, f'{target_year}_YTD_Sales')
        
        results[channel] = {
            'MTD': top10_mtd,
            'YTD': top10_ytd
        }
        
        print(f"\n{channel}:")
        print(f"  Total customers: {len(channel_data)}")
        print(f"  Top MTD customer: {top10_mtd.iloc[0]['Customer Name'] if len(top10_mtd) > 0 else 'N/A'}")
        print(f"  Top YTD customer: {top10_ytd.iloc[0]['Customer Name'] if len(top10_ytd) > 0 else 'N/A'}")
    
    # ============================================================
    # STEP 5: Export to Excel with formatting
    # ============================================================
    print("\n" + "=" * 70)
    print("STEP 5: Creating Excel report...")
    print("=" * 70)
    
    wb = Workbook()
    # Remove default sheet
    wb.remove(wb.active)
    
    # Styling
    header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
    header_font = Font(bold=True, color="FFFFFF", size=10)
    title_font = Font(bold=True, size=14)
    border = Border(
        left=Side(style='thin'),
        right=Side(style='thin'),
        top=Side(style='thin'),
        bottom=Side(style='thin')
    )
    
    # Color fills for different sections
    current_year_fill = PatternFill(start_color="E2EFDA", end_color="E2EFDA", fill_type="solid")
    prev_year_fill = PatternFill(start_color="FCE4D6", end_color="FCE4D6", fill_type="solid")
    achieved_fill = PatternFill(start_color="FFF2CC", end_color="FFF2CC", fill_type="solid")
    
    for channel, data in results.items():
        # Create sheet for this channel
        clean_channel = "".join(c if c.isalnum() or c in (' ', '-', '_') else '_' for c in channel)[:30]
        ws = wb.create_sheet(title=clean_channel)
        
        # ---- MTD Section ----
        ws.merge_cells('A1:H1')
        ws['A1'] = f"Top 10 MTD Customer Performance - {channel} (Month {target_month}/{target_year})"
        ws['A1'].font = title_font
        ws['A1'].alignment = Alignment(horizontal='center')
        
        # MTD Headers
        mtd_headers = ['Rank', 'Customer #', 'Customer Name', 
                       f'{target_year} MTD Sales', f'{prev_year} MTD Sales', '% Achieved']
        if sales_rep_col:
            mtd_headers.insert(3, 'Sales Rep')
        
        for col_idx, header in enumerate(mtd_headers, start=1):
            cell = ws.cell(row=3, column=col_idx, value=header)
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
            cell.border = border
        
        # MTD Data
        mtd_df = data['MTD']
        for row_idx, (_, row) in enumerate(mtd_df.iterrows(), start=4):
            rank = row_idx - 3
            
            col = 1
            # Rank
            cell = ws.cell(row=row_idx, column=col, value=rank)
            cell.border = border
            cell.alignment = Alignment(horizontal='center')
            col += 1
            
            # Customer #
            cell = ws.cell(row=row_idx, column=col, value=row['Customer #'])
            cell.border = border
            col += 1
            
            # Customer Name
            cell = ws.cell(row=row_idx, column=col, value=row['Customer Name'])
            cell.border = border
            col += 1
            
            # Sales Rep (if available)
            if sales_rep_col:
                cell = ws.cell(row=row_idx, column=col, value=row.get(sales_rep_col, ''))
                cell.border = border
                col += 1
            
            # Current Year MTD Sales
            cell = ws.cell(row=row_idx, column=col, value=row[f'{target_year}_MTD_Sales'])
            cell.number_format = '$#,##0.00'
            cell.fill = current_year_fill
            cell.border = border
            cell.alignment = Alignment(horizontal='right')
            col += 1
            
            # Previous Year MTD Sales
            cell = ws.cell(row=row_idx, column=col, value=row[f'{prev_year}_MTD_Sales'])
            cell.number_format = '$#,##0.00'
            cell.fill = prev_year_fill
            cell.border = border
            cell.alignment = Alignment(horizontal='right')
            col += 1
            
            # % Achieved
            cell = ws.cell(row=row_idx, column=col, value=row['MTD_Achieved_%'] / 100)
            cell.number_format = '0.00%'
            cell.fill = achieved_fill
            cell.border = border
            cell.alignment = Alignment(horizontal='right')
        
        # ---- YTD Section ----
        ytd_start_row = 4 + len(mtd_df) + 3
        
        ws.merge_cells(f'A{ytd_start_row}:H{ytd_start_row}')
        ws.cell(row=ytd_start_row, column=1, 
                value=f"Top 10 YTD Customer Performance - {channel} (Jan-{target_month}/{target_year})")
        ws.cell(row=ytd_start_row, column=1).font = title_font
        ws.cell(row=ytd_start_row, column=1).alignment = Alignment(horizontal='center')
        
        # YTD Headers
        ytd_headers = ['Rank', 'Customer #', 'Customer Name', 
                       f'{target_year} YTD Sales', f'{prev_year} YTD Sales', '% Achieved']
        if sales_rep_col:
            ytd_headers.insert(3, 'Sales Rep')
        
        header_row = ytd_start_row + 2
        for col_idx, header in enumerate(ytd_headers, start=1):
            cell = ws.cell(row=header_row, column=col_idx, value=header)
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
            cell.border = border
        
        # YTD Data
        ytd_df = data['YTD']
        for row_idx, (_, row) in enumerate(ytd_df.iterrows(), start=header_row + 1):
            rank = row_idx - header_row
            
            col = 1
            # Rank
            cell = ws.cell(row=row_idx, column=col, value=rank)
            cell.border = border
            cell.alignment = Alignment(horizontal='center')
            col += 1
            
            # Customer #
            cell = ws.cell(row=row_idx, column=col, value=row['Customer #'])
            cell.border = border
            col += 1
            
            # Customer Name
            cell = ws.cell(row=row_idx, column=col, value=row['Customer Name'])
            cell.border = border
            col += 1
            
            # Sales Rep (if available)
            if sales_rep_col:
                cell = ws.cell(row=row_idx, column=col, value=row.get(sales_rep_col, ''))
                cell.border = border
                col += 1
            
            # Current Year YTD Sales
            cell = ws.cell(row=row_idx, column=col, value=row[f'{target_year}_YTD_Sales'])
            cell.number_format = '$#,##0.00'
            cell.fill = current_year_fill
            cell.border = border
            cell.alignment = Alignment(horizontal='right')
            col += 1
            
            # Previous Year YTD Sales
            cell = ws.cell(row=row_idx, column=col, value=row[f'{prev_year}_YTD_Sales'])
            cell.number_format = '$#,##0.00'
            cell.fill = prev_year_fill
            cell.border = border
            cell.alignment = Alignment(horizontal='right')
            col += 1
            
            # % Achieved
            cell = ws.cell(row=row_idx, column=col, value=row['YTD_Achieved_%'] / 100)
            cell.number_format = '0.00%'
            cell.fill = achieved_fill
            cell.border = border
            cell.alignment = Alignment(horizontal='right')
        
        # Adjust column widths
        ws.column_dimensions['A'].width = 6
        ws.column_dimensions['B'].width = 12
        ws.column_dimensions['C'].width = 30
        if sales_rep_col:
            ws.column_dimensions['D'].width = 15
            ws.column_dimensions['E'].width = 18
            ws.column_dimensions['F'].width = 18
            ws.column_dimensions['G'].width = 12
        else:
            ws.column_dimensions['D'].width = 18
            ws.column_dimensions['E'].width = 18
            ws.column_dimensions['F'].width = 12
    
    # ============================================================
    # STEP 6: Add Summary Sheet with Charts
    # ============================================================
    ws_summary = wb.create_sheet(title="Summary", index=0)
    
    ws_summary.merge_cells('A1:F1')
    ws_summary['A1'] = f"Top 10 Customer Performance by Channel - Summary ({target_month}/{target_year})"
    ws_summary['A1'].font = Font(bold=True, size=16)
    ws_summary['A1'].alignment = Alignment(horizontal='center')
    
    # Summary table
    summary_headers = ['Channel', 'Total Customers', f'{target_year} MTD Total', 
                       f'{prev_year} MTD Total', f'{target_year} YTD Total', f'{prev_year} YTD Total']
    
    for col_idx, header in enumerate(summary_headers, start=1):
        cell = ws_summary.cell(row=3, column=col_idx, value=header)
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = Alignment(horizontal='center')
        cell.border = border
    
    row_idx = 4
    for channel in channels:
        channel_data = customer_metrics[customer_metrics[channel_col] == channel]
        
        ws_summary.cell(row=row_idx, column=1, value=channel).border = border
        ws_summary.cell(row=row_idx, column=2, value=len(channel_data)).border = border
        
        cell = ws_summary.cell(row=row_idx, column=3, value=channel_data[f'{target_year}_MTD_Sales'].sum())
        cell.number_format = '$#,##0.00'
        cell.border = border
        
        cell = ws_summary.cell(row=row_idx, column=4, value=channel_data[f'{prev_year}_MTD_Sales'].sum())
        cell.number_format = '$#,##0.00'
        cell.border = border
        
        cell = ws_summary.cell(row=row_idx, column=5, value=channel_data[f'{target_year}_YTD_Sales'].sum())
        cell.number_format = '$#,##0.00'
        cell.border = border
        
        cell = ws_summary.cell(row=row_idx, column=6, value=channel_data[f'{prev_year}_YTD_Sales'].sum())
        cell.number_format = '$#,##0.00'
        cell.border = border
        
        row_idx += 1
    
    # Adjust column widths for summary
    ws_summary.column_dimensions['A'].width = 20
    ws_summary.column_dimensions['B'].width = 15
    ws_summary.column_dimensions['C'].width = 18
    ws_summary.column_dimensions['D'].width = 18
    ws_summary.column_dimensions['E'].width = 18
    ws_summary.column_dimensions['F'].width = 18
    
    # Create summary chart
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    fig.suptitle(f'Sales by Channel - {target_month}/{target_year}', fontsize=14, fontweight='bold')
    
    channel_totals = []
    for channel in channels:
        channel_data = customer_metrics[customer_metrics[channel_col] == channel]
        channel_totals.append({
            'Channel': channel,
            f'{target_year}_MTD': channel_data[f'{target_year}_MTD_Sales'].sum(),
            f'{prev_year}_MTD': channel_data[f'{prev_year}_MTD_Sales'].sum(),
            f'{target_year}_YTD': channel_data[f'{target_year}_YTD_Sales'].sum(),
            f'{prev_year}_YTD': channel_data[f'{prev_year}_YTD_Sales'].sum()
        })
    
    chart_df = pd.DataFrame(channel_totals)
    
    # MTD Chart
    x = np.arange(len(channels))
    width = 0.35
    
    bars1 = axes[0].bar(x - width/2, chart_df[f'{target_year}_MTD'], width, 
                        label=str(target_year), color='#2E86AB', alpha=0.8)
    bars2 = axes[0].bar(x + width/2, chart_df[f'{prev_year}_MTD'], width, 
                        label=str(prev_year), color='#A23B72', alpha=0.8)
    
    axes[0].set_title('MTD Sales by Channel', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Sales Amount ($)')
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(channels, rotation=45, ha='right')
    axes[0].legend()
    axes[0].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))
    axes[0].grid(axis='y', alpha=0.3)
    
    # YTD Chart
    bars3 = axes[1].bar(x - width/2, chart_df[f'{target_year}_YTD'], width, 
                        label=str(target_year), color='#2E86AB', alpha=0.8)
    bars4 = axes[1].bar(x + width/2, chart_df[f'{prev_year}_YTD'], width, 
                        label=str(prev_year), color='#A23B72', alpha=0.8)
    
    axes[1].set_title('YTD Sales by Channel', fontsize=12, fontweight='bold')
    axes[1].set_ylabel('Sales Amount ($)')
    axes[1].set_xticks(x)
    axes[1].set_xticklabels(channels, rotation=45, ha='right')
    axes[1].legend()
    axes[1].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))
    axes[1].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    
    # Save chart to buffer and add to Excel
    img_buffer = io.BytesIO()
    fig.savefig(img_buffer, format='png', dpi=150, bbox_inches='tight')
    img_buffer.seek(0)
    plt.close(fig)
    
    img = XLImage(img_buffer)
    ws_summary.add_image(img, f'A{row_idx + 3}')
    
    # Save workbook
    wb.save(output_file)
    
    print(f"\n✓ Excel report saved to: {output_file}")
    print(f"  - Summary sheet with channel comparison")
    for channel in channels:
        print(f"  - {channel} sheet with Top 10 MTD and YTD customers")
    
    return results


# Run the function
results = generate_top10_customers_by_channel(
    sales_file="sales detail 2024-2025.xls",
    customer_list_file="SalesCustomerList_with_Channel.xlsx",
    target_month=10,
    target_year=2025,
    output_file="Top10_Customers_by_Channel.xlsx"
)

# Display preview for each channel
print("\n" + "=" * 70)
print("PREVIEW OF TOP 10 CUSTOMERS BY CHANNEL")
print("=" * 70)

for channel, data in results.items():
    print(f"\n{'='*50}")
    print(f"CHANNEL: {channel}")
    print(f"{'='*50}")
    
    print("\nTop 5 MTD Customers:")
    mtd_preview = data['MTD'][['Customer #', 'Customer Name', '2025_MTD_Sales', 'MTD_Achieved_%']].head(5)
    print(mtd_preview.to_string(index=False))
    
    print("\nTop 5 YTD Customers:")
    ytd_preview = data['YTD'][['Customer #', 'Customer Name', '2025_YTD_Sales', 'YTD_Achieved_%']].head(5)
    print(ytd_preview.to_string(index=False))

STEP 1: Reading and parsing sales data...
Parsed 55468 transaction rows
Unique customers: 351

STEP 2: Joining with customer list to get Channel...
Found Channel column: 'Channel'
Found Sales Rep column: 'Sales Rep'

Channel distribution in sales:
  Cash Sales: 2 customers
  Food Ethnic: 2 customers
  Food Service: 85 customers
  Indent: 4 customers
  Liquor Group: 21 customers
  Mainstream: 10 customers
  Retail Group: 187 customers
  Staff: 4 customers
  Sub - Distributor: 17 customers
  Unknown: 19 customers

STEP 3: Calculating MTD and YTD metrics by customer...
Total customers with metrics: 343

STEP 4: Generating Top 10 customers by Channel...
Channels found: ['Cash Sales', 'Food Ethnic', 'Food Service', 'Indent', 'Liquor Group', 'Mainstream', 'Retail Group', 'Staff', 'Sub - Distributor']

Cash Sales:
  Total customers: 2
  Top MTD customer: Cash Sales
  Top YTD customer: Cash Sales

Food Ethnic:
  Total customers: 2
  Top MTD customer: Fast Food Chinese & European Takeaway - YE 

In [5]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
import matplotlib.pyplot as plt
import numpy as np
import io
from openpyxl.drawing.image import Image as XLImage

def generate_top10_customers_by_channel(
    sales_file: str = "sales detail 2024-2025.xls",
    customer_list_file: str = "SalesCustomerList_with_Channel.xlsx",
    target_month: int = 10,
    target_year: int = 2025,
    output_file: str = "Top10_Customers_by_Channel.xlsx"
) -> dict:
    """
    Generate Top 10 Customer performance reports by Channel for both MTD and YTD.
    Also calculates MTD/YTD totals for each Channel.
    
    Args:
        sales_file: Path to the sales details Excel file
        customer_list_file: Path to the SalesCustomerList_with_Channel.xlsx file
        target_month: Month for MTD calculation
        target_year: Year for calculations
        output_file: Path to save the Excel report
    
    Returns:
        Dictionary with channel-wise top 10 customer data for MTD and YTD,
        plus channel totals
    """
    prev_year = target_year - 1
    
    # ============================================================
    # STEP 1: Parse sales data
    # ============================================================
    print("=" * 70)
    print("STEP 1: Reading and parsing sales data...")
    print("=" * 70)
    
    df_raw = pd.read_excel(sales_file, header=None)
    
    col_period = 1
    col_sales_amount = 17
    
    data_rows = []
    current_customer = None
    current_customer_name = None
    
    for idx in range(11, len(df_raw)):
        row = df_raw.iloc[idx]
        first_cell = row.iloc[0]
        
        if pd.isna(first_cell):
            continue
        
        first_cell_str = str(first_cell).strip()
        
        if first_cell_str and not first_cell_str.isdigit() and not first_cell_str.startswith('Item'):
            if any(c.isalpha() for c in first_cell_str) and any(c.isdigit() for c in first_cell_str):
                current_customer = first_cell_str
                cust_name = row.iloc[7] if pd.notna(row.iloc[7]) else ''
                current_customer_name = str(cust_name).strip() if cust_name else current_customer
                continue
        
        if first_cell_str.isdigit() and len(first_cell_str) == 4:
            year = int(first_cell_str)
            
            period_cell = row.iloc[col_period]
            month = None
            if pd.notna(period_cell):
                try:
                    month = int(float(period_cell))
                except (ValueError, TypeError):
                    month = None
            
            sales_amount = row.iloc[col_sales_amount]
            
            if pd.notna(sales_amount) and current_customer and month is not None:
                try:
                    sales_value = float(sales_amount)
                    data_rows.append({
                        'Customer #': current_customer,
                        'Customer Name': current_customer_name,
                        'Year': year,
                        'Month': month,
                        'Sales Amount': sales_value
                    })
                except (ValueError, TypeError):
                    pass
    
    df_sales = pd.DataFrame(data_rows)
    
    if df_sales.empty:
        print("No sales data found!")
        return {}
    
    print(f"Parsed {len(df_sales)} transaction rows")
    print(f"Unique customers: {df_sales['Customer #'].nunique()}")
    
    # ============================================================
    # STEP 2: Join with customer list to get Channel
    # ============================================================
    print("\n" + "=" * 70)
    print("STEP 2: Joining with customer list to get Channel...")
    print("=" * 70)
    
    customer_df = pd.read_excel(customer_list_file)
    
    # Find Channel and Sales Rep columns
    channel_col = None
    sales_rep_col = None
    for col in customer_df.columns:
        if 'channel' in col.lower():
            channel_col = col
        if 'sales' in col.lower() and 'rep' in col.lower():
            sales_rep_col = col
    
    if not channel_col:
        print("ERROR: Channel column not found in customer list!")
        return {}
    
    print(f"Found Channel column: '{channel_col}'")
    if sales_rep_col:
        print(f"Found Sales Rep column: '{sales_rep_col}'")
    
    # Create lookup for customer -> channel/sales rep
    customer_cols = ['Customer #', channel_col]
    if sales_rep_col:
        customer_cols.append(sales_rep_col)
    
    customer_lookup = customer_df[customer_cols].drop_duplicates(subset=['Customer #'])
    
    # Merge to add channel to sales data
    df_sales = df_sales.merge(customer_lookup, on='Customer #', how='left')
    
    # Fill missing channels with 'Unknown'
    df_sales[channel_col] = df_sales[channel_col].fillna('Unknown')
    
    print(f"\nChannel distribution in sales:")
    channel_counts = df_sales.groupby(channel_col)['Customer #'].nunique()
    for channel, count in channel_counts.items():
        print(f"  {channel}: {count} customers")
    
    # ============================================================
    # STEP 3: Calculate MTD and YTD metrics by customer
    # ============================================================
    print("\n" + "=" * 70)
    print("STEP 3: Calculating MTD and YTD metrics by customer...")
    print("=" * 70)
    
    # Filter for target years
    df_current = df_sales[df_sales['Year'] == target_year].copy()
    df_prev = df_sales[df_sales['Year'] == prev_year].copy()
    
    # Calculate MTD (specific month only)
    df_mtd_current = df_current[df_current['Month'] == target_month]
    df_mtd_prev = df_prev[df_prev['Month'] == target_month]
    
    # Calculate YTD (months 1 to target_month)
    df_ytd_current = df_current[df_current['Month'] <= target_month]
    df_ytd_prev = df_prev[df_prev['Month'] <= target_month]
    
    # Aggregate by customer
    def aggregate_by_customer(df, channel_col, sales_rep_col):
        agg_cols = ['Customer #', 'Customer Name', channel_col]
        if sales_rep_col and sales_rep_col in df.columns:
            agg_cols.append(sales_rep_col)
        
        result = df.groupby(agg_cols, dropna=False).agg({
            'Sales Amount': 'sum'
        }).reset_index()
        return result
    
    mtd_current_agg = aggregate_by_customer(df_mtd_current, channel_col, sales_rep_col)
    mtd_prev_agg = aggregate_by_customer(df_mtd_prev, channel_col, sales_rep_col)
    ytd_current_agg = aggregate_by_customer(df_ytd_current, channel_col, sales_rep_col)
    ytd_prev_agg = aggregate_by_customer(df_ytd_prev, channel_col, sales_rep_col)
    
    # Rename columns
    mtd_current_agg = mtd_current_agg.rename(columns={'Sales Amount': f'{target_year}_MTD_Sales'})
    mtd_prev_agg = mtd_prev_agg.rename(columns={'Sales Amount': f'{prev_year}_MTD_Sales'})
    ytd_current_agg = ytd_current_agg.rename(columns={'Sales Amount': f'{target_year}_YTD_Sales'})
    ytd_prev_agg = ytd_prev_agg.rename(columns={'Sales Amount': f'{prev_year}_YTD_Sales'})
    
    # Start with MTD current
    customer_metrics = mtd_current_agg.copy()
    
    # Merge MTD previous
    customer_metrics = customer_metrics.merge(
        mtd_prev_agg[['Customer #', f'{prev_year}_MTD_Sales']],
        on='Customer #',
        how='outer'
    )
    
    # Merge YTD current
    customer_metrics = customer_metrics.merge(
        ytd_current_agg[['Customer #', f'{target_year}_YTD_Sales']],
        on='Customer #',
        how='outer'
    )
    
    # Merge YTD previous
    customer_metrics = customer_metrics.merge(
        ytd_prev_agg[['Customer #', f'{prev_year}_YTD_Sales']],
        on='Customer #',
        how='outer'
    )
    
    # Fill NaN values with 0 for sales columns
    sales_cols = [f'{target_year}_MTD_Sales', f'{prev_year}_MTD_Sales', 
                  f'{target_year}_YTD_Sales', f'{prev_year}_YTD_Sales']
    for col in sales_cols:
        if col in customer_metrics.columns:
            customer_metrics[col] = customer_metrics[col].fillna(0)
    
    # Get complete customer info
    all_customers = pd.concat([
        df_sales[['Customer #', 'Customer Name', channel_col] + 
                 ([sales_rep_col] if sales_rep_col else [])]
    ]).drop_duplicates(subset=['Customer #'])
    
    # Update missing customer info
    customer_metrics = customer_metrics.drop(columns=['Customer Name', channel_col] + 
                                              ([sales_rep_col] if sales_rep_col and sales_rep_col in customer_metrics.columns else []), 
                                              errors='ignore')
    customer_metrics = customer_metrics.merge(all_customers, on='Customer #', how='left')
    
    # Calculate % Achieved
    customer_metrics['MTD_Achieved_%'] = customer_metrics.apply(
        lambda x: (x[f'{target_year}_MTD_Sales'] / x[f'{prev_year}_MTD_Sales'] * 100) 
        if x[f'{prev_year}_MTD_Sales'] > 0 else 0, axis=1
    )
    customer_metrics['YTD_Achieved_%'] = customer_metrics.apply(
        lambda x: (x[f'{target_year}_YTD_Sales'] / x[f'{prev_year}_YTD_Sales'] * 100) 
        if x[f'{prev_year}_YTD_Sales'] > 0 else 0, axis=1
    )
    
    print(f"Total customers with metrics: {len(customer_metrics)}")
    
    # ============================================================
    # STEP 4: Calculate Channel-Level MTD/YTD Totals
    # ============================================================
    print("\n" + "=" * 70)
    print("STEP 4: Calculating Channel-Level MTD/YTD Totals...")
    print("=" * 70)
    
    # Aggregate by Channel
    channel_metrics = customer_metrics.groupby(channel_col).agg({
        f'{target_year}_MTD_Sales': 'sum',
        f'{prev_year}_MTD_Sales': 'sum',
        f'{target_year}_YTD_Sales': 'sum',
        f'{prev_year}_YTD_Sales': 'sum',
        'Customer #': 'count'  # Count of customers
    }).reset_index()
    
    channel_metrics = channel_metrics.rename(columns={'Customer #': 'Customer_Count'})
    
    # Calculate % Achieved for channels
    channel_metrics['MTD_Achieved_%'] = channel_metrics.apply(
        lambda x: (x[f'{target_year}_MTD_Sales'] / x[f'{prev_year}_MTD_Sales'] * 100) 
        if x[f'{prev_year}_MTD_Sales'] > 0 else 0, axis=1
    )
    channel_metrics['YTD_Achieved_%'] = channel_metrics.apply(
        lambda x: (x[f'{target_year}_YTD_Sales'] / x[f'{prev_year}_YTD_Sales'] * 100) 
        if x[f'{prev_year}_YTD_Sales'] > 0 else 0, axis=1
    )
    
    # Calculate YoY Growth
    channel_metrics['MTD_YoY_Growth'] = channel_metrics.apply(
        lambda x: ((x[f'{target_year}_MTD_Sales'] - x[f'{prev_year}_MTD_Sales']) / x[f'{prev_year}_MTD_Sales'] * 100) 
        if x[f'{prev_year}_MTD_Sales'] > 0 else 0, axis=1
    )
    channel_metrics['YTD_YoY_Growth'] = channel_metrics.apply(
        lambda x: ((x[f'{target_year}_YTD_Sales'] - x[f'{prev_year}_YTD_Sales']) / x[f'{prev_year}_YTD_Sales'] * 100) 
        if x[f'{prev_year}_YTD_Sales'] > 0 else 0, axis=1
    )
    
    # Print Channel Summary
    print(f"\n{'Channel':<20} {'Customers':>10} {f'{target_year} MTD':>15} {f'{prev_year} MTD':>15} {'MTD %':>10} {f'{target_year} YTD':>15} {f'{prev_year} YTD':>15} {'YTD %':>10}")
    print("-" * 120)
    
    for _, row in channel_metrics.iterrows():
        if row[channel_col] != 'Unknown':
            print(f"{row[channel_col]:<20} {row['Customer_Count']:>10} "
                  f"${row[f'{target_year}_MTD_Sales']:>13,.0f} ${row[f'{prev_year}_MTD_Sales']:>13,.0f} "
                  f"{row['MTD_Achieved_%']:>9.1f}% "
                  f"${row[f'{target_year}_YTD_Sales']:>13,.0f} ${row[f'{prev_year}_YTD_Sales']:>13,.0f} "
                  f"{row['YTD_Achieved_%']:>9.1f}%")
    
    # Calculate Grand Total
    grand_total = {
        'Channel': 'GRAND TOTAL',
        'Customer_Count': channel_metrics['Customer_Count'].sum(),
        f'{target_year}_MTD_Sales': channel_metrics[f'{target_year}_MTD_Sales'].sum(),
        f'{prev_year}_MTD_Sales': channel_metrics[f'{prev_year}_MTD_Sales'].sum(),
        f'{target_year}_YTD_Sales': channel_metrics[f'{target_year}_YTD_Sales'].sum(),
        f'{prev_year}_YTD_Sales': channel_metrics[f'{prev_year}_YTD_Sales'].sum(),
    }
    grand_total['MTD_Achieved_%'] = (grand_total[f'{target_year}_MTD_Sales'] / grand_total[f'{prev_year}_MTD_Sales'] * 100) if grand_total[f'{prev_year}_MTD_Sales'] > 0 else 0
    grand_total['YTD_Achieved_%'] = (grand_total[f'{target_year}_YTD_Sales'] / grand_total[f'{prev_year}_YTD_Sales'] * 100) if grand_total[f'{prev_year}_YTD_Sales'] > 0 else 0
    
    print("-" * 120)
    print(f"{'GRAND TOTAL':<20} {grand_total['Customer_Count']:>10} "
          f"${grand_total[f'{target_year}_MTD_Sales']:>13,.0f} ${grand_total[f'{prev_year}_MTD_Sales']:>13,.0f} "
          f"{grand_total['MTD_Achieved_%']:>9.1f}% "
          f"${grand_total[f'{target_year}_YTD_Sales']:>13,.0f} ${grand_total[f'{prev_year}_YTD_Sales']:>13,.0f} "
          f"{grand_total['YTD_Achieved_%']:>9.1f}%")
    
    # ============================================================
    # STEP 5: Generate Top 10 by Channel for MTD and YTD
    # ============================================================
    print("\n" + "=" * 70)
    print("STEP 5: Generating Top 10 customers by Channel...")
    print("=" * 70)
    
    channels = customer_metrics[channel_col].dropna().unique()
    channels = [c for c in channels if c != 'Unknown']
    channels = sorted(channels)
    
    print(f"Channels found: {channels}")
    
    results = {
        'channel_metrics': channel_metrics,
        'grand_total': grand_total,
        'top10_by_channel': {}
    }
    
    for channel in channels:
        channel_data = customer_metrics[customer_metrics[channel_col] == channel].copy()
        
        if channel_data.empty:
            continue
        
        # Top 10 MTD
        top10_mtd = channel_data.nlargest(10, f'{target_year}_MTD_Sales')
        
        # Top 10 YTD
        top10_ytd = channel_data.nlargest(10, f'{target_year}_YTD_Sales')
        
        results['top10_by_channel'][channel] = {
            'MTD': top10_mtd,
            'YTD': top10_ytd
        }
        
        print(f"\n{channel}:")
        print(f"  Total customers: {len(channel_data)}")
        print(f"  Top MTD customer: {top10_mtd.iloc[0]['Customer Name'] if len(top10_mtd) > 0 else 'N/A'}")
        print(f"  Top YTD customer: {top10_ytd.iloc[0]['Customer Name'] if len(top10_ytd) > 0 else 'N/A'}")
    
    # ============================================================
    # STEP 6: Export to Excel with formatting
    # ============================================================
    print("\n" + "=" * 70)
    print("STEP 6: Creating Excel report...")
    print("=" * 70)
    
    wb = Workbook()
    wb.remove(wb.active)
    
    # Styling
    header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
    header_font = Font(bold=True, color="FFFFFF", size=10)
    title_font = Font(bold=True, size=14)
    subtitle_font = Font(bold=True, size=12)
    border = Border(
        left=Side(style='thin'),
        right=Side(style='thin'),
        top=Side(style='thin'),
        bottom=Side(style='thin')
    )
    
    # Color fills
    current_year_fill = PatternFill(start_color="E2EFDA", end_color="E2EFDA", fill_type="solid")
    prev_year_fill = PatternFill(start_color="FCE4D6", end_color="FCE4D6", fill_type="solid")
    achieved_fill = PatternFill(start_color="FFF2CC", end_color="FFF2CC", fill_type="solid")
    total_fill = PatternFill(start_color="DDEBF7", end_color="DDEBF7", fill_type="solid")
    
    # ============================================================
    # Create Summary Sheet with Channel MTD/YTD Totals
    # ============================================================
    ws_summary = wb.create_sheet(title="Channel Summary", index=0)
    
    # Title
    ws_summary.merge_cells('A1:I1')
    ws_summary['A1'] = f"Channel Performance Summary - MTD/YTD ({target_month}/{target_year})"
    ws_summary['A1'].font = Font(bold=True, size=16)
    ws_summary['A1'].alignment = Alignment(horizontal='center')
    
    # Channel Summary Headers
    summary_headers = [
        'Channel', 'Customers',
        f'{target_year} MTD', f'{prev_year} MTD', 'MTD %', 'MTD YoY Growth',
        f'{target_year} YTD', f'{prev_year} YTD', 'YTD %', 'YTD YoY Growth'
    ]
    
    for col_idx, header in enumerate(summary_headers, start=1):
        cell = ws_summary.cell(row=3, column=col_idx, value=header)
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
        cell.border = border
    
    # Write Channel Data
    row_idx = 4
    for _, ch_row in channel_metrics.iterrows():
        if ch_row[channel_col] == 'Unknown':
            continue
        
        ws_summary.cell(row=row_idx, column=1, value=ch_row[channel_col]).border = border
        ws_summary.cell(row=row_idx, column=2, value=ch_row['Customer_Count']).border = border
        
        # MTD Current Year
        cell = ws_summary.cell(row=row_idx, column=3, value=ch_row[f'{target_year}_MTD_Sales'])
        cell.number_format = '$#,##0.00'
        cell.fill = current_year_fill
        cell.border = border
        
        # MTD Previous Year
        cell = ws_summary.cell(row=row_idx, column=4, value=ch_row[f'{prev_year}_MTD_Sales'])
        cell.number_format = '$#,##0.00'
        cell.fill = prev_year_fill
        cell.border = border
        
        # MTD Achieved %
        cell = ws_summary.cell(row=row_idx, column=5, value=ch_row['MTD_Achieved_%'] / 100)
        cell.number_format = '0.0%'
        cell.fill = achieved_fill
        cell.border = border
        
        # MTD YoY Growth
        cell = ws_summary.cell(row=row_idx, column=6, value=ch_row['MTD_YoY_Growth'] / 100)
        cell.number_format = '+0.0%;-0.0%'
        cell.border = border
        
        # YTD Current Year
        cell = ws_summary.cell(row=row_idx, column=7, value=ch_row[f'{target_year}_YTD_Sales'])
        cell.number_format = '$#,##0.00'
        cell.fill = current_year_fill
        cell.border = border
        
        # YTD Previous Year
        cell = ws_summary.cell(row=row_idx, column=8, value=ch_row[f'{prev_year}_YTD_Sales'])
        cell.number_format = '$#,##0.00'
        cell.fill = prev_year_fill
        cell.border = border
        
        # YTD Achieved %
        cell = ws_summary.cell(row=row_idx, column=9, value=ch_row['YTD_Achieved_%'] / 100)
        cell.number_format = '0.0%'
        cell.fill = achieved_fill
        cell.border = border
        
        # YTD YoY Growth
        cell = ws_summary.cell(row=row_idx, column=10, value=ch_row['YTD_YoY_Growth'] / 100)
        cell.number_format = '+0.0%;-0.0%'
        cell.border = border
        
        row_idx += 1
    
    # Grand Total Row
    ws_summary.cell(row=row_idx, column=1, value='GRAND TOTAL').font = Font(bold=True)
    ws_summary.cell(row=row_idx, column=1).fill = total_fill
    ws_summary.cell(row=row_idx, column=1).border = border
    
    ws_summary.cell(row=row_idx, column=2, value=grand_total['Customer_Count']).font = Font(bold=True)
    ws_summary.cell(row=row_idx, column=2).fill = total_fill
    ws_summary.cell(row=row_idx, column=2).border = border
    
    cell = ws_summary.cell(row=row_idx, column=3, value=grand_total[f'{target_year}_MTD_Sales'])
    cell.number_format = '$#,##0.00'
    cell.font = Font(bold=True)
    cell.fill = total_fill
    cell.border = border
    
    cell = ws_summary.cell(row=row_idx, column=4, value=grand_total[f'{prev_year}_MTD_Sales'])
    cell.number_format = '$#,##0.00'
    cell.font = Font(bold=True)
    cell.fill = total_fill
    cell.border = border
    
    cell = ws_summary.cell(row=row_idx, column=5, value=grand_total['MTD_Achieved_%'] / 100)
    cell.number_format = '0.0%'
    cell.font = Font(bold=True)
    cell.fill = total_fill
    cell.border = border
    
    mtd_growth = ((grand_total[f'{target_year}_MTD_Sales'] - grand_total[f'{prev_year}_MTD_Sales']) / grand_total[f'{prev_year}_MTD_Sales'] * 100) if grand_total[f'{prev_year}_MTD_Sales'] > 0 else 0
    cell = ws_summary.cell(row=row_idx, column=6, value=mtd_growth / 100)
    cell.number_format = '+0.0%;-0.0%'
    cell.font = Font(bold=True)
    cell.fill = total_fill
    cell.border = border
    
    cell = ws_summary.cell(row=row_idx, column=7, value=grand_total[f'{target_year}_YTD_Sales'])
    cell.number_format = '$#,##0.00'
    cell.font = Font(bold=True)
    cell.fill = total_fill
    cell.border = border
    
    cell = ws_summary.cell(row=row_idx, column=8, value=grand_total[f'{prev_year}_YTD_Sales'])
    cell.number_format = '$#,##0.00'
    cell.font = Font(bold=True)
    cell.fill = total_fill
    cell.border = border
    
    cell = ws_summary.cell(row=row_idx, column=9, value=grand_total['YTD_Achieved_%'] / 100)
    cell.number_format = '0.0%'
    cell.font = Font(bold=True)
    cell.fill = total_fill
    cell.border = border
    
    ytd_growth = ((grand_total[f'{target_year}_YTD_Sales'] - grand_total[f'{prev_year}_YTD_Sales']) / grand_total[f'{prev_year}_YTD_Sales'] * 100) if grand_total[f'{prev_year}_YTD_Sales'] > 0 else 0
    cell = ws_summary.cell(row=row_idx, column=10, value=ytd_growth / 100)
    cell.number_format = '+0.0%;-0.0%'
    cell.font = Font(bold=True)
    cell.fill = total_fill
    cell.border = border
    
    # Adjust column widths
    ws_summary.column_dimensions['A'].width = 18
    ws_summary.column_dimensions['B'].width = 12
    ws_summary.column_dimensions['C'].width = 16
    ws_summary.column_dimensions['D'].width = 16
    ws_summary.column_dimensions['E'].width = 10
    ws_summary.column_dimensions['F'].width = 14
    ws_summary.column_dimensions['G'].width = 16
    ws_summary.column_dimensions['H'].width = 16
    ws_summary.column_dimensions['I'].width = 10
    ws_summary.column_dimensions['J'].width = 14
    
    ws_summary.row_dimensions[3].height = 30
    
    # ============================================================
    # Add Charts to Summary Sheet
    # ============================================================
    chart_start_row = row_idx + 3
    
    # Create charts
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f'Channel Performance Analysis - {target_month}/{target_year}', fontsize=14, fontweight='bold')
    
    # Filter out Unknown channel for charts
    chart_data = channel_metrics[channel_metrics[channel_col] != 'Unknown'].copy()
    chart_channels = chart_data[channel_col].tolist()
    
    x = np.arange(len(chart_channels))
    width = 0.35
    
    # Chart 1: MTD Sales by Channel
    bars1 = axes[0, 0].bar(x - width/2, chart_data[f'{target_year}_MTD_Sales'], width, 
                           label=str(target_year), color='#2E86AB', alpha=0.8)
    bars2 = axes[0, 0].bar(x + width/2, chart_data[f'{prev_year}_MTD_Sales'], width, 
                           label=str(prev_year), color='#A23B72', alpha=0.8)
    axes[0, 0].set_title('MTD Sales by Channel', fontsize=11, fontweight='bold')
    axes[0, 0].set_ylabel('Sales Amount ($)')
    axes[0, 0].set_xticks(x)
    axes[0, 0].set_xticklabels(chart_channels, rotation=45, ha='right', fontsize=9)
    axes[0, 0].legend(fontsize=9)
    axes[0, 0].yaxis.set_major_formatter(plt.FuncFormatter(lambda v, p: f'${v/1000:.0f}K'))
    axes[0, 0].grid(axis='y', alpha=0.3)
    
    # Chart 2: YTD Sales by Channel
    bars3 = axes[0, 1].bar(x - width/2, chart_data[f'{target_year}_YTD_Sales'], width, 
                           label=str(target_year), color='#2E86AB', alpha=0.8)
    bars4 = axes[0, 1].bar(x + width/2, chart_data[f'{prev_year}_YTD_Sales'], width, 
                           label=str(prev_year), color='#A23B72', alpha=0.8)
    axes[0, 1].set_title('YTD Sales by Channel', fontsize=11, fontweight='bold')
    axes[0, 1].set_ylabel('Sales Amount ($)')
    axes[0, 1].set_xticks(x)
    axes[0, 1].set_xticklabels(chart_channels, rotation=45, ha='right', fontsize=9)
    axes[0, 1].legend(fontsize=9)
    axes[0, 1].yaxis.set_major_formatter(plt.FuncFormatter(lambda v, p: f'${v/1000:.0f}K'))
    axes[0, 1].grid(axis='y', alpha=0.3)
    
    # Chart 3: MTD % Achieved by Channel
    colors = ['#28a745' if v >= 100 else '#dc3545' for v in chart_data['MTD_Achieved_%']]
    axes[1, 0].barh(chart_channels, chart_data['MTD_Achieved_%'], color=colors, alpha=0.8)
    axes[1, 0].axvline(x=100, color='black', linestyle='--', linewidth=1, label='100% Target')
    axes[1, 0].set_title('MTD % Achieved by Channel', fontsize=11, fontweight='bold')
    axes[1, 0].set_xlabel('% Achieved')
    for i, v in enumerate(chart_data['MTD_Achieved_%']):
        axes[1, 0].text(v + 2, i, f'{v:.1f}%', va='center', fontsize=9)
    axes[1, 0].grid(axis='x', alpha=0.3)
    
    # Chart 4: YTD % Achieved by Channel
    colors = ['#28a745' if v >= 100 else '#dc3545' for v in chart_data['YTD_Achieved_%']]
    axes[1, 1].barh(chart_channels, chart_data['YTD_Achieved_%'], color=colors, alpha=0.8)
    axes[1, 1].axvline(x=100, color='black', linestyle='--', linewidth=1, label='100% Target')
    axes[1, 1].set_title('YTD % Achieved by Channel', fontsize=11, fontweight='bold')
    axes[1, 1].set_xlabel('% Achieved')
    for i, v in enumerate(chart_data['YTD_Achieved_%']):
        axes[1, 1].text(v + 2, i, f'{v:.1f}%', va='center', fontsize=9)
    axes[1, 1].grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    
    # Save chart to buffer
    img_buffer = io.BytesIO()
    fig.savefig(img_buffer, format='png', dpi=150, bbox_inches='tight')
    img_buffer.seek(0)
    plt.close(fig)
    
    img = XLImage(img_buffer)
    ws_summary.add_image(img, f'A{chart_start_row}')
    
    # ============================================================
    # Create Individual Channel Sheets with Top 10 Customers
    # ============================================================
    for channel, data in results['top10_by_channel'].items():
        clean_channel = "".join(c if c.isalnum() or c in (' ', '-', '_') else '_' for c in channel)[:30]
        ws = wb.create_sheet(title=clean_channel)
        
        # Get channel totals
        ch_metrics = channel_metrics[channel_metrics[channel_col] == channel].iloc[0]
        
        # ---- Channel Summary Section ----
        ws.merge_cells('A1:G1')
        ws['A1'] = f"Channel: {channel} - Performance Summary ({target_month}/{target_year})"
        ws['A1'].font = title_font
        ws['A1'].alignment = Alignment(horizontal='center')
        
        # Channel Totals Table
        ws['A3'] = 'Metric'
        ws['B3'] = f'{target_year}'
        ws['C3'] = f'{prev_year}'
        ws['D3'] = '% Achieved'
        ws['E3'] = 'YoY Growth'
        
        for col in range(1, 6):
            ws.cell(row=3, column=col).font = header_font
            ws.cell(row=3, column=col).fill = header_fill
            ws.cell(row=3, column=col).border = border
            ws.cell(row=3, column=col).alignment = Alignment(horizontal='center')
        
        # MTD Row
        ws['A4'] = 'MTD Sales'
        ws['A4'].border = border
        
        cell = ws.cell(row=4, column=2, value=ch_metrics[f'{target_year}_MTD_Sales'])
        cell.number_format = '$#,##0.00'
        cell.fill = current_year_fill
        cell.border = border
        
        cell = ws.cell(row=4, column=3, value=ch_metrics[f'{prev_year}_MTD_Sales'])
        cell.number_format = '$#,##0.00'
        cell.fill = prev_year_fill
        cell.border = border
        
        cell = ws.cell(row=4, column=4, value=ch_metrics['MTD_Achieved_%'] / 100)
        cell.number_format = '0.0%'
        cell.fill = achieved_fill
        cell.border = border
        
        cell = ws.cell(row=4, column=5, value=ch_metrics['MTD_YoY_Growth'] / 100)
        cell.number_format = '+0.0%;-0.0%'
        cell.border = border
        
        # YTD Row
        ws['A5'] = 'YTD Sales'
        ws['A5'].border = border
        
        cell = ws.cell(row=5, column=2, value=ch_metrics[f'{target_year}_YTD_Sales'])
        cell.number_format = '$#,##0.00'
        cell.fill = current_year_fill
        cell.border = border
        
        cell = ws.cell(row=5, column=3, value=ch_metrics[f'{prev_year}_YTD_Sales'])
        cell.number_format = '$#,##0.00'
        cell.fill = prev_year_fill
        cell.border = border
        
        cell = ws.cell(row=5, column=4, value=ch_metrics['YTD_Achieved_%'] / 100)
        cell.number_format = '0.0%'
        cell.fill = achieved_fill
        cell.border = border
        
        cell = ws.cell(row=5, column=5, value=ch_metrics['YTD_YoY_Growth'] / 100)
        cell.number_format = '+0.0%;-0.0%'
        cell.border = border
        
        # ---- MTD Top 10 Section ----
        ws.merge_cells('A8:G8')
        ws['A8'] = f"Top 10 MTD Customers"
        ws['A8'].font = subtitle_font
        ws['A8'].alignment = Alignment(horizontal='center')
        
        mtd_headers = ['Rank', 'Customer #', 'Customer Name', 
                       f'{target_year} MTD Sales', f'{prev_year} MTD Sales', '% Achieved']
        if sales_rep_col:
            mtd_headers.insert(3, 'Sales Rep')
        
        for col_idx, header in enumerate(mtd_headers, start=1):
            cell = ws.cell(row=9, column=col_idx, value=header)
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
            cell.border = border
        
        mtd_df = data['MTD']
        for row_idx, (_, row) in enumerate(mtd_df.iterrows(), start=10):
            rank = row_idx - 9
            col = 1
            
            cell = ws.cell(row=row_idx, column=col, value=rank)
            cell.border = border
            cell.alignment = Alignment(horizontal='center')
            col += 1
            
            cell = ws.cell(row=row_idx, column=col, value=row['Customer #'])
            cell.border = border
            col += 1
            
            cell = ws.cell(row=row_idx, column=col, value=row['Customer Name'])
            cell.border = border
            col += 1
            
            if sales_rep_col:
                cell = ws.cell(row=row_idx, column=col, value=row.get(sales_rep_col, ''))
                cell.border = border
                col += 1
            
            cell = ws.cell(row=row_idx, column=col, value=row[f'{target_year}_MTD_Sales'])
            cell.number_format = '$#,##0.00'
            cell.fill = current_year_fill
            cell.border = border
            col += 1
            
            cell = ws.cell(row=row_idx, column=col, value=row[f'{prev_year}_MTD_Sales'])
            cell.number_format = '$#,##0.00'
            cell.fill = prev_year_fill
            cell.border = border
            col += 1
            
            cell = ws.cell(row=row_idx, column=col, value=row['MTD_Achieved_%'] / 100)
            cell.number_format = '0.00%'
            cell.fill = achieved_fill
            cell.border = border
        
        # ---- YTD Top 10 Section ----
        ytd_start_row = 10 + len(mtd_df) + 2
        
        ws.merge_cells(f'A{ytd_start_row}:G{ytd_start_row}')
        ws.cell(row=ytd_start_row, column=1, value="Top 10 YTD Customers")
        ws.cell(row=ytd_start_row, column=1).font = subtitle_font
        ws.cell(row=ytd_start_row, column=1).alignment = Alignment(horizontal='center')
        
        ytd_headers = ['Rank', 'Customer #', 'Customer Name', 
                       f'{target_year} YTD Sales', f'{prev_year} YTD Sales', '% Achieved']
        if sales_rep_col:
            ytd_headers.insert(3, 'Sales Rep')
        
        header_row = ytd_start_row + 1
        for col_idx, header in enumerate(ytd_headers, start=1):
            cell = ws.cell(row=header_row, column=col_idx, value=header)
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
            cell.border = border
        
        ytd_df = data['YTD']
        for row_idx, (_, row) in enumerate(ytd_df.iterrows(), start=header_row + 1):
            rank = row_idx - header_row
            col = 1
            
            cell = ws.cell(row=row_idx, column=col, value=rank)
            cell.border = border
            cell.alignment = Alignment(horizontal='center')
            col += 1
            
            cell = ws.cell(row=row_idx, column=col, value=row['Customer #'])
            cell.border = border
            col += 1
            
            cell = ws.cell(row=row_idx, column=col, value=row['Customer Name'])
            cell.border = border
            col += 1
            
            if sales_rep_col:
                cell = ws.cell(row=row_idx, column=col, value=row.get(sales_rep_col, ''))
                cell.border = border
                col += 1
            
            cell = ws.cell(row=row_idx, column=col, value=row[f'{target_year}_YTD_Sales'])
            cell.number_format = '$#,##0.00'
            cell.fill = current_year_fill
            cell.border = border
            col += 1
            
            cell = ws.cell(row=row_idx, column=col, value=row[f'{prev_year}_YTD_Sales'])
            cell.number_format = '$#,##0.00'
            cell.fill = prev_year_fill
            cell.border = border
            col += 1
            
            cell = ws.cell(row=row_idx, column=col, value=row['YTD_Achieved_%'] / 100)
            cell.number_format = '0.00%'
            cell.fill = achieved_fill
            cell.border = border
        
        # Adjust column widths
        ws.column_dimensions['A'].width = 8
        ws.column_dimensions['B'].width = 12
        ws.column_dimensions['C'].width = 30
        ws.column_dimensions['D'].width = 16
        ws.column_dimensions['E'].width = 16
        ws.column_dimensions['F'].width = 12
        ws.column_dimensions['G'].width = 12
    
    # Save workbook
    wb.save(output_file)
    
    print(f"\n✓ Excel report saved to: {output_file}")
    print(f"  - Channel Summary sheet with MTD/YTD totals by channel")
    for channel in channels:
        print(f"  - {channel} sheet with channel totals and Top 10 MTD/YTD customers")
    
    return results


# Run the function
results = generate_top10_customers_by_channel(
    sales_file="sales detail 2024-2025.xls",
    customer_list_file="SalesCustomerList_with_Channel.xlsx",
    target_month=10,
    target_year=2025,
    output_file="Top10_Customers_by_Channel.xlsx"
)

# Display Channel Summary
print("\n" + "=" * 70)
print("CHANNEL MTD/YTD SUMMARY")
print("=" * 70)
channel_df = results['channel_metrics']
print(channel_df.to_string(index=False))


STEP 1: Reading and parsing sales data...
Parsed 55468 transaction rows
Unique customers: 351

STEP 2: Joining with customer list to get Channel...
Found Channel column: 'Channel'
Found Sales Rep column: 'Sales Rep'

Channel distribution in sales:
  Cash Sales: 2 customers
  Food Ethnic: 2 customers
  Food Service: 85 customers
  Indent: 4 customers
  Liquor Group: 21 customers
  Mainstream: 10 customers
  Retail Group: 187 customers
  Staff: 4 customers
  Sub - Distributor: 17 customers
  Unknown: 19 customers

STEP 3: Calculating MTD and YTD metrics by customer...
Total customers with metrics: 343

STEP 4: Calculating Channel-Level MTD/YTD Totals...

Channel               Customers        2025 MTD        2024 MTD      MTD %        2025 YTD        2024 YTD      YTD %
------------------------------------------------------------------------------------------------------------------------
Cash Sales                    2 $        1,623 $          384     422.9% $       87,304 $       65,6

In [7]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
import matplotlib.pyplot as plt
import numpy as np
import io
from openpyxl.drawing.image import Image as XLImage

def generate_sales_rep_performance(
    sales_file: str = "sales detail 2024-2025.xls",
    customer_list_file: str = "SalesCustomerList_with_Channel.xlsx",
    target_month: int = 10,
    target_year: int = 2025,
    output_file: str = "Sales_Rep_Performance.xlsx"
) -> dict:
    """
    Generate MTD/YTD performance reports for each Sales Rep.
    
    Args:
        sales_file: Path to the sales details Excel file
        customer_list_file: Path to the SalesCustomerList_with_Channel.xlsx file
        target_month: Month for MTD calculation
        target_year: Year for calculations
        output_file: Path to save the Excel report
    
    Returns:
        Dictionary with sales rep performance data including:
        - sales_rep_metrics: DataFrame with MTD/YTD totals by sales rep
        - grand_total: Overall totals
        - top10_customers_by_rep: Top 10 customers for each sales rep
    """
    prev_year = target_year - 1
    
    # ============================================================
    # STEP 1: Parse sales data
    # ============================================================
    print("=" * 70)
    print("STEP 1: Reading and parsing sales data...")
    print("=" * 70)
    
    df_raw = pd.read_excel(sales_file, header=None)
    
    col_period = 1
    col_sales_amount = 17
    
    data_rows = []
    current_customer = None
    current_customer_name = None
    
    for idx in range(11, len(df_raw)):
        row = df_raw.iloc[idx]
        first_cell = row.iloc[0]
        
        if pd.isna(first_cell):
            continue
        
        first_cell_str = str(first_cell).strip()
        
        if first_cell_str and not first_cell_str.isdigit() and not first_cell_str.startswith('Item'):
            if any(c.isalpha() for c in first_cell_str) and any(c.isdigit() for c in first_cell_str):
                current_customer = first_cell_str
                cust_name = row.iloc[7] if pd.notna(row.iloc[7]) else ''
                current_customer_name = str(cust_name).strip() if cust_name else current_customer
                continue
        
        if first_cell_str.isdigit() and len(first_cell_str) == 4:
            year = int(first_cell_str)
            
            period_cell = row.iloc[col_period]
            month = None
            if pd.notna(period_cell):
                try:
                    month = int(float(period_cell))
                except (ValueError, TypeError):
                    month = None
            
            sales_amount = row.iloc[col_sales_amount]
            
            if pd.notna(sales_amount) and current_customer and month is not None:
                try:
                    sales_value = float(sales_amount)
                    data_rows.append({
                        'Customer #': current_customer,
                        'Customer Name': current_customer_name,
                        'Year': year,
                        'Month': month,
                        'Sales Amount': sales_value
                    })
                except (ValueError, TypeError):
                    pass
    
    df_sales = pd.DataFrame(data_rows)
    
    if df_sales.empty:
        print("No sales data found!")
        return {}
    
    print(f"Parsed {len(df_sales)} transaction rows")
    print(f"Unique customers: {df_sales['Customer #'].nunique()}")
    
    # ============================================================
    # STEP 2: Join with customer list to get Sales Rep and Channel
    # ============================================================
    print("\n" + "=" * 70)
    print("STEP 2: Joining with customer list to get Sales Rep...")
    print("=" * 70)
    
    customer_df = pd.read_excel(customer_list_file)
    
    # Find Sales Rep and Channel columns
    sales_rep_col = None
    channel_col = None
    for col in customer_df.columns:
        if 'sales' in col.lower() and 'rep' in col.lower():
            sales_rep_col = col
        if 'channel' in col.lower():
            channel_col = col
    
    if not sales_rep_col:
        print("ERROR: Sales Rep column not found in customer list!")
        return {}
    
    print(f"Found Sales Rep column: '{sales_rep_col}'")
    if channel_col:
        print(f"Found Channel column: '{channel_col}'")
    
    # Create lookup for customer -> sales rep/channel
    customer_cols = ['Customer #', sales_rep_col]
    if channel_col:
        customer_cols.append(channel_col)
    
    customer_lookup = customer_df[customer_cols].drop_duplicates(subset=['Customer #'])
    
    # Merge to add sales rep to sales data
    df_sales = df_sales.merge(customer_lookup, on='Customer #', how='left')
    
    # Fill missing sales reps with 'Unassigned'
    df_sales[sales_rep_col] = df_sales[sales_rep_col].fillna('Unassigned')
    
    print(f"\nSales Rep distribution in sales:")
    rep_counts = df_sales.groupby(sales_rep_col)['Customer #'].nunique()
    for rep, count in rep_counts.items():
        print(f"  {rep}: {count} customers")
    
    # ============================================================
    # STEP 3: Calculate MTD and YTD metrics by customer
    # ============================================================
    print("\n" + "=" * 70)
    print("STEP 3: Calculating MTD and YTD metrics by customer...")
    print("=" * 70)
    
    # Filter for target years
    df_current = df_sales[df_sales['Year'] == target_year].copy()
    df_prev = df_sales[df_sales['Year'] == prev_year].copy()
    
    # Calculate MTD (specific month only)
    df_mtd_current = df_current[df_current['Month'] == target_month]
    df_mtd_prev = df_prev[df_prev['Month'] == target_month]
    
    # Calculate YTD (months 1 to target_month)
    df_ytd_current = df_current[df_current['Month'] <= target_month]
    df_ytd_prev = df_prev[df_prev['Month'] <= target_month]
    
    # Aggregate by customer
    def aggregate_by_customer(df, sales_rep_col, channel_col):
        agg_cols = ['Customer #', 'Customer Name', sales_rep_col]
        if channel_col and channel_col in df.columns:
            agg_cols.append(channel_col)
        
        result = df.groupby(agg_cols, dropna=False).agg({
            'Sales Amount': 'sum'
        }).reset_index()
        return result
    
    mtd_current_agg = aggregate_by_customer(df_mtd_current, sales_rep_col, channel_col)
    mtd_prev_agg = aggregate_by_customer(df_mtd_prev, sales_rep_col, channel_col)
    ytd_current_agg = aggregate_by_customer(df_ytd_current, sales_rep_col, channel_col)
    ytd_prev_agg = aggregate_by_customer(df_ytd_prev, sales_rep_col, channel_col)
    
    # Rename columns
    mtd_current_agg = mtd_current_agg.rename(columns={'Sales Amount': f'{target_year}_MTD_Sales'})
    mtd_prev_agg = mtd_prev_agg.rename(columns={'Sales Amount': f'{prev_year}_MTD_Sales'})
    ytd_current_agg = ytd_current_agg.rename(columns={'Sales Amount': f'{target_year}_YTD_Sales'})
    ytd_prev_agg = ytd_prev_agg.rename(columns={'Sales Amount': f'{prev_year}_YTD_Sales'})
    
    # Start with MTD current
    customer_metrics = mtd_current_agg.copy()
    
    # Merge MTD previous
    customer_metrics = customer_metrics.merge(
        mtd_prev_agg[['Customer #', f'{prev_year}_MTD_Sales']],
        on='Customer #',
        how='outer'
    )
    
    # Merge YTD current
    customer_metrics = customer_metrics.merge(
        ytd_current_agg[['Customer #', f'{target_year}_YTD_Sales']],
        on='Customer #',
        how='outer'
    )
    
    # Merge YTD previous
    customer_metrics = customer_metrics.merge(
        ytd_prev_agg[['Customer #', f'{prev_year}_YTD_Sales']],
        on='Customer #',
        how='outer'
    )
    
    # Fill NaN values with 0 for sales columns
    sales_cols = [f'{target_year}_MTD_Sales', f'{prev_year}_MTD_Sales', 
                  f'{target_year}_YTD_Sales', f'{prev_year}_YTD_Sales']
    for col in sales_cols:
        if col in customer_metrics.columns:
            customer_metrics[col] = customer_metrics[col].fillna(0)
    
    # Get complete customer info
    info_cols = ['Customer #', 'Customer Name', sales_rep_col]
    if channel_col:
        info_cols.append(channel_col)
    
    all_customers = df_sales[info_cols].drop_duplicates(subset=['Customer #'])
    
    # Update missing customer info
    drop_cols = ['Customer Name', sales_rep_col]
    if channel_col and channel_col in customer_metrics.columns:
        drop_cols.append(channel_col)
    
    customer_metrics = customer_metrics.drop(columns=drop_cols, errors='ignore')
    customer_metrics = customer_metrics.merge(all_customers, on='Customer #', how='left')
    
    # Calculate % Achieved
    customer_metrics['MTD_Achieved_%'] = customer_metrics.apply(
        lambda x: (x[f'{target_year}_MTD_Sales'] / x[f'{prev_year}_MTD_Sales'] * 100) 
        if x[f'{prev_year}_MTD_Sales'] > 0 else 0, axis=1
    )
    customer_metrics['YTD_Achieved_%'] = customer_metrics.apply(
        lambda x: (x[f'{target_year}_YTD_Sales'] / x[f'{prev_year}_YTD_Sales'] * 100) 
        if x[f'{prev_year}_YTD_Sales'] > 0 else 0, axis=1
    )
    
    print(f"Total customers with metrics: {len(customer_metrics)}")
    
    # ============================================================
    # STEP 4: Calculate Sales Rep-Level MTD/YTD Totals
    # ============================================================
    print("\n" + "=" * 70)
    print("STEP 4: Calculating Sales Rep-Level MTD/YTD Totals...")
    print("=" * 70)
    
    # Aggregate by Sales Rep
    sales_rep_metrics = customer_metrics.groupby(sales_rep_col).agg({
        f'{target_year}_MTD_Sales': 'sum',
        f'{prev_year}_MTD_Sales': 'sum',
        f'{target_year}_YTD_Sales': 'sum',
        f'{prev_year}_YTD_Sales': 'sum',
        'Customer #': 'count'
    }).reset_index()
    
    sales_rep_metrics = sales_rep_metrics.rename(columns={'Customer #': 'Customer_Count'})
    
    # Calculate % Achieved for sales reps
    sales_rep_metrics['MTD_Achieved_%'] = sales_rep_metrics.apply(
        lambda x: (x[f'{target_year}_MTD_Sales'] / x[f'{prev_year}_MTD_Sales'] * 100) 
        if x[f'{prev_year}_MTD_Sales'] > 0 else 0, axis=1
    )
    sales_rep_metrics['YTD_Achieved_%'] = sales_rep_metrics.apply(
        lambda x: (x[f'{target_year}_YTD_Sales'] / x[f'{prev_year}_YTD_Sales'] * 100) 
        if x[f'{prev_year}_YTD_Sales'] > 0 else 0, axis=1
    )
    
    # Calculate YoY Growth
    sales_rep_metrics['MTD_YoY_Growth'] = sales_rep_metrics.apply(
        lambda x: ((x[f'{target_year}_MTD_Sales'] - x[f'{prev_year}_MTD_Sales']) / x[f'{prev_year}_MTD_Sales'] * 100) 
        if x[f'{prev_year}_MTD_Sales'] > 0 else 0, axis=1
    )
    sales_rep_metrics['YTD_YoY_Growth'] = sales_rep_metrics.apply(
        lambda x: ((x[f'{target_year}_YTD_Sales'] - x[f'{prev_year}_YTD_Sales']) / x[f'{prev_year}_YTD_Sales'] * 100) 
        if x[f'{prev_year}_YTD_Sales'] > 0 else 0, axis=1
    )
    
    # Sort by current year YTD sales descending
    sales_rep_metrics = sales_rep_metrics.sort_values(f'{target_year}_YTD_Sales', ascending=False)
    
    # Print Sales Rep Summary
    print(f"\n{'Sales Rep':<20} {'Customers':>10} {f'{target_year} MTD':>15} {f'{prev_year} MTD':>15} {'MTD %':>10} {f'{target_year} YTD':>15} {f'{prev_year} YTD':>15} {'YTD %':>10}")
    print("-" * 130)
    
    for _, row in sales_rep_metrics.iterrows():
        if row[sales_rep_col] != 'Unassigned':
            print(f"{str(row[sales_rep_col]):<20} {row['Customer_Count']:>10} "
                  f"${row[f'{target_year}_MTD_Sales']:>13,.0f} ${row[f'{prev_year}_MTD_Sales']:>13,.0f} "
                  f"{row['MTD_Achieved_%']:>9.1f}% "
                  f"${row[f'{target_year}_YTD_Sales']:>13,.0f} ${row[f'{prev_year}_YTD_Sales']:>13,.0f} "
                  f"{row['YTD_Achieved_%']:>9.1f}%")
    
    # Calculate Grand Total
    grand_total = {
        'Sales Rep': 'GRAND TOTAL',
        'Customer_Count': sales_rep_metrics['Customer_Count'].sum(),
        f'{target_year}_MTD_Sales': sales_rep_metrics[f'{target_year}_MTD_Sales'].sum(),
        f'{prev_year}_MTD_Sales': sales_rep_metrics[f'{prev_year}_MTD_Sales'].sum(),
        f'{target_year}_YTD_Sales': sales_rep_metrics[f'{target_year}_YTD_Sales'].sum(),
        f'{prev_year}_YTD_Sales': sales_rep_metrics[f'{prev_year}_YTD_Sales'].sum(),
    }
    grand_total['MTD_Achieved_%'] = (grand_total[f'{target_year}_MTD_Sales'] / grand_total[f'{prev_year}_MTD_Sales'] * 100) if grand_total[f'{prev_year}_MTD_Sales'] > 0 else 0
    grand_total['YTD_Achieved_%'] = (grand_total[f'{target_year}_YTD_Sales'] / grand_total[f'{prev_year}_YTD_Sales'] * 100) if grand_total[f'{prev_year}_YTD_Sales'] > 0 else 0
    
    print("-" * 130)
    print(f"{'GRAND TOTAL':<20} {grand_total['Customer_Count']:>10} "
          f"${grand_total[f'{target_year}_MTD_Sales']:>13,.0f} ${grand_total[f'{prev_year}_MTD_Sales']:>13,.0f} "
          f"{grand_total['MTD_Achieved_%']:>9.1f}% "
          f"${grand_total[f'{target_year}_YTD_Sales']:>13,.0f} ${grand_total[f'{prev_year}_YTD_Sales']:>13,.0f} "
          f"{grand_total['YTD_Achieved_%']:>9.1f}%")
    
    # ============================================================
    # STEP 5: Generate Top 10 Customers by Sales Rep for MTD and YTD
    # ============================================================
    print("\n" + "=" * 70)
    print("STEP 5: Generating Top 10 customers by Sales Rep...")
    print("=" * 70)
    
    sales_reps = customer_metrics[sales_rep_col].dropna().unique()
    sales_reps = [r for r in sales_reps if r != 'Unassigned']
    sales_reps = sorted(sales_reps)
    
    print(f"Sales Reps found: {sales_reps}")
    
    results = {
        'sales_rep_metrics': sales_rep_metrics,
        'grand_total': grand_total,
        'top10_by_rep': {},
        'sales_rep_col': sales_rep_col,
        'channel_col': channel_col
    }
    
    for rep in sales_reps:
        rep_data = customer_metrics[customer_metrics[sales_rep_col] == rep].copy()
        
        if rep_data.empty:
            continue
        
        # Top 10 MTD
        top10_mtd = rep_data.nlargest(10, f'{target_year}_MTD_Sales')
        
        # Top 10 YTD
        top10_ytd = rep_data.nlargest(10, f'{target_year}_YTD_Sales')
        
        results['top10_by_rep'][rep] = {
            'MTD': top10_mtd,
            'YTD': top10_ytd
        }
        
        print(f"\n{rep}:")
        print(f"  Total customers: {len(rep_data)}")
        print(f"  Top MTD customer: {top10_mtd.iloc[0]['Customer Name'] if len(top10_mtd) > 0 else 'N/A'}")
        print(f"  Top YTD customer: {top10_ytd.iloc[0]['Customer Name'] if len(top10_ytd) > 0 else 'N/A'}")
    
    # ============================================================
    # STEP 6: Export to Excel with formatting
    # ============================================================
    print("\n" + "=" * 70)
    print("STEP 6: Creating Excel report...")
    print("=" * 70)
    
    wb = Workbook()
    wb.remove(wb.active)
    
    # Styling
    header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
    header_font = Font(bold=True, color="FFFFFF", size=10)
    title_font = Font(bold=True, size=14)
    subtitle_font = Font(bold=True, size=12)
    border = Border(
        left=Side(style='thin'),
        right=Side(style='thin'),
        top=Side(style='thin'),
        bottom=Side(style='thin')
    )
    
    # Color fills
    current_year_fill = PatternFill(start_color="E2EFDA", end_color="E2EFDA", fill_type="solid")
    prev_year_fill = PatternFill(start_color="FCE4D6", end_color="FCE4D6", fill_type="solid")
    achieved_fill = PatternFill(start_color="FFF2CC", end_color="FFF2CC", fill_type="solid")
    total_fill = PatternFill(start_color="DDEBF7", end_color="DDEBF7", fill_type="solid")
    
    # ============================================================
    # Create Summary Sheet with Sales Rep MTD/YTD Totals
    # ============================================================
    ws_summary = wb.create_sheet(title="Sales Rep Summary", index=0)
    
    # Title
    ws_summary.merge_cells('A1:J1')
    ws_summary['A1'] = f"Sales Rep Performance Summary - MTD/YTD ({target_month}/{target_year})"
    ws_summary['A1'].font = Font(bold=True, size=16)
    ws_summary['A1'].alignment = Alignment(horizontal='center')
    
    # Sales Rep Summary Headers
    summary_headers = [
        'Sales Rep', 'Customers',
        f'{target_year} MTD', f'{prev_year} MTD', 'MTD %', 'MTD YoY Growth',
        f'{target_year} YTD', f'{prev_year} YTD', 'YTD %', 'YTD YoY Growth'
    ]
    
    for col_idx, header in enumerate(summary_headers, start=1):
        cell = ws_summary.cell(row=3, column=col_idx, value=header)
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
        cell.border = border
    
    # Write Sales Rep Data
    row_idx = 4
    for _, rep_row in sales_rep_metrics.iterrows():
        if rep_row[sales_rep_col] == 'Unassigned':
            continue
        
        ws_summary.cell(row=row_idx, column=1, value=rep_row[sales_rep_col]).border = border
        ws_summary.cell(row=row_idx, column=2, value=rep_row['Customer_Count']).border = border
        
        # MTD Current Year
        cell = ws_summary.cell(row=row_idx, column=3, value=rep_row[f'{target_year}_MTD_Sales'])
        cell.number_format = '$#,##0.00'
        cell.fill = current_year_fill
        cell.border = border
        
        # MTD Previous Year
        cell = ws_summary.cell(row=row_idx, column=4, value=rep_row[f'{prev_year}_MTD_Sales'])
        cell.number_format = '$#,##0.00'
        cell.fill = prev_year_fill
        cell.border = border
        
        # MTD Achieved %
        cell = ws_summary.cell(row=row_idx, column=5, value=rep_row['MTD_Achieved_%'] / 100)
        cell.number_format = '0.0%'
        cell.fill = achieved_fill
        cell.border = border
        
        # MTD YoY Growth
        cell = ws_summary.cell(row=row_idx, column=6, value=rep_row['MTD_YoY_Growth'] / 100)
        cell.number_format = '+0.0%;-0.0%'
        cell.border = border
        
        # YTD Current Year
        cell = ws_summary.cell(row=row_idx, column=7, value=rep_row[f'{target_year}_YTD_Sales'])
        cell.number_format = '$#,##0.00'
        cell.fill = current_year_fill
        cell.border = border
        
        # YTD Previous Year
        cell = ws_summary.cell(row=row_idx, column=8, value=rep_row[f'{prev_year}_YTD_Sales'])
        cell.number_format = '$#,##0.00'
        cell.fill = prev_year_fill
        cell.border = border
        
        # YTD Achieved %
        cell = ws_summary.cell(row=row_idx, column=9, value=rep_row['YTD_Achieved_%'] / 100)
        cell.number_format = '0.0%'
        cell.fill = achieved_fill
        cell.border = border
        
        # YTD YoY Growth
        cell = ws_summary.cell(row=row_idx, column=10, value=rep_row['YTD_YoY_Growth'] / 100)
        cell.number_format = '+0.0%;-0.0%'
        cell.border = border
        
        row_idx += 1
    
    # Grand Total Row
    ws_summary.cell(row=row_idx, column=1, value='GRAND TOTAL').font = Font(bold=True)
    ws_summary.cell(row=row_idx, column=1).fill = total_fill
    ws_summary.cell(row=row_idx, column=1).border = border
    
    ws_summary.cell(row=row_idx, column=2, value=grand_total['Customer_Count']).font = Font(bold=True)
    ws_summary.cell(row=row_idx, column=2).fill = total_fill
    ws_summary.cell(row=row_idx, column=2).border = border
    
    cell = ws_summary.cell(row=row_idx, column=3, value=grand_total[f'{target_year}_MTD_Sales'])
    cell.number_format = '$#,##0.00'
    cell.font = Font(bold=True)
    cell.fill = total_fill
    cell.border = border
    
    cell = ws_summary.cell(row=row_idx, column=4, value=grand_total[f'{prev_year}_MTD_Sales'])
    cell.number_format = '$#,##0.00'
    cell.font = Font(bold=True)
    cell.fill = total_fill
    cell.border = border
    
    cell = ws_summary.cell(row=row_idx, column=5, value=grand_total['MTD_Achieved_%'] / 100)
    cell.number_format = '0.0%'
    cell.font = Font(bold=True)
    cell.fill = total_fill
    cell.border = border
    
    mtd_growth = ((grand_total[f'{target_year}_MTD_Sales'] - grand_total[f'{prev_year}_MTD_Sales']) / grand_total[f'{prev_year}_MTD_Sales'] * 100) if grand_total[f'{prev_year}_MTD_Sales'] > 0 else 0
    cell = ws_summary.cell(row=row_idx, column=6, value=mtd_growth / 100)
    cell.number_format = '+0.0%;-0.0%'
    cell.font = Font(bold=True)
    cell.fill = total_fill
    cell.border = border
    
    cell = ws_summary.cell(row=row_idx, column=7, value=grand_total[f'{target_year}_YTD_Sales'])
    cell.number_format = '$#,##0.00'
    cell.font = Font(bold=True)
    cell.fill = total_fill
    cell.border = border
    
    cell = ws_summary.cell(row=row_idx, column=8, value=grand_total[f'{prev_year}_YTD_Sales'])
    cell.number_format = '$#,##0.00'
    cell.font = Font(bold=True)
    cell.fill = total_fill
    cell.border = border
    
    cell = ws_summary.cell(row=row_idx, column=9, value=grand_total['YTD_Achieved_%'] / 100)
    cell.number_format = '0.0%'
    cell.font = Font(bold=True)
    cell.fill = total_fill
    cell.border = border
    
    ytd_growth = ((grand_total[f'{target_year}_YTD_Sales'] - grand_total[f'{prev_year}_YTD_Sales']) / grand_total[f'{prev_year}_YTD_Sales'] * 100) if grand_total[f'{prev_year}_YTD_Sales'] > 0 else 0
    cell = ws_summary.cell(row=row_idx, column=10, value=ytd_growth / 100)
    cell.number_format = '+0.0%;-0.0%'
    cell.font = Font(bold=True)
    cell.fill = total_fill
    cell.border = border
    
    # Adjust column widths
    ws_summary.column_dimensions['A'].width = 20
    ws_summary.column_dimensions['B'].width = 12
    ws_summary.column_dimensions['C'].width = 16
    ws_summary.column_dimensions['D'].width = 16
    ws_summary.column_dimensions['E'].width = 10
    ws_summary.column_dimensions['F'].width = 14
    ws_summary.column_dimensions['G'].width = 16
    ws_summary.column_dimensions['H'].width = 16
    ws_summary.column_dimensions['I'].width = 10
    ws_summary.column_dimensions['J'].width = 14
    
    ws_summary.row_dimensions[3].height = 30
    
    # ============================================================
    # Add Charts to Summary Sheet
    # ============================================================
    chart_start_row = row_idx + 3
    
    # Create charts
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f'Sales Rep Performance Analysis - {target_month}/{target_year}', fontsize=14, fontweight='bold')
    
    # Filter out Unassigned for charts
    chart_data = sales_rep_metrics[sales_rep_metrics[sales_rep_col] != 'Unassigned'].copy()
    chart_reps = chart_data[sales_rep_col].tolist()
    
    x = np.arange(len(chart_reps))
    width = 0.35
    
    # Chart 1: MTD Sales by Sales Rep
    bars1 = axes[0, 0].bar(x - width/2, chart_data[f'{target_year}_MTD_Sales'], width, 
                           label=str(target_year), color='#2E86AB', alpha=0.8)
    bars2 = axes[0, 0].bar(x + width/2, chart_data[f'{prev_year}_MTD_Sales'], width, 
                           label=str(prev_year), color='#A23B72', alpha=0.8)
    axes[0, 0].set_title('MTD Sales by Sales Rep', fontsize=11, fontweight='bold')
    axes[0, 0].set_ylabel('Sales Amount ($)')
    axes[0, 0].set_xticks(x)
    axes[0, 0].set_xticklabels(chart_reps, rotation=45, ha='right', fontsize=9)
    axes[0, 0].legend(fontsize=9)
    axes[0, 0].yaxis.set_major_formatter(plt.FuncFormatter(lambda v, p: f'${v/1000:.0f}K'))
    axes[0, 0].grid(axis='y', alpha=0.3)
    
    # Chart 2: YTD Sales by Sales Rep
    bars3 = axes[0, 1].bar(x - width/2, chart_data[f'{target_year}_YTD_Sales'], width, 
                           label=str(target_year), color='#2E86AB', alpha=0.8)
    bars4 = axes[0, 1].bar(x + width/2, chart_data[f'{prev_year}_YTD_Sales'], width, 
                           label=str(prev_year), color='#A23B72', alpha=0.8)
    axes[0, 1].set_title('YTD Sales by Sales Rep', fontsize=11, fontweight='bold')
    axes[0, 1].set_ylabel('Sales Amount ($)')
    axes[0, 1].set_xticks(x)
    axes[0, 1].set_xticklabels(chart_reps, rotation=45, ha='right', fontsize=9)
    axes[0, 1].legend(fontsize=9)
    axes[0, 1].yaxis.set_major_formatter(plt.FuncFormatter(lambda v, p: f'${v/1000:.0f}K'))
    axes[0, 1].grid(axis='y', alpha=0.3)
    
    # Chart 3: MTD % Achieved by Sales Rep
    colors = ['#28a745' if v >= 100 else '#dc3545' for v in chart_data['MTD_Achieved_%']]
    axes[1, 0].barh(chart_reps, chart_data['MTD_Achieved_%'], color=colors, alpha=0.8)
    axes[1, 0].axvline(x=100, color='black', linestyle='--', linewidth=1, label='100% Target')
    axes[1, 0].set_title('MTD % Achieved by Sales Rep', fontsize=11, fontweight='bold')
    axes[1, 0].set_xlabel('% Achieved')
    for i, v in enumerate(chart_data['MTD_Achieved_%']):
        axes[1, 0].text(v + 2, i, f'{v:.1f}%', va='center', fontsize=9)
    axes[1, 0].grid(axis='x', alpha=0.3)
    
    # Chart 4: YTD % Achieved by Sales Rep
    colors = ['#28a745' if v >= 100 else '#dc3545' for v in chart_data['YTD_Achieved_%']]
    axes[1, 1].barh(chart_reps, chart_data['YTD_Achieved_%'], color=colors, alpha=0.8)
    axes[1, 1].axvline(x=100, color='black', linestyle='--', linewidth=1, label='100% Target')
    axes[1, 1].set_title('YTD % Achieved by Sales Rep', fontsize=11, fontweight='bold')
    axes[1, 1].set_xlabel('% Achieved')
    for i, v in enumerate(chart_data['YTD_Achieved_%']):
        axes[1, 1].text(v + 2, i, f'{v:.1f}%', va='center', fontsize=9)
    axes[1, 1].grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    
    # Save chart to buffer
    img_buffer = io.BytesIO()
    fig.savefig(img_buffer, format='png', dpi=150, bbox_inches='tight')
    img_buffer.seek(0)
    plt.close(fig)
    
    img = XLImage(img_buffer)
    ws_summary.add_image(img, f'A{chart_start_row}')
    
    # ============================================================
    # Create Individual Sales Rep Sheets with Top 10 Customers
    # ============================================================
    for rep, data in results['top10_by_rep'].items():
        clean_rep = "".join(c if c.isalnum() or c in (' ', '-', '_') else '_' for c in str(rep))[:30]
        ws = wb.create_sheet(title=clean_rep)
        
        # Get sales rep totals
        rep_metrics = sales_rep_metrics[sales_rep_metrics[sales_rep_col] == rep].iloc[0]
        
        # ---- Sales Rep Summary Section ----
        ws.merge_cells('A1:G1')
        ws['A1'] = f"Sales Rep: {rep} - Performance Summary ({target_month}/{target_year})"
        ws['A1'].font = title_font
        ws['A1'].alignment = Alignment(horizontal='center')
        
        # Sales Rep Totals Table
        ws['A3'] = 'Metric'
        ws['B3'] = f'{target_year}'
        ws['C3'] = f'{prev_year}'
        ws['D3'] = '% Achieved'
        ws['E3'] = 'YoY Growth'
        
        for col in range(1, 6):
            ws.cell(row=3, column=col).font = header_font
            ws.cell(row=3, column=col).fill = header_fill
            ws.cell(row=3, column=col).border = border
            ws.cell(row=3, column=col).alignment = Alignment(horizontal='center')
        
        # MTD Row
        ws['A4'] = 'MTD Sales'
        ws['A4'].border = border
        
        cell = ws.cell(row=4, column=2, value=rep_metrics[f'{target_year}_MTD_Sales'])
        cell.number_format = '$#,##0.00'
        cell.fill = current_year_fill
        cell.border = border
        
        cell = ws.cell(row=4, column=3, value=rep_metrics[f'{prev_year}_MTD_Sales'])
        cell.number_format = '$#,##0.00'
        cell.fill = prev_year_fill
        cell.border = border
        
        cell = ws.cell(row=4, column=4, value=rep_metrics['MTD_Achieved_%'] / 100)
        cell.number_format = '0.0%'
        cell.fill = achieved_fill
        cell.border = border
        
        cell = ws.cell(row=4, column=5, value=rep_metrics['MTD_YoY_Growth'] / 100)
        cell.number_format = '+0.0%;-0.0%'
        cell.border = border
        
        # YTD Row
        ws['A5'] = 'YTD Sales'
        ws['A5'].border = border
        
        cell = ws.cell(row=5, column=2, value=rep_metrics[f'{target_year}_YTD_Sales'])
        cell.number_format = '$#,##0.00'
        cell.fill = current_year_fill
        cell.border = border
        
        cell = ws.cell(row=5, column=3, value=rep_metrics[f'{prev_year}_YTD_Sales'])
        cell.number_format = '$#,##0.00'
        cell.fill = prev_year_fill
        cell.border = border
        
        cell = ws.cell(row=5, column=4, value=rep_metrics['YTD_Achieved_%'] / 100)
        cell.number_format = '0.0%'
        cell.fill = achieved_fill
        cell.border = border
        
        cell = ws.cell(row=5, column=5, value=rep_metrics['YTD_YoY_Growth'] / 100)
        cell.number_format = '+0.0%;-0.0%'
        cell.border = border
        
        # Customer Count
        ws['A6'] = 'Total Customers'
        ws['A6'].border = border
        ws.cell(row=6, column=2, value=rep_metrics['Customer_Count']).border = border
        
        # ---- MTD Top 10 Section ----
        ws.merge_cells('A9:G9')
        ws['A9'] = f"Top 10 MTD Customers"
        ws['A9'].font = subtitle_font
        ws['A9'].alignment = Alignment(horizontal='center')
        
        mtd_headers = ['Rank', 'Customer #', 'Customer Name', 
                       f'{target_year} MTD Sales', f'{prev_year} MTD Sales', '% Achieved']
        if channel_col:
            mtd_headers.insert(3, 'Channel')
        
        for col_idx, header in enumerate(mtd_headers, start=1):
            cell = ws.cell(row=10, column=col_idx, value=header)
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
            cell.border = border
        
        mtd_df = data['MTD']
        for row_idx, (_, row) in enumerate(mtd_df.iterrows(), start=11):
            rank = row_idx - 10
            col = 1
            
            cell = ws.cell(row=row_idx, column=col, value=rank)
            cell.border = border
            cell.alignment = Alignment(horizontal='center')
            col += 1
            
            cell = ws.cell(row=row_idx, column=col, value=row['Customer #'])
            cell.border = border
            col += 1
            
            cell = ws.cell(row=row_idx, column=col, value=row['Customer Name'])
            cell.border = border
            col += 1
            
            if channel_col:
                cell = ws.cell(row=row_idx, column=col, value=row.get(channel_col, ''))
                cell.border = border
                col += 1
            
            cell = ws.cell(row=row_idx, column=col, value=row[f'{target_year}_MTD_Sales'])
            cell.number_format = '$#,##0.00'
            cell.fill = current_year_fill
            cell.border = border
            col += 1
            
            cell = ws.cell(row=row_idx, column=col, value=row[f'{prev_year}_MTD_Sales'])
            cell.number_format = '$#,##0.00'
            cell.fill = prev_year_fill
            cell.border = border
            col += 1
            
            cell = ws.cell(row=row_idx, column=col, value=row['MTD_Achieved_%'] / 100)
            cell.number_format = '0.00%'
            cell.fill = achieved_fill
            cell.border = border
        
        # ---- YTD Top 10 Section ----
        ytd_start_row = 11 + len(mtd_df) + 2
        
        ws.merge_cells(f'A{ytd_start_row}:G{ytd_start_row}')
        ws.cell(row=ytd_start_row, column=1, value="Top 10 YTD Customers")
        ws.cell(row=ytd_start_row, column=1).font = subtitle_font
        ws.cell(row=ytd_start_row, column=1).alignment = Alignment(horizontal='center')
        
        ytd_headers = ['Rank', 'Customer #', 'Customer Name', 
                       f'{target_year} YTD Sales', f'{prev_year} YTD Sales', '% Achieved']
        if channel_col:
            ytd_headers.insert(3, 'Channel')
        
        header_row = ytd_start_row + 1
        for col_idx, header in enumerate(ytd_headers, start=1):
            cell = ws.cell(row=header_row, column=col_idx, value=header)
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
            cell.border = border
        
        ytd_df = data['YTD']
        for row_idx, (_, row) in enumerate(ytd_df.iterrows(), start=header_row + 1):
            rank = row_idx - header_row
            col = 1
            
            cell = ws.cell(row=row_idx, column=col, value=rank)
            cell.border = border
            cell.alignment = Alignment(horizontal='center')
            col += 1
            
            cell = ws.cell(row=row_idx, column=col, value=row['Customer #'])
            cell.border = border
            col += 1
            
            cell = ws.cell(row=row_idx, column=col, value=row['Customer Name'])
            cell.border = border
            col += 1
            
            if channel_col:
                cell = ws.cell(row=row_idx, column=col, value=row.get(channel_col, ''))
                cell.border = border
                col += 1
            
            cell = ws.cell(row=row_idx, column=col, value=row[f'{target_year}_YTD_Sales'])
            cell.number_format = '$#,##0.00'
            cell.fill = current_year_fill
            cell.border = border
            col += 1
            
            cell = ws.cell(row=row_idx, column=col, value=row[f'{prev_year}_YTD_Sales'])
            cell.number_format = '$#,##0.00'
            cell.fill = prev_year_fill
            cell.border = border
            col += 1
            
            cell = ws.cell(row=row_idx, column=col, value=row['YTD_Achieved_%'] / 100)
            cell.number_format = '0.00%'
            cell.fill = achieved_fill
            cell.border = border
        
        # Adjust column widths
        ws.column_dimensions['A'].width = 8
        ws.column_dimensions['B'].width = 12
        ws.column_dimensions['C'].width = 30
        ws.column_dimensions['D'].width = 16
        ws.column_dimensions['E'].width = 16
        ws.column_dimensions['F'].width = 16
        ws.column_dimensions['G'].width = 12
    
    # Save workbook
    wb.save(output_file)
    
    print(f"\n✓ Excel report saved to: {output_file}")
    print(f"  - Sales Rep Summary sheet with MTD/YTD totals by sales rep")
    for rep in sales_reps:
        print(f"  - {rep} sheet with rep totals and Top 10 MTD/YTD customers")
    
    return results


# Run the function
results = generate_sales_rep_performance(
    sales_file="sales detail 2024-2025.xls",
    customer_list_file="SalesCustomerList_with_Channel.xlsx",
    target_month=10,
    target_year=2025,
    output_file="Sales_Rep_Performance.xlsx"
)

# Display Sales Rep Summary
print("\n" + "=" * 70)
print("SALES REP MTD/YTD SUMMARY")
print("=" * 70)
sales_rep_df = results['sales_rep_metrics']
print(sales_rep_df.to_string(index=False))

STEP 1: Reading and parsing sales data...
Parsed 55468 transaction rows
Unique customers: 351

STEP 2: Joining with customer list to get Sales Rep...
Found Sales Rep column: 'Sales Rep'
Found Channel column: 'Channel'

Sales Rep distribution in sales:
  A: 124 customers
  DAN: 1 customers
  E: 49 customers
  O: 82 customers
  RM: 6 customers
  S: 66 customers
  T: 5 customers
  Unassigned: 18 customers

STEP 3: Calculating MTD and YTD metrics by customer...
Total customers with metrics: 343

STEP 4: Calculating Sales Rep-Level MTD/YTD Totals...

Sales Rep             Customers        2025 MTD        2024 MTD      MTD %        2025 YTD        2024 YTD      YTD %
----------------------------------------------------------------------------------------------------------------------------------
RM                            6 $      230,450 $      219,129     105.2% $    2,057,132 $    2,091,730      98.3%
O                            81 $      169,872 $      129,104     131.6% $    1,512,4

In [9]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
import matplotlib.pyplot as plt
import numpy as np
import io
from openpyxl.drawing.image import Image as XLImage

def generate_sales_rep_performance(
    sales_file: str = "sales detail 2024-2025.xls",
    customer_list_file: str = "SalesCustomerList_with_Channel.xlsx",
    target_month: int = 10,
    target_year: int = 2025,
    output_file: str = "Sales_Rep_Performance.xlsx"
) -> dict:
    """
    Generate MTD/YTD performance reports for each Sales Rep.
    
    Args:
        sales_file: Path to the sales details Excel file
        customer_list_file: Path to the SalesCustomerList_with_Channel.xlsx file
        target_month: Month for MTD calculation
        target_year: Year for calculations
        output_file: Path to save the Excel report
    
    Returns:
        Dictionary with sales rep performance data including:
        - sales_rep_metrics: DataFrame with MTD/YTD totals by sales rep
        - grand_total: Overall totals
    """
    prev_year = target_year - 1
    
    # ============================================================
    # STEP 1: Parse sales data
    # ============================================================
    print("=" * 70)
    print("STEP 1: Reading and parsing sales data...")
    print("=" * 70)
    
    df_raw = pd.read_excel(sales_file, header=None)
    
    col_period = 1
    col_sales_amount = 17
    
    data_rows = []
    current_customer = None
    current_customer_name = None
    
    for idx in range(11, len(df_raw)):
        row = df_raw.iloc[idx]
        first_cell = row.iloc[0]
        
        if pd.isna(first_cell):
            continue
        
        first_cell_str = str(first_cell).strip()
        
        if first_cell_str and not first_cell_str.isdigit() and not first_cell_str.startswith('Item'):
            if any(c.isalpha() for c in first_cell_str) and any(c.isdigit() for c in first_cell_str):
                current_customer = first_cell_str
                cust_name = row.iloc[7] if pd.notna(row.iloc[7]) else ''
                current_customer_name = str(cust_name).strip() if cust_name else current_customer
                continue
        
        if first_cell_str.isdigit() and len(first_cell_str) == 4:
            year = int(first_cell_str)
            
            period_cell = row.iloc[col_period]
            month = None
            if pd.notna(period_cell):
                try:
                    month = int(float(period_cell))
                except (ValueError, TypeError):
                    month = None
            
            sales_amount = row.iloc[col_sales_amount]
            
            if pd.notna(sales_amount) and current_customer and month is not None:
                try:
                    sales_value = float(sales_amount)
                    data_rows.append({
                        'Customer #': current_customer,
                        'Customer Name': current_customer_name,
                        'Year': year,
                        'Month': month,
                        'Sales Amount': sales_value
                    })
                except (ValueError, TypeError):
                    pass
    
    df_sales = pd.DataFrame(data_rows)
    
    if df_sales.empty:
        print("No sales data found!")
        return {}
    
    print(f"Parsed {len(df_sales)} transaction rows")
    print(f"Unique customers: {df_sales['Customer #'].nunique()}")
    
    # ============================================================
    # STEP 2: Join with customer list to get Sales Rep
    # ============================================================
    print("\n" + "=" * 70)
    print("STEP 2: Joining with customer list to get Sales Rep...")
    print("=" * 70)
    
    customer_df = pd.read_excel(customer_list_file)
    
    # Find Sales Rep column
    sales_rep_col = None
    for col in customer_df.columns:
        if 'sales' in col.lower() and 'rep' in col.lower():
            sales_rep_col = col
            break
    
    if not sales_rep_col:
        print("ERROR: Sales Rep column not found in customer list!")
        return {}
    
    print(f"Found Sales Rep column: '{sales_rep_col}'")
    
    # Create lookup for customer -> sales rep
    customer_lookup = customer_df[['Customer #', sales_rep_col]].drop_duplicates(subset=['Customer #'])
    
    # Merge to add sales rep to sales data
    df_sales = df_sales.merge(customer_lookup, on='Customer #', how='left')
    
    # Fill missing sales reps with 'Unassigned'
    df_sales[sales_rep_col] = df_sales[sales_rep_col].fillna('Unassigned')
    
    print(f"\nSales Rep distribution in sales:")
    rep_counts = df_sales.groupby(sales_rep_col)['Customer #'].nunique()
    for rep, count in rep_counts.items():
        print(f"  {rep}: {count} customers")
    
    # ============================================================
    # STEP 3: Calculate MTD and YTD metrics by Sales Rep
    # ============================================================
    print("\n" + "=" * 70)
    print("STEP 3: Calculating MTD and YTD metrics by Sales Rep...")
    print("=" * 70)
    
    # Filter for target years
    df_current = df_sales[df_sales['Year'] == target_year].copy()
    df_prev = df_sales[df_sales['Year'] == prev_year].copy()
    
    # Calculate MTD (specific month only)
    df_mtd_current = df_current[df_current['Month'] == target_month]
    df_mtd_prev = df_prev[df_prev['Month'] == target_month]
    
    # Calculate YTD (months 1 to target_month)
    df_ytd_current = df_current[df_current['Month'] <= target_month]
    df_ytd_prev = df_prev[df_prev['Month'] <= target_month]
    
    # Aggregate by Sales Rep
    def aggregate_by_rep(df, sales_rep_col):
        return df.groupby(sales_rep_col).agg({
            'Sales Amount': 'sum',
            'Customer #': 'nunique'
        }).reset_index()
    
    mtd_current_agg = aggregate_by_rep(df_mtd_current, sales_rep_col)
    mtd_prev_agg = aggregate_by_rep(df_mtd_prev, sales_rep_col)
    ytd_current_agg = aggregate_by_rep(df_ytd_current, sales_rep_col)
    ytd_prev_agg = aggregate_by_rep(df_ytd_prev, sales_rep_col)
    
    # Rename columns
    mtd_current_agg = mtd_current_agg.rename(columns={
        'Sales Amount': f'{target_year}_MTD_Sales',
        'Customer #': 'Customer_Count'
    })
    mtd_prev_agg = mtd_prev_agg.rename(columns={
        'Sales Amount': f'{prev_year}_MTD_Sales',
        'Customer #': 'Customer_Count_Prev'
    })
    ytd_current_agg = ytd_current_agg.rename(columns={
        'Sales Amount': f'{target_year}_YTD_Sales',
        'Customer #': 'Customer_Count_YTD'
    })
    ytd_prev_agg = ytd_prev_agg.rename(columns={
        'Sales Amount': f'{prev_year}_YTD_Sales',
        'Customer #': 'Customer_Count_YTD_Prev'
    })
    
    # Merge all metrics
    sales_rep_metrics = mtd_current_agg[[sales_rep_col, f'{target_year}_MTD_Sales', 'Customer_Count']].copy()
    
    sales_rep_metrics = sales_rep_metrics.merge(
        mtd_prev_agg[[sales_rep_col, f'{prev_year}_MTD_Sales']],
        on=sales_rep_col, how='outer'
    )
    sales_rep_metrics = sales_rep_metrics.merge(
        ytd_current_agg[[sales_rep_col, f'{target_year}_YTD_Sales']],
        on=sales_rep_col, how='outer'
    )
    sales_rep_metrics = sales_rep_metrics.merge(
        ytd_prev_agg[[sales_rep_col, f'{prev_year}_YTD_Sales']],
        on=sales_rep_col, how='outer'
    )
    
    # Fill NaN with 0
    for col in [f'{target_year}_MTD_Sales', f'{prev_year}_MTD_Sales', 
                f'{target_year}_YTD_Sales', f'{prev_year}_YTD_Sales', 'Customer_Count']:
        if col in sales_rep_metrics.columns:
            sales_rep_metrics[col] = sales_rep_metrics[col].fillna(0)
    
    # Calculate % Achieved
    sales_rep_metrics['MTD_Achieved_%'] = sales_rep_metrics.apply(
        lambda x: (x[f'{target_year}_MTD_Sales'] / x[f'{prev_year}_MTD_Sales'] * 100) 
        if x[f'{prev_year}_MTD_Sales'] > 0 else 0, axis=1
    )
    sales_rep_metrics['YTD_Achieved_%'] = sales_rep_metrics.apply(
        lambda x: (x[f'{target_year}_YTD_Sales'] / x[f'{prev_year}_YTD_Sales'] * 100) 
        if x[f'{prev_year}_YTD_Sales'] > 0 else 0, axis=1
    )
    
    # Calculate YoY Growth
    sales_rep_metrics['MTD_YoY_Growth'] = sales_rep_metrics.apply(
        lambda x: ((x[f'{target_year}_MTD_Sales'] - x[f'{prev_year}_MTD_Sales']) / x[f'{prev_year}_MTD_Sales'] * 100) 
        if x[f'{prev_year}_MTD_Sales'] > 0 else 0, axis=1
    )
    sales_rep_metrics['YTD_YoY_Growth'] = sales_rep_metrics.apply(
        lambda x: ((x[f'{target_year}_YTD_Sales'] - x[f'{prev_year}_YTD_Sales']) / x[f'{prev_year}_YTD_Sales'] * 100) 
        if x[f'{prev_year}_YTD_Sales'] > 0 else 0, axis=1
    )
    
    # Sort by current year YTD sales descending
    sales_rep_metrics = sales_rep_metrics.sort_values(f'{target_year}_YTD_Sales', ascending=False)
    
    # Print Sales Rep Summary
    print(f"\n{'Sales Rep':<20} {'Customers':>10} {f'{target_year} MTD':>15} {f'{prev_year} MTD':>15} {'MTD %':>10} {f'{target_year} YTD':>15} {f'{prev_year} YTD':>15} {'YTD %':>10}")
    print("-" * 130)
    
    for _, row in sales_rep_metrics.iterrows():
        if row[sales_rep_col] != 'Unassigned':
            print(f"{str(row[sales_rep_col]):<20} {int(row['Customer_Count']):>10} "
                  f"${row[f'{target_year}_MTD_Sales']:>13,.0f} ${row[f'{prev_year}_MTD_Sales']:>13,.0f} "
                  f"{row['MTD_Achieved_%']:>9.1f}% "
                  f"${row[f'{target_year}_YTD_Sales']:>13,.0f} ${row[f'{prev_year}_YTD_Sales']:>13,.0f} "
                  f"{row['YTD_Achieved_%']:>9.1f}%")
    
    # Calculate Grand Total
    grand_total = {
        'Sales Rep': 'GRAND TOTAL',
        'Customer_Count': int(sales_rep_metrics['Customer_Count'].sum()),
        f'{target_year}_MTD_Sales': sales_rep_metrics[f'{target_year}_MTD_Sales'].sum(),
        f'{prev_year}_MTD_Sales': sales_rep_metrics[f'{prev_year}_MTD_Sales'].sum(),
        f'{target_year}_YTD_Sales': sales_rep_metrics[f'{target_year}_YTD_Sales'].sum(),
        f'{prev_year}_YTD_Sales': sales_rep_metrics[f'{prev_year}_YTD_Sales'].sum(),
    }
    grand_total['MTD_Achieved_%'] = (grand_total[f'{target_year}_MTD_Sales'] / grand_total[f'{prev_year}_MTD_Sales'] * 100) if grand_total[f'{prev_year}_MTD_Sales'] > 0 else 0
    grand_total['YTD_Achieved_%'] = (grand_total[f'{target_year}_YTD_Sales'] / grand_total[f'{prev_year}_YTD_Sales'] * 100) if grand_total[f'{prev_year}_YTD_Sales'] > 0 else 0
    
    print("-" * 130)
    print(f"{'GRAND TOTAL':<20} {grand_total['Customer_Count']:>10} "
          f"${grand_total[f'{target_year}_MTD_Sales']:>13,.0f} ${grand_total[f'{prev_year}_MTD_Sales']:>13,.0f} "
          f"{grand_total['MTD_Achieved_%']:>9.1f}% "
          f"${grand_total[f'{target_year}_YTD_Sales']:>13,.0f} ${grand_total[f'{prev_year}_YTD_Sales']:>13,.0f} "
          f"{grand_total['YTD_Achieved_%']:>9.1f}%")
    
    # ============================================================
    # STEP 4: Export to Excel with formatting
    # ============================================================
    print("\n" + "=" * 70)
    print("STEP 4: Creating Excel report...")
    print("=" * 70)
    
    wb = Workbook()
    ws = wb.active
    ws.title = "Sales Rep Summary"
    
    # Styling
    header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
    header_font = Font(bold=True, color="FFFFFF", size=10)
    border = Border(
        left=Side(style='thin'),
        right=Side(style='thin'),
        top=Side(style='thin'),
        bottom=Side(style='thin')
    )
    
    # Color fills
    current_year_fill = PatternFill(start_color="E2EFDA", end_color="E2EFDA", fill_type="solid")
    prev_year_fill = PatternFill(start_color="FCE4D6", end_color="FCE4D6", fill_type="solid")
    achieved_fill = PatternFill(start_color="FFF2CC", end_color="FFF2CC", fill_type="solid")
    total_fill = PatternFill(start_color="DDEBF7", end_color="DDEBF7", fill_type="solid")
    
    # Title
    ws.merge_cells('A1:J1')
    ws['A1'] = f"Sales Rep Performance Summary - MTD/YTD ({target_month}/{target_year})"
    ws['A1'].font = Font(bold=True, size=16)
    ws['A1'].alignment = Alignment(horizontal='center')
    
    # Headers
    headers = [
        'Sales Rep', 'Customers',
        f'{target_year} MTD', f'{prev_year} MTD', 'MTD %', 'MTD YoY Growth',
        f'{target_year} YTD', f'{prev_year} YTD', 'YTD %', 'YTD YoY Growth'
    ]
    
    for col_idx, header in enumerate(headers, start=1):
        cell = ws.cell(row=3, column=col_idx, value=header)
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
        cell.border = border
    
    # Write Sales Rep Data
    row_idx = 4
    for _, rep_row in sales_rep_metrics.iterrows():
        if rep_row[sales_rep_col] == 'Unassigned':
            continue
        
        ws.cell(row=row_idx, column=1, value=rep_row[sales_rep_col]).border = border
        ws.cell(row=row_idx, column=2, value=int(rep_row['Customer_Count'])).border = border
        
        # MTD Current Year
        cell = ws.cell(row=row_idx, column=3, value=rep_row[f'{target_year}_MTD_Sales'])
        cell.number_format = '$#,##0.00'
        cell.fill = current_year_fill
        cell.border = border
        
        # MTD Previous Year
        cell = ws.cell(row=row_idx, column=4, value=rep_row[f'{prev_year}_MTD_Sales'])
        cell.number_format = '$#,##0.00'
        cell.fill = prev_year_fill
        cell.border = border
        
        # MTD Achieved %
        cell = ws.cell(row=row_idx, column=5, value=rep_row['MTD_Achieved_%'] / 100)
        cell.number_format = '0.0%'
        cell.fill = achieved_fill
        cell.border = border
        
        # MTD YoY Growth
        cell = ws.cell(row=row_idx, column=6, value=rep_row['MTD_YoY_Growth'] / 100)
        cell.number_format = '+0.0%;-0.0%'
        cell.border = border
        
        # YTD Current Year
        cell = ws.cell(row=row_idx, column=7, value=rep_row[f'{target_year}_YTD_Sales'])
        cell.number_format = '$#,##0.00'
        cell.fill = current_year_fill
        cell.border = border
        
        # YTD Previous Year
        cell = ws.cell(row=row_idx, column=8, value=rep_row[f'{prev_year}_YTD_Sales'])
        cell.number_format = '$#,##0.00'
        cell.fill = prev_year_fill
        cell.border = border
        
        # YTD Achieved %
        cell = ws.cell(row=row_idx, column=9, value=rep_row['YTD_Achieved_%'] / 100)
        cell.number_format = '0.0%'
        cell.fill = achieved_fill
        cell.border = border
        
        # YTD YoY Growth
        cell = ws.cell(row=row_idx, column=10, value=rep_row['YTD_YoY_Growth'] / 100)
        cell.number_format = '+0.0%;-0.0%'
        cell.border = border
        
        row_idx += 1
    
    # Grand Total Row
    for col in range(1, 11):
        ws.cell(row=row_idx, column=col).fill = total_fill
        ws.cell(row=row_idx, column=col).border = border
        ws.cell(row=row_idx, column=col).font = Font(bold=True)
    
    ws.cell(row=row_idx, column=1, value='GRAND TOTAL')
    ws.cell(row=row_idx, column=2, value=grand_total['Customer_Count'])
    
    cell = ws.cell(row=row_idx, column=3, value=grand_total[f'{target_year}_MTD_Sales'])
    cell.number_format = '$#,##0.00'
    
    cell = ws.cell(row=row_idx, column=4, value=grand_total[f'{prev_year}_MTD_Sales'])
    cell.number_format = '$#,##0.00'
    
    cell = ws.cell(row=row_idx, column=5, value=grand_total['MTD_Achieved_%'] / 100)
    cell.number_format = '0.0%'
    
    mtd_growth = ((grand_total[f'{target_year}_MTD_Sales'] - grand_total[f'{prev_year}_MTD_Sales']) / grand_total[f'{prev_year}_MTD_Sales'] * 100) if grand_total[f'{prev_year}_MTD_Sales'] > 0 else 0
    cell = ws.cell(row=row_idx, column=6, value=mtd_growth / 100)
    cell.number_format = '+0.0%;-0.0%'
    
    cell = ws.cell(row=row_idx, column=7, value=grand_total[f'{target_year}_YTD_Sales'])
    cell.number_format = '$#,##0.00'
    
    cell = ws.cell(row=row_idx, column=8, value=grand_total[f'{prev_year}_YTD_Sales'])
    cell.number_format = '$#,##0.00'
    
    cell = ws.cell(row=row_idx, column=9, value=grand_total['YTD_Achieved_%'] / 100)
    cell.number_format = '0.0%'
    
    ytd_growth = ((grand_total[f'{target_year}_YTD_Sales'] - grand_total[f'{prev_year}_YTD_Sales']) / grand_total[f'{prev_year}_YTD_Sales'] * 100) if grand_total[f'{prev_year}_YTD_Sales'] > 0 else 0
    cell = ws.cell(row=row_idx, column=10, value=ytd_growth / 100)
    cell.number_format = '+0.0%;-0.0%'
    
    # Adjust column widths
    ws.column_dimensions['A'].width = 20
    ws.column_dimensions['B'].width = 12
    ws.column_dimensions['C'].width = 16
    ws.column_dimensions['D'].width = 16
    ws.column_dimensions['E'].width = 10
    ws.column_dimensions['F'].width = 14
    ws.column_dimensions['G'].width = 16
    ws.column_dimensions['H'].width = 16
    ws.column_dimensions['I'].width = 10
    ws.column_dimensions['J'].width = 14
    
    ws.row_dimensions[3].height = 30
    
    # ============================================================
    # Add Charts
    # ============================================================
    chart_start_row = row_idx + 3
    
    # Filter out Unassigned for charts
    chart_data = sales_rep_metrics[sales_rep_metrics[sales_rep_col] != 'Unassigned'].copy()
    chart_reps = chart_data[sales_rep_col].tolist()
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f'Sales Rep Performance Analysis - {target_month}/{target_year}', fontsize=14, fontweight='bold')
    
    x = np.arange(len(chart_reps))
    width = 0.35
    
    # Chart 1: MTD Sales by Sales Rep
    axes[0, 0].bar(x - width/2, chart_data[f'{target_year}_MTD_Sales'], width, 
                   label=str(target_year), color='#2E86AB', alpha=0.8)
    axes[0, 0].bar(x + width/2, chart_data[f'{prev_year}_MTD_Sales'], width, 
                   label=str(prev_year), color='#A23B72', alpha=0.8)
    axes[0, 0].set_title('MTD Sales by Sales Rep', fontsize=11, fontweight='bold')
    axes[0, 0].set_ylabel('Sales Amount ($)')
    axes[0, 0].set_xticks(x)
    axes[0, 0].set_xticklabels(chart_reps, rotation=45, ha='right', fontsize=9)
    axes[0, 0].legend(fontsize=9)
    axes[0, 0].yaxis.set_major_formatter(plt.FuncFormatter(lambda v, p: f'${v/1000:.0f}K'))
    axes[0, 0].grid(axis='y', alpha=0.3)
    
    # Chart 2: YTD Sales by Sales Rep
    axes[0, 1].bar(x - width/2, chart_data[f'{target_year}_YTD_Sales'], width, 
                   label=str(target_year), color='#2E86AB', alpha=0.8)
    axes[0, 1].bar(x + width/2, chart_data[f'{prev_year}_YTD_Sales'], width, 
                   label=str(prev_year), color='#A23B72', alpha=0.8)
    axes[0, 1].set_title('YTD Sales by Sales Rep', fontsize=11, fontweight='bold')
    axes[0, 1].set_ylabel('Sales Amount ($)')
    axes[0, 1].set_xticks(x)
    axes[0, 1].set_xticklabels(chart_reps, rotation=45, ha='right', fontsize=9)
    axes[0, 1].legend(fontsize=9)
    axes[0, 1].yaxis.set_major_formatter(plt.FuncFormatter(lambda v, p: f'${v/1000:.0f}K'))
    axes[0, 1].grid(axis='y', alpha=0.3)
    
    # Chart 3: MTD % Achieved by Sales Rep
    colors = ['#28a745' if v >= 100 else '#dc3545' for v in chart_data['MTD_Achieved_%']]
    axes[1, 0].barh(chart_reps, chart_data['MTD_Achieved_%'], color=colors, alpha=0.8)
    axes[1, 0].axvline(x=100, color='black', linestyle='--', linewidth=1)
    axes[1, 0].set_title('MTD % Achieved by Sales Rep', fontsize=11, fontweight='bold')
    axes[1, 0].set_xlabel('% Achieved')
    for i, v in enumerate(chart_data['MTD_Achieved_%']):
        axes[1, 0].text(v + 2, i, f'{v:.1f}%', va='center', fontsize=9)
    axes[1, 0].grid(axis='x', alpha=0.3)
    
    # Chart 4: YTD % Achieved by Sales Rep
    colors = ['#28a745' if v >= 100 else '#dc3545' for v in chart_data['YTD_Achieved_%']]
    axes[1, 1].barh(chart_reps, chart_data['YTD_Achieved_%'], color=colors, alpha=0.8)
    axes[1, 1].axvline(x=100, color='black', linestyle='--', linewidth=1)
    axes[1, 1].set_title('YTD % Achieved by Sales Rep', fontsize=11, fontweight='bold')
    axes[1, 1].set_xlabel('% Achieved')
    for i, v in enumerate(chart_data['YTD_Achieved_%']):
        axes[1, 1].text(v + 2, i, f'{v:.1f}%', va='center', fontsize=9)
    axes[1, 1].grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    
    # Save chart to buffer
    img_buffer = io.BytesIO()
    fig.savefig(img_buffer, format='png', dpi=150, bbox_inches='tight')
    img_buffer.seek(0)
    plt.close(fig)
    
    img = XLImage(img_buffer)
    ws.add_image(img, f'A{chart_start_row}')
    
    # Save workbook
    wb.save(output_file)
    
    print(f"\n✓ Excel report saved to: {output_file}")
    
    results = {
        'sales_rep_metrics': sales_rep_metrics,
        'grand_total': grand_total,
        'sales_rep_col': sales_rep_col
    }
    
    return results


# Run the function
results = generate_sales_rep_performance(
    sales_file="sales detail 2024-2025.xls",
    customer_list_file="SalesCustomerList_with_Channel.xlsx",
    target_month=10,
    target_year=2025,
    output_file="Sales_Rep_Performance.xlsx"
)

# Display Sales Rep Summary
print("\n" + "=" * 70)
print("SALES REP MTD/YTD SUMMARY")
print("=" * 70)
if results:
    sales_rep_df = results['sales_rep_metrics']
    print(sales_rep_df.to_string(index=False))

STEP 1: Reading and parsing sales data...
Parsed 55468 transaction rows
Unique customers: 351

STEP 2: Joining with customer list to get Sales Rep...
Found Sales Rep column: 'Sales Rep'

Sales Rep distribution in sales:
  A: 124 customers
  DAN: 1 customers
  E: 49 customers
  O: 82 customers
  RM: 6 customers
  S: 66 customers
  T: 5 customers
  Unassigned: 18 customers

STEP 3: Calculating MTD and YTD metrics by Sales Rep...

Sales Rep             Customers        2025 MTD        2024 MTD      MTD %        2025 YTD        2024 YTD      YTD %
----------------------------------------------------------------------------------------------------------------------------------
RM                            5 $      230,450 $      219,129     105.2% $    2,057,132 $    2,091,730      98.3%
O                            35 $      169,872 $      129,104     131.6% $    1,512,485 $    1,642,443      92.1%
S                            31 $      102,713 $      116,912      87.9% $    1,195,544 $  